# Introduction

In this assignment, you will construct a lemmatizer for English verbs using weighted finite state transducers (wFSTs), using `wfst4str`, a Python library build on top of the `rustfst` library (a Rust port of the well-known OpenFST library).

## Pedagogical Goals

* Understand an wFST as an instance of a graphical model
* Identify the morphotactic patterns in a set of lexical data and represent them using an FST
* Identify the allomorphic patterns in a set of lexical data and represent them using an FST
* Reason about basic operations on wFSTs
* Reason about weights in a wFST and their interaction with operations

## Evaluation

You will be evaluated on three validation data sets and three held-out data sets:

- In vocabulary
- Out of vocabulary but attested ("real" words that are not in the lexicon)
- Nonce (invented words)

You will receive full credit if your scores are above:

- 98% (in vocabulary)
- 65% (out of vocabulary)
- 75% (nonce)

For both the validation set and the held out test set.

## Important hints and tricks
- You will only be evaluated on your wfsts, so you can add any assert tests/ code for debugging as per your requirement.
- Do not remove any test cases. It would be difficult to understand why you're failing
- `wfst4str` library has a lot of useful methods which have been talked about the handout
- Do not change the commands of building the wfst. You may edit only the tests, and the wfst structure itself
- Make sure there are no trailing or leading white spaces in everyline of your wfst in the At&T repreesentation

# Imports and definitions

Here, we import `wfst4str` and `os.path`, which will be used throughout the exercise, and define some consonants (lists of single-character symbols).

In [ ]:
%%capture
!pip install wfst4str

In [ ]:
import wfst4str

SYMS = list("'\")*-./3;@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzäæéëïôöœ‘’“”^#") + ['<oth>', '<c>', '<v>', '<g>']
VOWELS = list('aeiou')
GLIDES = list('yw')
CONSONANTS = list('bcdfghjklmnpqrstvxz')

## Preprocessing
This FST inserts a word boundary (`#`) at the end of each input.

In [ ]:
#DO NOT CHANGE THIS

preproc = '''
0 0 <oth> <oth> 0.0
0 1 <eps> # 0.0
1 0.0
'''

preproc_fst = wfst4str.WeightedFst()
preproc_fst.set_input_symbols(SYMS)
preproc_fst.set_output_symbols(SYMS)
preproc_fst.add_state()
preproc_fst.set_start(0)
preproc_fst.populate_from_att(preproc)
preproc_fst.explode_oth({'#', '^'})


assert preproc_fst.apply('eat') == {'eat#'}
assert preproc_fst.apply('zoke') == {'zoke#'}
assert preproc_fst.apply('fly') == {'fly#'}

## Postprocessing
This wFST deteles morpheme boundaries (`^`) and word boundaries (`#`) from strings.

In [ ]:
# DO NOT CHANGE THIS

postproc = '''
0 0 <oth> <oth> 0.0
0 0 ^ <eps> 0.0
0 1 # <eps> 0.0
1 0.0
'''

postproc_fst = wfst4str.WeightedFst()
postproc_fst.set_input_symbols(SYMS)
postproc_fst.set_output_symbols(SYMS)
postproc_fst.add_state()
postproc_fst.set_start(0)
postproc_fst.populate_from_att(postproc)
postproc_fst.explode_oth({'#', '^'})
# postproc_fst = postproc_fst.determinize()

assert postproc_fst.best_output('zok^ed#') == 'zoked'
assert postproc_fst.best_output('eat^ing#') == 'eating'


In [ ]:
#@title data (this is VERY long - make sure to hide this by double clicking the right half of the cell!!)

CARNEGLISH = '''andrew	andrewed
andrew	andrews
andrew	andrewing
andrew	andrew
tiss	tisses
tiss	tissing
tiss	tissed
tiss	tiss
exmogrify	exmogrified
exmogrify	exmogrifying
exmogrify	exmogrifies
exmogrify	exmogrify
blognicate	blognicate
blognicate	blognicates
blognicate	blognicating
blognicate	blognicated
andrew	andrewed
andrew	andrews
andrew	andrewing
andrew	andrew
squig	squigging
squig	squigged
squig	squigs
squig	squig
squig	squigging
squig	squigged
squig	squigs
squig	squig
felex	felex
felex	felexes
felex	felexing
felex	felexed
ree	ree
ree	rees
ree	reeing
ree	reed
splankle	splankling
splankle	splankled
splankle	splankles
splankle	splankle
footh	footh
footh	fooths
footh	foothing
footh	foothed
kurry	kurry
kurry	kurries
kurry	kurrying
kurry	kurried'''

INVOCAB = '''abandon	abandoned
abandon	abandoned
abandon	abandoning
abandon	abandons
abandon	abandon
able	abled
able	abled
able	ables
able	able
able	abling
absent	absented
absent	absented
absent	absenting
absent	absents
absent	absent
abstract	abstracted
abstract	abstracted
abstract	abstracting
abstract	abstracts
abstract	abstract
abuse	abused
abuse	abused
abuse	abuses
abuse	abuse
abuse	abusing
accent	accented
accent	accented
accent	accenting
accent	accents
accent	accent
accept	accepted
accept	accepted
accept	accepting
accept	accepts
accept	accept
access	accessed
access	accessed
access	accesses
access	accessing
access	access
accession	accessioned
accession	accessioned
accession	accessioning
accession	accessions
accession	accession
acclaim	acclaimed
acclaim	acclaimed
acclaim	acclaiming
acclaim	acclaims
acclaim	acclaim
accommodate	accommodated
accommodate	accommodated
accommodate	accommodates
accommodate	accommodate
accommodate	accommodating
accompany	accompanied
accompany	accompanied
accompany	accompanies
accompany	accompanying
accompany	accompany
account	accounted
account	accounted
account	accounting
account	accounts
account	account
ace	aced
ace	aced
ace	aces
ace	ace
ace	acing
achieve	achieved
achieve	achieved
achieve	achieves
achieve	achieve
achieve	achieving
acquire	acquired
acquire	acquired
acquire	acquires
acquire	acquire
acquire	acquiring
act	acted
act	acted
act	acting
act	acts
act	act
action	actioned
action	actioned
action	actioning
action	actions
action	action
acute	acuted
acute	acuted
acute	acutes
acute	acute
acute	acuting
adapt	adapted
adapt	adapted
adapt	adapting
adapt	adapts
adapt	adapt
add	added
add	added
add	adding
add	adds
add	add
address	addressed
address	addressed
address	addresses
address	addressing
address	address
address	addrest
address	addrest
adequate	adequated
adequate	adequated
adequate	adequates
adequate	adequate
adequate	adequating
adopt	adopted
adopt	adopted
adopt	adopting
adopt	adopts
adopt	adopt
adult	adulted
adult	adulted
adult	adulting
adult	adults
adult	adult
advance	advanced
advance	advanced
advance	advances
advance	advance
advance	advancing
advantage	advantaged
advantage	advantaged
advantage	advantages
advantage	advantage
advantage	advantaging
adventure	adventured
adventure	adventured
adventure	adventures
adventure	adventure
adventure	adventuring
advocate	advocated
advocate	advocated
advocate	advocates
advocate	advocate
advocate	advocating
affect	affected
affect	affected
affect	affecting
affect	affects
affect	affect
affiliate	affiliated
affiliate	affiliated
affiliate	affiliates
affiliate	affiliate
affiliate	affiliating
afford	afforded
afford	afforded
afford	affording
afford	affords
afford	afford
age	aged
age	aged
age	ageing
age	ages
age	age
age	aging
aggregate	aggregated
aggregate	aggregated
aggregate	aggregates
aggregate	aggregate
aggregate	aggregating
agree	agreed
agree	agreed
agree	agreeing
agree	agrees
agree	agree
aid	aided
aid	aided
aid	aiding
aid	aids
aid	aid
aim	aimed
aim	aimed
aim	aiming
aim	aims
aim	aim
air	aired
air	aired
air	airing
air	airs
air	air
alert	alerted
alert	alerted
alert	alerting
alert	alerts
alert	alert
alias	aliased
alias	aliased
alias	aliases
alias	aliasing
alias	alias
alien	aliened
alien	aliened
alien	aliening
alien	aliens
alien	alien
align	aligned
align	aligned
align	aligning
align	aligns
align	align
alliance	allianced
alliance	allianced
alliance	alliances
alliance	alliance
alliance	alliancing
allow	allowed
allow	allowed
allow	allowing
allow	allows
allow	allow
ally	allied
ally	allied
ally	allies
ally	allying
ally	ally
alphabet	alphabeted
alphabet	alphabeted
alphabet	alphabeting
alphabet	alphabets
alphabet	alphabet
alter	altered
alter	altered
alter	altering
alter	alters
alter	alter
alternate	alternated
alternate	alternated
alternate	alternates
alternate	alternate
alternate	alternating
amber	ambered
amber	ambered
amber	ambering
amber	ambers
amber	amber
amnesty	amnestied
amnesty	amnestied
amnesty	amnesties
amnesty	amnestying
amnesty	amnesty
amount	amounted
amount	amounted
amount	amounting
amount	amounts
amount	amount
amp	amped
amp	amped
amp	amping
amp	amps
amp	amp
anchor	anchored
anchor	anchored
anchor	anchoring
anchor	anchors
anchor	anchor
and	anded
and	anded
and	anding
and	ands
and	and
angel	angeled
angel	angeled
angel	angeling
angel	angels
angel	angel
anger	angered
anger	angered
anger	angering
anger	angers
anger	anger
angle	angled
angle	angled
angle	angles
angle	angle
angle	angling
answer	answered
answer	answered
answer	answering
answer	answers
answer	answer
anthem	anthemed
anthem	anthemed
anthem	antheming
anthem	anthems
anthem	anthem
appeal	appealed
appeal	appealed
appeal	appealing
appeal	appeals
appeal	appeal
appear	appeared
appear	appeared
appear	appearing
appear	appears
appear	appear
apply	applied
apply	applied
apply	applies
apply	applying
apply	apply
appoint	appointed
appoint	appointed
appoint	appointing
appoint	appoints
appoint	appoint
apprentice	apprenticed
apprentice	apprenticed
apprentice	apprentices
apprentice	apprentice
apprentice	apprenticing
approach	approached
approach	approached
approach	approaches
approach	approaching
approach	approach
appropriate	appropriated
appropriate	appropriated
appropriate	appropriates
appropriate	appropriate
appropriate	appropriating
approximate	approximated
approximate	approximated
approximate	approximates
approximate	approximate
approximate	approximating
arc	arced
arc	arced
arc	arcing
arc	arcked
arc	arcked
arc	arcking
arc	arcs
arc	arc
arch	arched
arch	arched
arch	arches
arch	arching
arch	arch
architect	architected
architect	architected
architect	architecting
architect	architects
architect	architect
archive	archived
archive	archived
archive	archives
archive	archive
archive	archiving
argue	argued
argue	argued
argue	argues
argue	argue
argue	arguing
arise	arisen
arise	arises
arise	arise
arise	arising
arise	arose
arm	armed
arm	armed
arm	arming
arm	arms
arm	arm
armor	armored
armor	armored
armor	armoring
armor	armors
armor	armor
armour	armoured
armour	armoured
armour	armouring
armour	armours
armour	armour
array	arrayed
array	arrayed
array	arraying
array	arrays
array	array
arrest	arrested
arrest	arrested
arrest	arresting
arrest	arrests
arrest	arrest
arrive	arrived
arrive	arrived
arrive	arrives
arrive	arrive
arrive	arriving
arrow	arrowed
arrow	arrowed
arrow	arrowing
arrow	arrows
arrow	arrow
article	articled
article	articled
article	articles
article	article
article	articling
ash	ashed
ash	ashed
ash	ashes
ash	ashing
ash	ash
ask	asked
ask	asked
ask	asking
ask	asks
ask	ask
assault	assaulted
assault	assaulted
assault	assaulting
assault	assaults
assault	assault
assess	assessed
assess	assessed
assess	assesses
assess	assessing
assess	assess
assist	assisted
assist	assisted
assist	assisting
assist	assists
assist	assist
associate	associated
associate	associated
associate	associates
associate	associate
associate	associating
assume	assumed
assume	assumed
assume	assumes
assume	assume
assume	assuming
attack	attacked
attack	attacked
attack	attacking
attack	attacks
attack	attack
attempt	attempted
attempt	attempted
attempt	attempting
attempt	attempts
attempt	attempt
attend	attended
attend	attended
attend	attending
attend	attends
attend	attend
attitude	attituded
attitude	attituded
attitude	attitudes
attitude	attitude
attitude	attituding
attract	attracted
attract	attracted
attract	attracting
attract	attracts
attract	attract
auction	auctioned
auction	auctioned
auction	auctioning
auction	auctions
auction	auction
audit	audited
audit	audited
audit	auditing
audit	audits
audit	audit
audition	auditioned
audition	auditioned
audition	auditioning
audition	auditions
audition	audition
august	augusted
august	augusted
august	augusting
august	augusts
august	august
author	authored
author	authored
author	authoring
author	authors
author	author
automobile	automobiled
automobile	automobiled
automobile	automobiles
automobile	automobile
automobile	automobiling
average	averaged
average	averaged
average	averages
average	average
average	averaging
avoid	avoided
avoid	avoided
avoid	avoiding
avoid	avoids
avoid	avoid
award	awarded
award	awarded
award	awarding
award	awards
award	award
ba	baed
ba	baed
ba	baing
ba	bas
ba	ba
baby	babied
baby	babied
baby	babies
baby	babying
baby	baby
bach	bached
bach	bached
bach	baches
bach	baching
bach	bach
back	backed
back	backed
back	backing
back	backs
back	back
background	backgrounded
background	backgrounded
background	backgrounding
background	backgrounds
background	background
bad	badded
bad	badded
bad	badding
bad	bads
bad	bad
badge	badged
badge	badged
badge	badges
badge	badge
badge	badging
bag	bagged
bag	bagged
bag	bagging
bag	bags
bag	bag
balance	balanced
balance	balanced
balance	balances
balance	balance
balance	balancing
ballad	balladed
ballad	balladed
ballad	ballading
ballad	ballads
ballad	ballad
ball	balled
ball	balled
ball	balling
ball	balls
ball	ball
balloon	ballooned
balloon	ballooned
balloon	ballooning
balloon	balloons
balloon	balloon
ballot	balloted
ballot	balloted
ballot	balloting
ballot	ballots
ballot	ballot
ban	banned
ban	banned
ban	banning
ban	bans
ban	ban
band	banded
band	banded
band	banding
band	bands
band	band
bang	banged
bang	banged
bang	banging
bang	bangs
bang	bang
bank	banked
bank	banked
bank	banking
bank	banks
bank	bank
bar	barred
bar	barred
bar	barring
bar	bars
bar	bar
barber	barbered
barber	barbered
barber	barbering
barber	barbers
barber	barber
bare	bared
bare	bared
bare	bares
bare	bare
bare	baring
bark	barked
bark	barked
bark	barking
bark	barks
bark	bark
barn	barned
barn	barned
barn	barning
barn	barns
barn	barn
barrel	barreled
barrel	barreled
barrel	barreling
barrel	barrelled
barrel	barrelled
barrel	barrelling
barrel	barrels
barrel	barrel
base	based
base	based
base	bases
base	base
base	basing
basket	basketed
basket	basketed
basket	basketing
basket	baskets
basket	basket
bass	bassed
bass	bassed
bass	basses
bass	bassing
bass	bass
bat	bats
bat	batted
bat	batted
bat	batting
bat	bat
batch	batched
batch	batched
batch	batches
batch	batching
batch	batch
bath	bathed
bath	bathed
bath	bathing
bath	baths
bath	bath
battalion	battalioned
battalion	battalioned
battalion	battalioning
battalion	battalions
battalion	battalion
battle	battled
battle	battled
battle	battles
battle	battle
battle	battling
bay	bayed
bay	bayed
bay	baying
bay	bays
bay	bay
beach	beached
beach	beached
beach	beaches
beach	beaching
beach	beach
beam	beamed
beam	beamed
beam	beaming
beam	beams
beam	beam
bear	bare
bear	beared
bear	beared
bear	bearing
bear	bears
bear	bear
bear	bore
bear	borne
bear	born
beast	beasted
beast	beasted
beast	beasting
beast	beasts
beast	beast
beat	beaten
beat	beating
beat	beats
beat	beat
beat	beat
beat	beat
be	been
beck	becked
beck	becked
beck	becking
beck	becks
beck	beck
become	became
become	becomed
become	becomes
become	become
become	become
become	becoming
bed	bedded
bed	bedded
bed	bedding
bed	beds
bed	bed
beef	beefed
beef	beefed
beef	beefing
beef	beefs
beef	beef
beer	beered
beer	beered
beer	beering
beer	beers
beer	beer
beetle	beetled
beetle	beetled
beetle	beetles
beetle	beetle
beetle	beetling
begin	began
begin	beginning
begin	begins
begin	begin
begin	begun
believe	believed
believe	believed
believe	believes
believe	believe
believe	believing
bell	belled
bell	belled
bell	belling
bell	bells
bell	bell
belong	belonged
belong	belonged
belong	belonging
belong	belongs
belong	belong
belt	belted
belt	belted
belt	belting
belt	belts
belt	belt
bench	benched
bench	benched
bench	benches
bench	benching
bench	bench
bend	bended
bend	bended
bend	bending
bend	bends
bend	bend
bend	bent
bend	bent
benefit	benefited
benefit	benefited
benefit	benefiting
benefit	benefits
benefit	benefitted
benefit	benefitted
benefit	benefitting
benefit	benefit
berry	berried
berry	berried
berry	berries
berry	berrying
berry	berry
best	bested
best	bested
best	besting
best	bests
best	best
beta	betaed
beta	betaed
beta	betaing
beta	betas
beta	beta
bet	bets
bet	betted
bet	betted
bet	betting
bet	bet
bet	bet
bet	bet
better	bettered
better	bettered
better	bettering
better	betters
better	better
bias	biased
bias	biased
bias	biases
bias	biasing
bias	biassed
bias	biassed
bias	biasses
bias	biassing
bias	bias
bicycle	bicycled
bicycle	bicycled
bicycle	bicycles
bicycle	bicycle
bicycle	bicycling
bid	bade
bid	bidden
bid	bidding
bid	bids
bid	bid
bid	bid
bid	bid
big	bigged
big	bigged
big	bigging
big	bigs
big	big
bigger	biggered
bigger	biggered
bigger	biggering
bigger	biggers
bigger	bigger
bike	biked
bike	biked
bike	bikes
bike	bike
bike	biking
bill	billed
bill	billed
bill	billing
bill	bills
bill	bill
bin	binned
bin	binned
bin	binning
bin	bins
bin	bin
biography	biographied
biography	biographied
biography	biographies
biography	biographying
biography	biography
bird	birded
bird	birded
bird	birding
bird	birds
bird	bird
birth	birthed
birth	birthed
birth	birthing
birth	births
birth	birth
bishop	bishoped
bishop	bishoped
bishop	bishoping
bishop	bishopped
bishop	bishopped
bishop	bishopping
bishop	bishops
bishop	bishop
bit	bits
bit	bitted
bit	bitted
bit	bitting
bit	bit
bitter	bittered
bitter	bittered
bitter	bittering
bitter	bitters
bitter	bitter
black	blacked
black	blacked
black	blacking
black	blacks
black	black
blade	bladed
blade	bladed
blade	blades
blade	blade
blade	blading
blake	blaked
blake	blaked
blake	blakes
blake	blake
blake	blaking
blame	blamed
blame	blamed
blame	blames
blame	blame
blame	blaming
blast	blasted
blast	blasted
blast	blasting
blast	blasts
blast	blast
blend	blended
blend	blended
blend	blending
blend	blends
blend	blend
blend	blent
blend	blent
blind	blinded
blind	blinded
blind	blinding
blind	blinds
blind	blind
blockade	blockaded
blockade	blockaded
blockade	blockades
blockade	blockade
blockade	blockading
block	blocked
block	blocked
block	blocking
block	blocks
block	block
blog	blogged
blog	blogged
blog	blogging
blog	blogs
blog	blog
blood	blooded
blood	blooded
blood	blooding
blood	bloods
blood	blood
bloody	bloodied
bloody	bloodied
bloody	bloodies
bloody	bloodying
bloody	bloody
blow	blew
blow	blowing
blow	blown
blow	blows
blow	blow
blue	blued
blue	blued
blue	blueing
blue	blues
blue	blue
blue	bluing
board	boarded
board	boarded
board	boarding
board	boards
board	board
boat	boated
boat	boated
boat	boating
boat	boats
boat	boat
bob	bobbed
bob	bobbed
bob	bobbing
bob	bobs
bob	bob
body	bodied
body	bodied
body	bodies
body	bodying
body	body
bold	bolded
bold	bolded
bold	bolding
bold	bolds
bold	bold
bomb	bombed
bomb	bombed
bomb	bombing
bomb	bombs
bomb	bomb
bond	bonded
bond	bonded
bond	bonding
bond	bonds
bond	bond
bone	boned
bone	boned
bone	bones
bone	bone
bone	boning
bonus	bonused
bonus	bonused
bonus	bonuses
bonus	bonusing
bonus	bonus
book	booked
book	booked
book	booking
book	books
book	book
boom	boomed
boom	boomed
boom	booming
boom	booms
boom	boom
boost	boosted
boost	boosted
boost	boosting
boost	boosts
boost	boost
boot	booted
boot	booted
boot	booting
boot	boots
boot	boot
border	bordered
border	bordered
border	bordering
border	borders
border	border
bore	bored
bore	bored
bore	bores
bore	bore
bore	boring
born	bornin
born	borns
born	bornt
born	bornt
born	born
boss	bossed
boss	bossed
boss	bosses
boss	bossing
boss	boss
bottle	bottled
bottle	bottled
bottle	bottles
bottle	bottle
bottle	bottling
bottom	bottomed
bottom	bottomed
bottom	bottoming
bottom	bottoms
bottom	bottom
bound	bounded
bound	bounded
bound	bounding
bound	bounds
bound	bound
bout	bouted
bout	bouted
bout	bouting
bout	bouts
bout	bout
bow	bowed
bow	bowed
bow	bowing
bow	bows
bow	bow
bowl	bowled
bowl	bowled
bowl	bowling
bowl	bowls
bowl	bowl
box	boxed
box	boxed
box	boxes
box	boxing
box	box
boy	boyed
boy	boyed
boy	boying
boy	boys
boy	boy
brain	brained
brain	brained
brain	braining
brain	brains
brain	brain
brake	braked
brake	braked
brake	brakes
brake	brake
brake	braking
branch	branched
branch	branched
branch	branches
branch	branching
branch	branch
brand	branded
brand	branded
brand	branding
brand	brands
brand	brand
brave	braved
brave	braved
brave	braves
brave	brave
brave	braving
breach	breached
breach	breached
breach	breaches
breach	breaching
breach	breach
bread	breaded
bread	breaded
bread	breading
bread	breads
bread	bread
break	breaking
break	breaks
break	break
break	broken
break	broke
breakfast	breakfasted
breakfast	breakfasted
breakfast	breakfasting
breakfast	breakfasts
breakfast	breakfast
breast	breasted
breast	breasted
breast	breasting
breast	breasts
breast	breast
breed	bred
breed	bred
breed	breeding
breed	breeds
breed	breed
brian	brianed
brian	brianed
brian	brianing
brian	brians
brian	brian
brick	bricked
brick	bricked
brick	bricking
brick	bricks
brick	brick
bride	brided
bride	brided
bride	brides
bride	bride
bride	briding
bridge	bridged
bridge	bridged
bridge	bridges
bridge	bridge
bridge	bridging
brief	briefed
brief	briefed
brief	briefing
brief	briefs
brief	brief
brigade	brigaded
brigade	brigaded
brigade	brigades
brigade	brigade
brigade	brigading
bring	bringing
bring	brings
bring	bring
bring	brought
bring	brought
broadcast	broadcasted
broadcast	broadcasted
broadcast	broadcasting
broadcast	broadcasts
broadcast	broadcast
broadcast	broadcast
broadcast	broadcast
broke	broked
broke	broked
broke	brokes
broke	broke
broke	broking
bronze	bronzed
bronze	bronzed
bronze	bronzes
bronze	bronze
bronze	bronzing
brook	brooked
brook	brooked
brook	brooking
brook	brooks
brook	brook
brother	brothered
brother	brothered
brother	brothering
brother	brothers
brother	brother
brown	browned
brown	browned
brown	browning
brown	browns
brown	brown
buck	bucked
buck	bucked
buck	bucking
buck	bucks
buck	buck
buddy	buddied
buddy	buddied
buddy	buddies
buddy	buddying
buddy	buddy
budget	budgeted
budget	budgeted
budget	budgeting
budget	budgets
budget	budget
buffalo	buffaloed
buffalo	buffaloed
buffalo	buffaloes
buffalo	buffaloing
buffalo	buffalo
build	building
build	builds
build	build
build	built
build	built
bulk	bulked
bulk	bulked
bulk	bulking
bulk	bulks
bulk	bulk
bull	bulled
bull	bulled
bull	bulling
bull	bulls
bull	bull
bullet	bulleted
bullet	bulleted
bullet	bulleting
bullet	bullets
bullet	bullet
bulletin	bulletined
bulletin	bulletined
bulletin	bulletining
bulletin	bulletins
bulletin	bulletin
burden	burdened
burden	burdened
burden	burdening
burden	burdens
burden	burden
burke	burked
burke	burked
burke	burkes
burke	burke
burke	burking
burn	burned
burn	burned
burn	burning
burn	burns
burn	burnt
burn	burnt
burn	burn
burst	brast
burst	bursten
burst	bursting
burst	bursts
burst	burst
burst	burst
burst	burst
bury	buried
bury	buried
bury	buries
bury	burying
bury	bury
bus	bused
bus	bused
bus	buses
bus	busing
bus	bussed
bus	bussed
bus	busses
bus	bussing
bus	bus
bush	bushed
bush	bushed
bush	bushes
bush	bushing
bush	bush
busy	busied
busy	busied
busy	busies
busy	busying
busy	busy
butler	butlered
butler	butlered
butler	butlering
butler	butlers
butler	butler
butterfly	butterflied
butterfly	butterflied
butterfly	butterflies
butterfly	butterflying
butterfly	butterfly
button	buttoned
button	buttoned
button	buttoning
button	buttons
button	button
buy	boughten
buy	bought
buy	bought
buy	buying
buy	buys
buy	buy
bypass	bypassed
bypass	bypassed
bypass	bypasses
bypass	bypassing
bypass	bypass
cab	cabbed
cab	cabbed
cab	cabbing
cab	cabs
cab	cab
cabin	cabined
cabin	cabined
cabin	cabining
cabin	cabins
cabin	cabin
cable	cabled
cable	cabled
cable	cables
cable	cable
cable	cabling
cage	caged
cage	caged
cage	cages
cage	cage
cage	caging
cake	caked
cake	caked
cake	cakes
cake	cake
cake	caking
calendar	calendared
calendar	calendared
calendar	calendaring
calendar	calendars
calendar	calendar
call	called
call	called
call	calling
call	calls
call	call
calm	calmed
calm	calmed
calm	calming
calm	calms
calm	calm
cameo	cameoed
cameo	cameoed
cameo	cameoing
cameo	cameos
cameo	cameo
campaign	campaigned
campaign	campaigned
campaign	campaigning
campaign	campaigns
campaign	campaign
camp	camped
camp	camped
camp	camping
camp	camps
camp	camp
campus	campused
campus	campused
campus	campuses
campus	campusing
campus	campus
canal	canaled
canal	canaled
canal	canaling
canal	canalled
canal	canalled
canal	canalling
canal	canals
canal	canal
can	canned
can	canned
can	canning
can	cans
can	can
can	can
can	could
can	couth
candy	candied
candy	candied
candy	candying
candy	candy
candy	candy
cane	caned
cane	caned
cane	canes
cane	cane
cane	caning
cannon	cannoned
cannon	cannoned
cannon	cannoning
cannon	cannons
cannon	cannon
canoe	canoed
canoe	canoed
canoe	canoeing
canoe	canoes
canoe	canoe
canton	cantoned
canton	cantoned
canton	cantoning
canton	cantons
canton	canton
canvas	canvased
canvas	canvased
canvas	canvases
canvas	canvasing
canvas	canvas
cap	capped
cap	capped
cap	capping
cap	caps
cap	cap
cape	caped
cape	caped
cape	capes
cape	cape
cape	caping
captain	captained
captain	captained
captain	captaining
captain	captains
captain	captain
capture	captured
capture	captured
capture	captures
capture	capture
capture	capturing
card	carded
card	carded
card	carding
card	cards
card	card
care	cared
care	cared
care	cares
care	care
care	caring
career	careered
career	careered
career	careering
career	careers
career	career
carl	carled
carl	carled
carl	carling
carl	carls
carl	carl
carol	caroled
carol	caroled
carol	caroling
carol	carolled
carol	carolled
carol	carolling
carol	carols
carol	carol
carry	carried
carry	carried
carry	carries
carry	carrying
carry	carry
cartoon	cartooned
cartoon	cartooned
cartoon	cartooning
cartoon	cartoons
cartoon	cartoon
case	cased
case	cased
case	cases
case	case
case	casing
cash	cashed
cash	cashed
cash	cashes
cash	cashing
cash	cash
cast	casting
cast	casts
cast	cast
cast	cast
cast	cast
castle	castled
castle	castled
castle	castles
castle	castle
castle	castling
catalog	cataloged
catalog	cataloged
catalog	cataloging
catalog	catalogs
catalog	catalog
catalogue	catalogued
catalogue	catalogued
catalogue	catalogues
catalogue	catalogue
catalogue	cataloguing
cat	cats
cat	catted
cat	catted
cat	catting
cat	cat
catch	catches
catch	catching
catch	catch
catch	caught
catch	caught
caucus	caucused
caucus	caucused
caucus	caucuses
caucus	caucusing
caucus	caucus
cause	caused
cause	caused
cause	causes
cause	cause
cause	causing
cave	caved
cave	caved
cave	caves
cave	cave
cave	caving
cease	ceased
cease	ceased
cease	ceases
cease	cease
cease	ceasing
celebrate	celebrated
celebrate	celebrated
celebrate	celebrates
celebrate	celebrate
celebrate	celebrating
cell	celled
cell	celled
cell	celling
cell	cells
cell	cell
cement	cemented
cement	cemented
cement	cementing
cement	cements
cement	cement
census	censused
census	censused
census	censuses
census	censusing
census	census
center	centered
center	centered
center	centering
center	centers
center	center
centre	centred
centre	centred
centre	centres
centre	centre
centre	centring
certificate	certificated
certificate	certificated
certificate	certificates
certificate	certificate
certificate	certificating
chain	chained
chain	chained
chain	chaining
chain	chains
chain	chain
chair	chaired
chair	chaired
chair	chairing
chair	chairs
chair	chair
challenge	challenged
challenge	challenged
challenge	challenges
challenge	challenge
challenge	challenging
chamber	chambered
chamber	chambered
chamber	chambering
chamber	chambers
chamber	chamber
champion	championed
champion	championed
champion	championing
champion	champions
champion	champion
chance	chanced
chance	chanced
chance	chances
chance	chance
chance	chancing
change	changed
change	changed
change	changes
change	change
change	changing
channel	channeled
channel	channeled
channel	channeling
channel	channelled
channel	channelled
channel	channelling
channel	channels
channel	channel
chapel	chapelled
chapel	chapelled
chapel	chapelling
chapel	chapels
chapel	chapel
chapter	chaptered
chapter	chaptered
chapter	chaptering
chapter	chapters
chapter	chapter
character	charactered
character	charactered
character	charactering
character	characters
character	character
charge	charged
charge	charged
charge	charges
charge	charge
charge	charging
charleston	charlestoned
charleston	charlestoned
charleston	charlestoning
charleston	charlestons
charleston	charleston
chart	charted
chart	charted
chart	charting
chart	charts
chart	chart
charter	chartered
charter	chartered
charter	chartering
charter	charters
charter	charter
chase	chased
chase	chased
chase	chases
chase	chase
chase	chasing
cheap	cheaped
cheap	cheaped
cheap	cheaping
cheap	cheaps
cheap	cheap
check	checked
check	checked
check	checking
check	checks
check	check
cheese	cheesed
cheese	cheesed
cheese	cheeses
cheese	cheese
cheese	cheesing
chest	chested
chest	chested
chest	chesting
chest	chests
chest	chest
chicken	chickened
chicken	chickened
chicken	chickening
chicken	chickens
chicken	chicken
chip	chipped
chip	chipped
chip	chipping
chip	chips
chip	chip
choose	chooses
choose	choose
choose	choosing
choose	chosen
choose	chose
chord	chorded
chord	chorded
chord	chording
chord	chords
chord	chord
chorus	chorused
chorus	chorused
chorus	choruses
chorus	chorusing
chorus	chorussed
chorus	chorussed
chorus	chorussing
chorus	chorus
chronicle	chronicled
chronicle	chronicled
chronicle	chronicles
chronicle	chronicle
chronicle	chronicling
chuck	chucked
chuck	chucked
chuck	chucking
chuck	chucks
chuck	chuck
church	churched
church	churched
church	churches
church	churching
church	church
circle	circled
circle	circled
circle	circles
circle	circle
circle	circling
circuit	circuited
circuit	circuited
circuit	circuiting
circuit	circuits
circuit	circuit
claim	claimed
claim	claimed
claim	claiming
claim	claims
claim	claim
clash	clashed
clash	clashed
clash	clashes
clash	clashing
clash	clash
class	classed
class	classed
class	classes
class	classing
class	class
clause	claused
clause	claused
clause	clauses
clause	clause
clause	clausing
clay	clayed
clay	clayed
clay	claying
clay	clays
clay	clay
clean	cleaned
clean	cleaned
clean	cleaning
clean	cleans
clean	clean
clear	cleared
clear	cleared
clear	clearing
clear	clears
clear	clear
clerk	clerked
clerk	clerked
clerk	clerking
clerk	clerks
clerk	clerk
climate	climated
climate	climated
climate	climates
climate	climate
climate	climating
climb	climbed
climb	climbed
climb	climbing
climb	climbs
climb	climb
clip	clipped
clip	clipped
clip	clipping
clip	clips
clip	clipt
clip	clipt
clip	clip
clock	clocked
clock	clocked
clock	clocking
clock	clocks
clock	clock
close	closed
close	closed
close	closes
close	close
close	closing
cloud	clouded
cloud	clouded
cloud	clouding
cloud	clouds
cloud	cloud
club	clubbed
club	clubbed
club	clubbing
club	clubs
club	club
cluster	clustered
cluster	clustered
cluster	clustering
cluster	clusters
cluster	cluster
coach	coached
coach	coached
coach	coaches
coach	coaching
coach	coach
coal	coaled
coal	coaled
coal	coaling
coal	coals
coal	coal
coast	coasted
coast	coasted
coast	coasting
coast	coasts
coast	coast
coat	coated
coat	coated
coat	coating
coat	coats
coat	coat
code	coded
code	coded
code	codes
code	code
code	coding
coffee	coffeed
coffee	coffeed
coffee	coffeeing
coffee	coffees
coffee	coffee
coin	coined
coin	coined
coin	coining
coin	coins
coin	coin
collapse	collapsed
collapse	collapsed
collapse	collapses
collapse	collapse
collapse	collapsing
colleague	colleagued
colleague	colleagued
colleague	colleagues
colleague	colleague
colleague	colleaguing
collect	collected
collect	collected
collect	collecting
collect	collects
collect	collect
color	colored
color	colored
color	coloring
color	colors
color	color
colour	coloured
colour	coloured
colour	colouring
colour	colours
colour	colour
colt	colted
colt	colted
colt	colting
colt	colts
colt	colt
combat	combated
combat	combated
combat	combating
combat	combats
combat	combatted
combat	combatted
combat	combatting
combat	combat
combine	combined
combine	combined
combine	combines
combine	combine
combine	combining
come	came
come	comen
come	comes
come	come
come	come
come	coming
comfort	comforted
comfort	comforted
comfort	comforting
comfort	comforts
comfort	comfort
command	commanded
command	commanded
command	commanding
command	commands
command	command
commemorate	commemorated
commemorate	commemorated
commemorate	commemorates
commemorate	commemorate
commemorate	commemorating
comment	commented
comment	commented
comment	commenting
comment	comments
comment	comment
commerce	commerced
commerce	commerced
commerce	commerces
commerce	commerce
commerce	commercing
commission	commissioned
commission	commissioned
commission	commissioning
commission	commissions
commission	commission
commit	commits
commit	committed
commit	committed
commit	committing
commit	commit
common	commoned
common	commoned
common	commoning
common	commons
common	common
commune	communed
commune	communed
commune	communes
commune	commune
commune	communing
communicate	communicated
communicate	communicated
communicate	communicates
communicate	communicate
communicate	communicating
compact	compacted
compact	compacted
compact	compacting
compact	compacts
compact	compact
companion	companioned
companion	companioned
companion	companioning
companion	companions
companion	companion
company	companied
company	companied
company	companies
company	companying
company	company
compare	compared
compare	compared
compare	compares
compare	compare
compare	comparing
compete	competed
compete	competed
compete	competes
compete	compete
compete	competing
complement	complemented
complement	complemented
complement	complementing
complement	complements
complement	complement
complete	completed
complete	completed
complete	completes
complete	complete
complete	completing
complex	complexed
complex	complexed
complex	complexes
complex	complexing
complex	complex
comply	complied
comply	complied
comply	complies
comply	complying
comply	comply
composite	composited
composite	composited
composite	composites
composite	composite
composite	compositing
compound	compounded
compound	compounded
compound	compounding
compound	compounds
compound	compound
comprise	comprised
comprise	comprised
comprise	comprises
comprise	comprise
comprise	comprising
compromise	compromised
compromise	compromised
compromise	compromises
compromise	compromise
compromise	compromising
concentrate	concentrated
concentrate	concentrated
concentrate	concentrates
concentrate	concentrate
concentrate	concentrating
concern	concerned
concern	concerned
concern	concerning
concern	concerns
concern	concern
concert	concerted
concert	concerted
concert	concerting
concert	concerts
concert	concert
con	conned
con	conned
con	conning
con	cons
con	con
concrete	concreted
concrete	concreted
concrete	concretes
concrete	concrete
concrete	concreting
condition	conditioned
condition	conditioned
condition	conditioning
condition	conditions
condition	condition
conduct	conducted
conduct	conducted
conduct	conducting
conduct	conducts
conduct	conduct
cone	coned
cone	coned
cone	cones
cone	cone
cone	coning
confederate	confederated
confederate	confederated
confederate	confederates
confederate	confederate
confederate	confederating
conference	conferenced
conference	conferenced
conference	conferences
conference	conference
conference	conferencing
confirm	confirmed
confirm	confirmed
confirm	confirming
confirm	confirms
confirm	confirm
conflict	conflicted
conflict	conflicted
conflict	conflicting
conflict	conflicts
conflict	conflict
connect	connected
connect	connected
connect	connecting
connect	connects
connect	connect
conquest	conquested
conquest	conquested
conquest	conquesting
conquest	conquests
conquest	conquest
consent	consented
consent	consented
consent	consenting
consent	consents
consent	consent
consequence	consequenced
consequence	consequenced
consequence	consequences
consequence	consequence
consequence	consequencing
consider	considered
consider	considered
consider	considering
consider	considers
consider	consider
consist	consisted
consist	consisted
consist	consisting
consist	consists
consist	consist
console	consoled
console	consoled
console	consoles
console	console
console	consoling
consort	consorted
consort	consorted
consort	consorting
consort	consorts
consort	consort
constitute	constituted
constitute	constituted
constitute	constitutes
constitute	constitute
constitute	constituting
construct	constructed
construct	constructed
construct	constructing
construct	constructs
construct	construct
contact	contacted
contact	contacted
contact	contacting
contact	contacts
contact	contact
contain	contained
contain	contained
contain	containing
contain	contains
contain	contain
content	contented
content	contented
content	contenting
content	contents
content	content
contest	contested
contest	contested
contest	contesting
contest	contests
contest	contest
context	contexted
context	contexted
context	contexting
context	contexts
context	context
continue	continued
continue	continued
continue	continues
continue	continue
continue	continuing
contract	contracted
contract	contracted
contract	contracting
contract	contracts
contract	contract
contrary	contraried
contrary	contraried
contrary	contraries
contrary	contrarying
contrary	contrary
contrast	contrasted
contrast	contrasted
contrast	contrasting
contrast	contrasts
contrast	contrast
contribute	contributed
contribute	contributed
contribute	contributes
contribute	contribute
contribute	contributing
control	controlled
control	controlled
control	controlling
control	controls
control	control
convent	convented
convent	convented
convent	conventing
convent	convents
convent	convent
conversation	conversationed
conversation	conversationed
conversation	conversationing
conversation	conversations
conversation	conversation
convert	converted
convert	converted
convert	converting
convert	converts
convert	convert
convince	convinced
convince	convinced
convince	convinces
convince	convince
convince	convincing
convoy	convoyed
convoy	convoyed
convoy	convoying
convoy	convoys
convoy	convoy
cook	cooked
cook	cooked
cook	cooking
cook	cooks
cook	cook
cool	cooled
cool	cooled
cool	cooling
cool	cools
cool	cool
cooper	coopered
cooper	coopered
cooper	coopering
cooper	coopers
cooper	cooper
coordinate	coordinated
coordinate	coordinated
coordinate	coordinates
coordinate	coordinate
coordinate	coordinating
cope	coped
cope	coped
cope	copes
cope	cope
cope	coping
copper	coppered
copper	coppered
copper	coppering
copper	coppers
copper	copper
copy	copied
copy	copied
copy	copies
copy	copying
copy	copy
copyright	copyrighted
copyright	copyrighted
copyright	copyrighting
copyright	copyrights
copyright	copyright
cord	corded
cord	corded
cord	cording
cord	cords
cord	cord
core	cored
core	cored
core	cores
core	core
core	coring
cork	corked
cork	corked
cork	corking
cork	corks
cork	cork
corn	corned
corn	corned
corn	corning
corn	corns
corn	corn
corner	cornered
corner	cornered
corner	cornering
corner	corners
corner	corner
corporate	corporated
corporate	corporated
corporate	corporates
corporate	corporate
corporate	corporating
correct	corrected
correct	corrected
correct	correcting
correct	corrects
correct	correct
corrupt	corrupted
corrupt	corrupted
corrupt	corrupting
corrupt	corrupts
corrupt	corrupt
cost	costed
cost	costed
cost	costing
cost	costs
cost	cost
cost	cost
cost	cost
costume	costumed
costume	costumed
costume	costumes
costume	costume
costume	costuming
cottage	cottaged
cottage	cottaged
cottage	cottages
cottage	cottage
cottage	cottaging
cotton	cottoned
cotton	cottoned
cotton	cottoning
cotton	cottons
cotton	cotton
counsel	counseled
counsel	counseled
counsel	counseling
counsel	counselled
counsel	counselled
counsel	counselling
counsel	counsels
counsel	counsel
count	counted
count	counted
count	counting
count	counts
count	count
counter	countered
counter	countered
counter	countering
counter	counters
counter	counter
couple	coupled
couple	coupled
couple	couples
couple	couple
couple	coupling
courage	couraged
courage	couraged
courage	courages
courage	courage
courage	couraging
course	coursed
course	coursed
course	courses
course	course
course	coursing
court	courted
court	courted
court	courting
court	courts
court	court
cove	coved
cove	coved
cove	coves
cove	cove
cove	coving
cover	covered
cover	covered
cover	covering
cover	covers
cover	cover
cow	cowed
cow	cowed
cow	cowing
cow	cows
cow	cow
cox	coxed
cox	coxed
cox	coxes
cox	coxing
cox	cox
craft	crafted
craft	crafted
craft	crafting
craft	crafts
craft	craft
crane	craned
crane	craned
crane	cranes
crane	crane
crane	craning
crash	crashed
crash	crashed
crash	crashes
crash	crashing
crash	crash
crater	cratered
crater	cratered
crater	cratering
crater	craters
crater	crater
cream	creamed
cream	creamed
cream	creaming
cream	creams
cream	cream
create	created
create	created
create	creates
create	create
create	creating
credit	credited
credit	credited
credit	crediting
credit	credits
credit	credit
crescent	crescented
crescent	crescented
crescent	crescenting
crescent	crescents
crescent	crescent
crest	crested
crest	crested
crest	cresting
crest	crests
crest	crest
crew	crewed
crew	crewed
crew	crewing
crew	crews
crew	crew
cricket	cricketed
cricket	cricketed
cricket	cricketing
cricket	crickets
cricket	cricket
crime	crimed
crime	crimed
crime	crimes
crime	crime
crime	criming
crimson	crimsoned
crimson	crimsoned
crimson	crimsoning
crimson	crimsons
crimson	crimson
critic	criticked
critic	criticked
critic	criticking
critic	critics
critic	critic
crop	cropped
crop	cropped
crop	cropping
crop	crops
crop	crop
cross	crossed
cross	crossed
cross	crosses
cross	crossing
cross	cross
crow	crew
crow	crowed
crow	crowed
crow	crowing
crow	crows
crow	crow
crowd	crowded
crowd	crowded
crowd	crowding
crowd	crowds
crowd	crowd
crown	crowned
crown	crowned
crown	crowning
crown	crowns
crown	crown
cruise	cruised
cruise	cruised
cruise	cruises
cruise	cruise
cruise	cruising
crush	crushed
crush	crushed
crush	crushes
crush	crushing
crush	crush
cry	cried
cry	cried
cry	cries
cry	crying
cry	cry
culture	cultured
culture	cultured
culture	cultures
culture	culture
culture	culturing
cup	cupped
cup	cupped
cup	cupping
cup	cups
cup	cup
cure	cured
cure	cured
cure	cures
cure	cure
cure	curing
curse	cursed
curse	cursed
curse	curses
curse	curse
curse	cursing
curse	curst
curse	curst
curve	curved
curve	curved
curve	curves
curve	curve
curve	curving
custom	customed
custom	customed
custom	customing
custom	customs
custom	custom
cut	cuts
cut	cutting
cut	cut
cut	cut
cut	cut
cycle	cycled
cycle	cycled
cycle	cycles
cycle	cycle
cycle	cycling
damage	damaged
damage	damaged
damage	damages
damage	damage
damage	damaging
dam	dammed
dam	dammed
dam	damming
dam	dams
dam	dam
dance	danced
dance	danced
dance	dances
dance	dance
dance	dancing
danger	dangered
danger	dangered
danger	dangering
danger	dangers
danger	danger
dash	dashed
dash	dashed
dash	dashes
dash	dashing
dash	dash
database	databased
database	databased
database	databases
database	database
database	databasing
date	dated
date	dated
date	dates
date	date
date	dating
dave	daved
dave	daved
dave	daves
dave	dave
dave	daving
dawn	dawned
dawn	dawned
dawn	dawning
dawn	dawns
dawn	dawn
day	dayed
day	dayed
day	daying
day	days
day	day
dead	deaded
dead	deaded
dead	deading
dead	deads
dead	dead
deadline	deadlined
deadline	deadlined
deadline	deadlines
deadline	deadline
deadline	deadlining
deaf	deafed
deaf	deafed
deaf	deafing
deaf	deafs
deaf	deaf
deal	dealing
deal	deals
deal	dealt
deal	dealt
deal	deal
dean	deaned
dean	deaned
dean	deaning
dean	deans
dean	dean
debate	debated
debate	debated
debate	debates
debate	debate
debate	debating
debut	debuted
debut	debuted
debut	debuting
debut	debuts
debut	debut
decay	decayed
decay	decayed
decay	decaying
decay	decays
decay	decay
decide	decided
decide	decided
decide	decides
decide	decide
decide	deciding
decision	decisioned
decision	decisioned
decision	decisioning
decision	decisions
decision	decision
deck	decked
deck	decked
deck	decking
deck	decks
deck	deck
declare	declared
declare	declared
declare	declares
declare	declare
declare	declaring
decline	declined
decline	declined
decline	declines
decline	decline
decline	declining
decrease	decreased
decrease	decreased
decrease	decreases
decrease	decrease
decrease	decreasing
decree	decreed
decree	decreed
decree	decreeing
decree	decrees
decree	decree
de	dein
de	de
de	did
de	diz
de	dyun
dee	deein
dee	dee
dee	diz
dee	dyun
dee	dyun
default	defaulted
default	defaulted
default	defaulting
default	defaults
default	default
defeat	defeated
defeat	defeated
defeat	defeating
defeat	defeats
defeat	defeat
defence	defenced
defence	defenced
defence	defences
defence	defence
defence	defencing
defend	defended
defend	defended
defend	defending
defend	defends
defend	defend
define	defined
define	defined
define	defines
define	define
define	defining
defunct	defuncted
defunct	defuncted
defunct	defuncting
defunct	defuncts
defunct	defunct
delay	delayed
delay	delayed
delay	delaying
delay	delays
delay	delay
delegate	delegated
delegate	delegated
delegate	delegates
delegate	delegate
delegate	delegating
deliver	delivered
deliver	delivered
deliver	delivering
deliver	delivers
deliver	deliver
demand	demanded
demand	demanded
demand	demanding
demand	demands
demand	demand
demise	demised
demise	demised
demise	demises
demise	demise
demise	demising
demo	demoed
demo	demoed
demo	demoing
demo	demos
demo	demo
demonstrate	demonstrated
demonstrate	demonstrated
demonstrate	demonstrates
demonstrate	demonstrate
demonstrate	demonstrating
den	denned
den	denned
den	denning
den	dens
den	den
depend	depended
depend	depended
depend	depending
depend	depends
depend	depend
depict	depicted
depict	depicted
depict	depicting
depict	depicts
depict	depict
deposit	deposited
deposit	deposited
deposit	depositing
deposit	deposits
deposit	deposit
describe	described
describe	described
describe	describes
describe	describe
describe	describing
desert	deserted
desert	deserted
desert	deserting
desert	deserts
desert	desert
design	designed
design	designed
design	designing
design	designs
design	design
desire	desired
desire	desired
desire	desires
desire	desire
desire	desiring
desk	desked
desk	desked
desk	desking
desk	desks
desk	desk
despite	despited
despite	despited
despite	despites
despite	despite
despite	despiting
destroy	destroyed
destroy	destroyed
destroy	destroying
destroy	destroys
destroy	destroy
detail	detailed
detail	detailed
detail	detailing
detail	details
detail	detail
detect	detected
detect	detected
detect	detecting
detect	detects
detect	detect
determine	determined
determine	determined
determine	determines
determine	determine
determine	determining
develop	developed
develop	developed
develop	developing
develop	develops
develop	developt
develop	developt
develop	develop
devil	deviled
devil	deviled
devil	deviling
devil	devilled
devil	devilled
devil	devilling
devil	devils
devil	devil
dialogue	dialogued
dialogue	dialogued
dialogue	dialogues
dialogue	dialogue
dialogue	dialoguing
diamond	diamonded
diamond	diamonded
diamond	diamonding
diamond	diamonds
diamond	diamond
dick	dicked
dick	dicked
dick	dicking
dick	dicks
dick	dick
dictionary	dictionaried
dictionary	dictionaried
dictionary	dictionaries
dictionary	dictionarying
dictionary	dictionary
die	died
die	died
die	dies
die	die
die	dying
diesel	dieseled
diesel	dieseled
diesel	dieseling
diesel	diesels
diesel	diesel
diet	dieted
diet	dieted
diet	dieting
diet	diets
diet	diet
differ	differed
differ	differed
differ	differing
differ	differs
differ	differ
difference	differenced
difference	differenced
difference	differences
difference	difference
difference	differencing
difficult	difficulted
difficult	difficulted
difficult	difficulting
difficult	difficults
difficult	difficult
dimension	dimensioned
dimension	dimensioned
dimension	dimensioning
dimension	dimensions
dimension	dimension
direct	directed
direct	directed
direct	directing
direct	directs
direct	direct
dirt	dirted
dirt	dirted
dirt	dirting
dirt	dirts
dirt	dirt
dirty	dirtied
dirty	dirtied
dirty	dirties
dirty	dirtying
dirty	dirty
discharge	discharged
discharge	discharged
discharge	discharges
discharge	discharge
discharge	discharging
discipline	disciplined
discipline	disciplined
discipline	disciplines
discipline	discipline
discipline	disciplining
disco	discoed
disco	discoed
disco	discoing
disco	discos
disco	disco
discourse	discoursed
discourse	discoursed
discourse	discourses
discourse	discourse
discourse	discoursing
discover	discovered
discover	discovered
discover	discovering
discover	discovers
discover	discover
discuss	discussed
discuss	discussed
discuss	discusses
discuss	discussing
discuss	discuss
disease	diseased
disease	diseased
disease	diseases
disease	disease
disease	diseasing
dish	dished
dish	dished
dish	dishes
dish	dishing
dish	dish
disk	disked
disk	disked
disk	disking
disk	disks
disk	disk
display	displayed
display	displayed
display	displaying
display	displays
display	display
dispute	disputed
dispute	disputed
dispute	disputes
dispute	dispute
dispute	disputing
distance	distanced
distance	distanced
distance	distances
distance	distance
distance	distancing
distinguish	distinguished
distinguish	distinguished
distinguish	distinguishes
distinguish	distinguishing
distinguish	distinguish
distribute	distributed
distribute	distributed
distribute	distributes
distribute	distribute
distribute	distributing
district	districted
district	districted
district	districting
district	districts
district	district
dive	dived
dive	dived
dive	dives
dive	dive
dive	diving
dive	dove
dive	dove
divide	divided
divide	divided
divide	divides
divide	divide
divide	dividing
divine	divined
divine	divined
divine	divines
divine	divine
divine	divining
divorce	divorced
divorce	divorced
divorce	divorces
divorce	divorce
divorce	divorcing
dock	docked
dock	docked
dock	docking
dock	docks
dock	dock
doctorate	doctorated
doctorate	doctorated
doctorate	doctorates
doctorate	doctorate
doctorate	doctorating
doctor	doctored
doctor	doctored
doctor	doctoring
doctor	doctors
doctor	doctor
document	documented
document	documented
document	documenting
document	documents
document	document
dodge	dodged
dodge	dodged
dodge	dodges
dodge	dodge
dodge	dodging
do	did
do	does
do	doing
do	done
do	do
dog	dogged
dog	dogged
dog	dogging
dog	dogs
dog	dog
dominate	dominated
dominate	dominated
dominate	dominates
dominate	dominate
dominate	dominating
don	donned
don	donned
don	donning
don	dons
don	don
door	doored
door	doored
door	dooring
door	doors
door	door
dot	dots
dot	dotted
dot	dotted
dot	dotting
dot	dot
double	doubled
double	doubled
double	doubles
double	double
double	doubling
doubt	doubted
doubt	doubted
doubt	doubting
doubt	doubts
doubt	doubt
down	downed
down	downed
down	downing
down	downs
down	down
download	downloaded
download	downloaded
download	downloading
download	downloads
download	download
draft	drafted
draft	drafted
draft	drafting
draft	drafts
draft	draft
drag	dragged
drag	dragged
drag	dragging
drag	drags
drag	drag
drag	drug
drag	drug
draw	drawing
draw	drawn
draw	draws
draw	draw
draw	drew
dream	dreamed
dream	dreamed
dream	dreaming
dream	dreams
dream	dreamt
dream	dreamt
dream	dream
dream	drempt
dream	drempt
dress	dressed
dress	dressed
dress	dresses
dress	dressing
dress	dress
dress	drest
drill	drilled
drill	drilled
drill	drilling
drill	drills
drill	drill
drink	drank
drink	drank
drink	drinked
drink	drinked
drink	drinking
drink	drinks
drink	drink
drink	drunken
drink	drunk
drink	drunk
drive	drave
drive	driven
drive	drives
drive	drive
drive	driving
drive	drove
drop	dropped
drop	dropped
drop	dropping
drop	drops
drop	drop
drove	droved
drove	droved
drove	droves
drove	drove
drove	droving
drug	drugged
drug	drugged
drug	drugging
drug	drugs
drug	drug
drum	drummed
drum	drummed
drum	drumming
drum	drums
drum	drum
dry	dried
dry	dried
dry	dries
dry	drying
dry	dry
duchess	duchessed
duchess	duchessed
duchess	duchesses
duchess	duchessing
duchess	duchess
duck	ducked
duck	ducked
duck	ducking
duck	ducks
duck	duck
duel	dueled
duel	dueled
duel	dueling
duel	duelled
duel	duelled
duel	duelling
duel	duels
duel	duel
duet	dueted
duet	dueted
duet	dueting
duet	duets
duet	duet
duke	duked
duke	duked
duke	dukes
duke	duke
duke	duking
dust	dusted
dust	dusted
dust	dusting
dust	dusts
dust	dust
dwarf	dwarfed
dwarf	dwarfed
dwarf	dwarfing
dwarf	dwarfs
dwarf	dwarf
eagle	eagled
eagle	eagled
eagle	eagles
eagle	eagle
eagle	eagling
ear	eared
ear	eared
ear	earing
ear	ears
ear	ear
earn	earned
earn	earned
earn	earning
earn	earns
earn	earn
earth	earthed
earth	earthed
earth	earthing
earth	earths
earth	earth
ease	eased
ease	eased
ease	eases
ease	ease
ease	easing
easy	easied
easy	easied
easy	easies
easy	easying
easy	easy
eat	ate
eat	eaten
eat	eating
eat	eats
eat	eat
eat	et
echo	echoed
echo	echoed
echo	echoes
echo	echoing
echo	echo
eclipse	eclipsed
eclipse	eclipsed
eclipse	eclipses
eclipse	eclipse
eclipse	eclipsing
edge	edged
edge	edged
edge	edges
edge	edge
edge	edging
effect	effected
effect	effected
effect	effecting
effect	effects
effect	effect
effort	efforted
effort	efforted
effort	efforting
effort	efforts
effort	effort
egg	egged
egg	egged
egg	egging
egg	eggs
egg	egg
elaborate	elaborated
elaborate	elaborated
elaborate	elaborates
elaborate	elaborate
elaborate	elaborating
elder	eldered
elder	eldered
elder	eldering
elder	elders
elder	elder
elect	elected
elect	elected
elect	electing
elect	elects
elect	elect
eliminate	eliminated
eliminate	eliminated
eliminate	eliminates
eliminate	eliminate
eliminate	eliminating
email	emailed
email	emailed
email	emailing
email	emails
email	email
emerge	emerged
emerge	emerged
emerge	emerges
emerge	emerge
emerge	emerging
employ	employed
employ	employed
employ	employing
employ	employs
employ	employ
empress	empressed
empress	empressed
empress	empresses
empress	empressing
empress	empress
empty	emptied
empty	emptied
empty	empties
empty	emptying
empty	empty
enable	enabled
enable	enabled
enable	enables
enable	enable
enable	enabling
encounter	encountered
encounter	encountered
encounter	encountering
encounter	encounters
encounter	encounter
encourage	encouraged
encourage	encouraged
encourage	encourages
encourage	encourage
encourage	encouraging
end	ended
end	ended
end	ending
end	ends
end	end
engage	engaged
engage	engaged
engage	engages
engage	engage
engage	engaging
engine	engined
engine	engined
engine	engines
engine	engine
engine	engining
engineer	engineered
engineer	engineered
engineer	engineering
engineer	engineers
engineer	engineer
enhance	enhanced
enhance	enhanced
enhance	enhances
enhance	enhance
enhance	enhancing
enjoy	enjoyed
enjoy	enjoyed
enjoy	enjoying
enjoy	enjoys
enjoy	enjoy
ensure	ensured
ensure	ensured
ensure	ensures
ensure	ensure
ensure	ensuring
enter	entered
enter	entered
enter	entering
enter	enters
enter	enter
enterprise	enterprised
enterprise	enterprised
enterprise	enterprises
enterprise	enterprise
enterprise	enterprising
entrance	entranced
entrance	entranced
entrance	entrances
entrance	entrance
entrance	entrancing
equal	equaled
equal	equaled
equal	equaling
equal	equalled
equal	equalled
equal	equalling
equal	equals
equal	equal
equivalent	equivalented
equivalent	equivalented
equivalent	equivalenting
equivalent	equivalents
equivalent	equivalent
erect	erected
erect	erected
erect	erecting
erect	erects
erect	erect
error	errored
error	errored
error	erroring
error	errors
error	error
escape	escaped
escape	escaped
escape	escapes
escape	escape
escape	escaping
escort	escorted
escort	escorted
escort	escorting
escort	escorts
escort	escort
essay	essayed
essay	essayed
essay	essaying
essay	essays
essay	essay
establish	established
establish	established
establish	establishes
establish	establishing
establish	establish
estimate	estimated
estimate	estimated
estimate	estimates
estimate	estimate
estimate	estimating
eve	eved
eve	eved
eve	eves
eve	eve
eve	eving
even	evened
even	evened
even	evening
even	evens
even	even
evidence	evidenced
evidence	evidenced
evidence	evidences
evidence	evidence
evidence	evidencing
exact	exacted
exact	exacted
exact	exacting
exact	exacts
exact	exact
examine	examined
examine	examined
examine	examines
examine	examine
examine	examining
example	exampled
example	exampled
example	examples
example	example
example	exampling
exceed	exceeded
exceed	exceeded
exceed	exceeding
exceed	exceeds
exceed	exceed
except	excepted
except	excepted
except	excepting
except	excepts
except	except
exchange	exchanged
exchange	exchanged
exchange	exchanges
exchange	exchange
exchange	exchanging
exercise	exercised
exercise	exercised
exercise	exercises
exercise	exercise
exercise	exercising
ex	exed
ex	exed
ex	exes
ex	exing
ex	ex
exhibit	exhibited
exhibit	exhibited
exhibit	exhibiting
exhibit	exhibits
exhibit	exhibit
exile	exiled
exile	exiled
exile	exiles
exile	exile
exile	exiling
exist	existed
exist	existed
exist	existing
exist	exists
exist	exist
exit	exited
exit	exited
exit	exiting
exit	exits
exit	exit
expand	expanded
expand	expanded
expand	expanding
expand	expands
expand	expand
expect	expected
expect	expected
expect	expecting
expect	expects
expect	expect
expense	expensed
expense	expensed
expense	expenses
expense	expense
expense	expensing
experience	experienced
experience	experienced
experience	experiences
experience	experience
experience	experiencing
experiment	experimented
experiment	experimented
experiment	experimenting
experiment	experiments
experiment	experiment
explain	explained
explain	explained
explain	explaining
explain	explains
explain	explain
explore	explored
explore	explored
explore	explores
explore	explore
explore	exploring
export	exported
export	exported
export	exporting
export	exports
export	export
express	expressed
express	expressed
express	expresses
express	expressing
express	express
extend	extended
extend	extended
extend	extending
extend	extends
extend	extend
extract	extracted
extract	extracted
extract	extracting
extract	extracts
extract	extract
eye	eyed
eye	eyed
eye	eyeing
eye	eyes
eye	eye
eye	eying
facebook	facebooked
facebook	facebooked
facebook	facebooking
facebook	facebooks
facebook	facebook
face	faced
face	faced
face	faces
face	face
face	facing
facilitate	facilitated
facilitate	facilitated
facilitate	facilitates
facilitate	facilitate
facilitate	facilitating
factor	factored
factor	factored
factor	factoring
factor	factors
factor	factor
fail	failed
fail	failed
fail	failing
fail	fails
fail	fail
fair	faired
fair	faired
fair	fairing
fair	fairs
fair	fair
fake	faked
fake	faked
fake	fakes
fake	fake
fake	faking
falcon	falconed
falcon	falconed
falcon	falconing
falcon	falcons
falcon	falcon
fall	fallen
fall	falling
fall	falls
fall	fall
fall	felled
fall	felled
fall	fell
fame	famed
fame	famed
fame	fames
fame	fame
fame	faming
fan	fanned
fan	fanned
fan	fanning
fan	fans
fan	fan
fantasy	fantasied
fantasy	fantasied
fantasy	fantasies
fantasy	fantasying
fantasy	fantasy
fare	fared
fare	fared
fare	faren
fare	fares
fare	fare
fare	faring
fare	fore
farm	farmed
farm	farmed
farm	farming
farm	farms
farm	farm
fashion	fashioned
fashion	fashioned
fashion	fashioning
fashion	fashions
fashion	fashion
fast	fasted
fast	fasted
fast	fasting
fast	fasts
fast	fast
fate	fated
fate	fated
fate	fates
fate	fate
fate	fating
fat	fats
fat	fatted
fat	fatted
fat	fatting
fat	fat
father	fathered
father	fathered
father	fathering
father	fathers
father	father
fault	faulted
fault	faulted
fault	faulting
fault	faults
fault	fault
favor	favored
favor	favored
favor	favoring
favor	favors
favor	favor
favorite	favorited
favorite	favorited
favorite	favorites
favorite	favorite
favorite	favoriting
favour	favoured
favour	favoured
favour	favouring
favour	favours
favour	favour
favourite	favourited
favourite	favourited
favourite	favourites
favourite	favourite
favourite	favouriting
fear	feared
fear	feared
fear	fearing
fear	fears
fear	fear
feast	feasted
feast	feasted
feast	feasting
feast	feasts
feast	feast
feat	feated
feat	feated
feat	feating
feat	feats
feat	feat
feature	featured
feature	featured
feature	features
feature	feature
feature	featuring
feedback	feedbacked
feedback	feedbacked
feedback	feedbacking
feedback	feedbacks
feedback	feedback
feed	fed
feed	fed
feed	feeding
feed	feeds
feed	feed
fee	feed
fee	feed
fee	feeing
fee	fees
fee	fee
feel	feeling
feel	feels
feel	feel
feel	felt
feel	felt
fell	felled
fell	felled
fell	felling
fell	fells
fell	fell
fellow	fellowed
fellow	fellowed
fellow	fellowing
fellow	fellows
fellow	fellow
fellowship	fellowshipped
fellowship	fellowshipped
fellowship	fellowshipping
fellowship	fellowships
fellowship	fellowship
felt	felted
felt	felted
felt	felting
felt	felts
felt	felt
fence	fenced
fence	fenced
fence	fences
fence	fence
fence	fencing
ferry	ferried
ferry	ferried
ferry	ferries
ferry	ferrying
ferry	ferry
feud	feuded
feud	feuded
feud	feuding
feud	feuds
feud	feud
fever	fevered
fever	fevered
fever	fevering
fever	fevers
fever	fever
field	fielded
field	fielded
field	fielding
field	fields
field	field
fight	fighting
fight	fights
fight	fight
fight	foughten
fight	fought
fight	fought
figure	figured
figure	figured
figure	figures
figure	figure
figure	figuring
file	filed
file	filed
file	files
file	file
file	filing
fill	filled
fill	filled
fill	filling
fill	fills
fill	fill
film	filmed
film	filmed
film	filming
film	films
film	film
filter	filtered
filter	filtered
filter	filtering
filter	filters
filter	filter
finance	financed
finance	financed
finance	finances
finance	finance
finance	financing
find	fand
find	finding
find	finds
find	find
find	founden
find	found
find	found
fine	fined
fine	fined
fine	fines
fine	fine
fine	fining
fin	finned
fin	finned
fin	finning
fin	fins
fin	fin
finger	fingered
finger	fingered
finger	fingering
finger	fingers
finger	finger
finish	finished
finish	finished
finish	finishes
finish	finishing
finish	finish
fire	fired
fire	fired
fire	fires
fire	fire
fire	firing
firm	firmed
firm	firmed
firm	firming
firm	firms
firm	firm
fish	fished
fish	fished
fish	fishes
fish	fishing
fish	fish
fit	fits
fit	fitted
fit	fitted
fit	fitting
fit	fit
fit	fit
fit	fit
fix	fixed
fix	fixed
fix	fixes
fix	fixing
fix	fixt
fix	fixt
fix	fix
fixture	fixtured
fixture	fixtured
fixture	fixtures
fixture	fixture
fixture	fixturing
flag	flagged
flag	flagged
flag	flagging
flag	flags
flag	flag
flame	flamed
flame	flamed
flame	flames
flame	flame
flame	flaming
flank	flanked
flank	flanked
flank	flanking
flank	flanks
flank	flank
flash	flashed
flash	flashed
flash	flashes
flash	flashing
flash	flash
flat	flats
flat	flatted
flat	flatted
flat	flatting
flat	flat
flee	fled
flee	fled
flee	fleeing
flee	flees
flee	flee
fleet	fleeted
fleet	fleeted
fleet	fleeting
fleet	fleets
fleet	fleet
flemish	flemished
flemish	flemished
flemish	flemishes
flemish	flemishing
flemish	flemish
flesh	fleshed
flesh	fleshed
flesh	fleshes
flesh	fleshing
flesh	flesh
flight	flighted
flight	flighted
flight	flighting
flight	flights
flight	flight
flint	flinted
flint	flinted
flint	flinting
flint	flints
flint	flint
flood	flooded
flood	flooded
flood	flooding
flood	floods
flood	flood
floor	floored
floor	floored
floor	flooring
floor	floors
floor	floor
flour	floured
flour	floured
flour	flouring
flour	flours
flour	flour
flower	flowered
flower	flowered
flower	flowering
flower	flowers
flower	flower
flow	flowed
flow	flowed
flow	flowing
flow	flows
flow	flow
flute	fluted
flute	fluted
flute	flutes
flute	flute
flute	fluting
fly	flew
fly	flied
fly	flied
fly	flies
fly	flown
fly	flying
fly	fly
focus	focused
focus	focused
focus	focuses
focus	focusing
focus	focussed
focus	focussed
focus	focusses
focus	focussing
focus	focus
fold	feld
fold	folded
fold	folded
fold	folden
fold	folding
fold	folds
fold	fold
follow	followed
follow	followed
follow	following
follow	follows
follow	follow
foot	footed
foot	footed
foot	footing
foot	foots
foot	foot
force	forced
force	forced
force	forces
force	force
force	forcing
ford	forded
ford	forded
ford	fording
ford	fords
ford	ford
forest	forested
forest	forested
forest	foresting
forest	forests
forest	forest
forget	forgat
forget	forgets
forget	forgetting
forget	forget
forget	forgotten
forget	forgot
forget	forgot
fork	forked
fork	forked
fork	forking
fork	forks
fork	fork
format	formats
format	formatted
format	formatted
format	formatting
format	format
form	formed
form	formed
form	forming
form	forms
form	form
fortress	fortressed
fortress	fortressed
fortress	fortresses
fortress	fortressing
fortress	fortress
fortune	fortuned
fortune	fortuned
fortune	fortunes
fortune	fortune
fortune	fortuning
forward	forwarded
forward	forwarded
forward	forwarding
forward	forwards
forward	forward
foster	fostered
foster	fostered
foster	fostering
foster	fosters
foster	foster
founder	foundered
founder	foundered
founder	foundering
founder	founders
founder	founder
found	founded
found	founded
found	founding
found	founds
found	found
fountain	fountained
fountain	fountained
fountain	fountaining
fountain	fountains
fountain	fountain
fox	foxed
fox	foxed
fox	foxes
fox	foxing
fox	fox
fraction	fractioned
fraction	fractioned
fraction	fractioning
fraction	fractions
fraction	fraction
frame	framed
frame	framed
frame	frames
frame	frame
frame	framing
franchise	franchised
franchise	franchised
franchise	franchises
franchise	franchise
franchise	franchising
frank	franked
frank	franked
frank	franking
frank	franks
frank	frank
fraud	frauded
fraud	frauded
fraud	frauding
fraud	frauds
fraud	fraud
free	freed
free	freed
free	freeing
free	frees
free	free
freelance	freelanced
freelance	freelanced
freelance	freelances
freelance	freelance
freelance	freelancing
freestyle	freestyled
freestyle	freestyled
freestyle	freestyles
freestyle	freestyle
freestyle	freestyling
freight	freighted
freight	freighted
freight	freighting
freight	freights
freight	freight
french	frenched
french	frenched
french	frenches
french	frenching
french	french
frequent	frequented
frequent	frequented
frequent	frequenting
frequent	frequents
frequent	frequent
friend	friended
friend	friended
friend	friending
friend	friends
friend	friend
fringe	fringed
fringe	fringed
fringe	fringes
fringe	fringe
fringe	fringing
fritz	fritzed
fritz	fritzed
fritz	fritzes
fritz	fritzing
fritz	fritz
frog	frogged
frog	frogged
frog	frogging
frog	frogs
frog	frog
front	fronted
front	fronted
front	fronting
front	fronts
front	front
frost	frosted
frost	frosted
frost	frosting
frost	frosts
frost	frost
fruit	fruited
fruit	fruited
fruit	fruiting
fruit	fruits
fruit	fruit
fuel	fueled
fuel	fueled
fuel	fueling
fuel	fuelled
fuel	fuelled
fuel	fuelling
fuel	fuels
fuel	fuel
fulfill	fulfilled
fulfill	fulfilled
fulfill	fulfilling
fulfill	fulfills
fulfill	fulfill
fuller	fullered
fuller	fullered
fuller	fullering
fuller	fullers
fuller	fuller
full	fulled
full	fulled
full	fulling
full	fulls
full	full
function	functioned
function	functioned
function	functioning
function	functions
function	function
fund	funded
fund	funded
fund	funding
fund	funds
fund	fund
fun	funned
fun	funned
fun	funning
fun	funs
fun	fun
funk	funked
funk	funked
funk	funking
funk	funks
funk	funk
fur	furred
fur	furred
fur	furring
fur	furs
fur	fur
further	furthered
further	furthered
further	furthering
further	furthers
further	further
gain	gained
gain	gained
gain	gaining
gain	gains
gain	gain
game	gamed
game	gamed
game	games
game	game
game	gaming
gang	ganged
gang	ganged
gang	ganging
gang	gangs
gang	gang
gap	gapped
gap	gapped
gap	gapping
gap	gaps
gap	gap
garage	garaged
garage	garaged
garage	garages
garage	garage
garage	garaging
garden	gardened
garden	gardened
garden	gardening
garden	gardens
garden	garden
garrison	garrisoned
garrison	garrisoned
garrison	garrisoning
garrison	garrisons
garrison	garrison
gas	gases
gas	gassed
gas	gassed
gas	gasses
gas	gassing
gas	gas
gate	gated
gate	gated
gate	gates
gate	gate
gate	gating
gateway	gatewayed
gateway	gatewayed
gateway	gatewaying
gateway	gateways
gateway	gateway
gather	gathered
gather	gathered
gather	gathering
gather	gathers
gather	gather
gauge	gauged
gauge	gauged
gauge	gauges
gauge	gauge
gauge	gauging
gay	gayed
gay	gayed
gay	gaying
gay	gays
gay	gay
gazette	gazetted
gazette	gazetted
gazette	gazettes
gazette	gazette
gazette	gazetting
gear	geared
gear	geared
gear	gearing
gear	gears
gear	gear
gender	gendered
gender	gendered
gender	gendering
gender	genders
gender	gender
general	generaled
general	generaled
general	generaling
general	generalled
general	generalled
general	generalling
general	generals
general	general
generate	generated
generate	generated
generate	generates
generate	generate
generate	generating
genocide	genocided
genocide	genocided
genocide	genocides
genocide	genocide
genocide	genociding
get	gets
get	getting
get	get
get	gotten
get	got
get	got
ghost	ghosted
ghost	ghosted
ghost	ghosting
ghost	ghosts
ghost	ghost
gift	gifted
gift	gifted
gift	gifting
gift	gifts
gift	gift
gill	gilled
gill	gilled
gill	gilling
gill	gills
gill	gill
give	gave
give	given
give	gives
give	give
give	giving
glass	glassed
glass	glassed
glass	glasses
glass	glassing
glass	glass
globe	globed
globe	globed
globe	globes
globe	globe
globe	globing
glory	gloried
glory	gloried
glory	glories
glory	glorying
glory	glory
god	godded
god	godded
god	godding
god	gods
god	god
go	goes
go	going
go	gone
go	go
golden	goldened
golden	goldened
golden	goldening
golden	goldens
golden	golden
gold	golded
gold	golded
gold	golding
gold	golds
gold	gold
golf	golfed
golf	golfed
golf	golfing
golf	golfs
golf	golf
good	gooded
good	gooded
good	gooding
good	goods
good	good
google	googled
google	googled
google	googles
google	google
google	googling
gorge	gorged
gorge	gorged
gorge	gorges
gorge	gorge
gorge	gorging
gospel	gospelled
gospel	gospelled
gospel	gospelling
gospel	gospels
gospel	gospel
go	went
grace	graced
grace	graced
grace	graces
grace	grace
grace	gracing
grade	graded
grade	graded
grade	grades
grade	grade
grade	grading
graduate	graduated
graduate	graduated
graduate	graduates
graduate	graduate
graduate	graduating
grain	grained
grain	grained
grain	graining
grain	grains
grain	grain
grammar	grammared
grammar	grammared
grammar	grammaring
grammar	grammars
grammar	grammar
grandfather	grandfathered
grandfather	grandfathered
grandfather	grandfathering
grandfather	grandfathers
grandfather	grandfather
grant	granted
grant	granted
grant	granting
grant	grants
grant	grant
graph	graphed
graph	graphed
graph	graphing
graph	graphs
graph	graph
grass	grassed
grass	grassed
grass	grasses
grass	grassing
grass	grass
grave	graved
grave	graved
grave	graven
grave	graves
grave	grave
grave	graving
grave	grove
gravel	gravelled
gravel	gravelled
gravel	gravelling
gravel	gravels
gravel	gravel
gray	grayed
gray	grayed
gray	graying
gray	grays
gray	gray
greek	greeked
greek	greeked
greek	greeking
greek	greeks
greek	greek
green	greened
green	greened
green	greening
green	greens
green	green
grey	greyed
grey	greyed
grey	greying
grey	greys
grey	grey
grid	gridded
grid	gridded
grid	gridding
grid	grids
grid	grid
gross	grossed
gross	grossed
gross	grosses
gross	grossing
gross	gross
ground	grounded
ground	grounded
ground	grounding
ground	grounds
ground	ground
group	grouped
group	grouped
group	grouping
group	groups
group	group
grow	grew
grow	growing
grow	grown
grow	grows
grow	grow
guarantee	guaranteed
guarantee	guaranteed
guarantee	guaranteeing
guarantee	guarantees
guarantee	guarantee
guard	guarded
guard	guarded
guard	guarding
guard	guards
guard	guard
guest	guested
guest	guested
guest	guesting
guest	guests
guest	guest
guide	guided
guide	guided
guide	guides
guide	guide
guide	guiding
guilt	guilted
guilt	guilted
guilt	guilting
guilt	guilts
guilt	guilt
guitar	guitared
guitar	guitared
guitar	guitaring
guitar	guitars
guitar	guitar
gun	gunned
gun	gunned
gun	gunning
gun	guns
gun	gun
guy	guyed
guy	guyed
guy	guying
guy	guys
guy	guy
gym	gymed
gym	gymed
gym	gyming
gym	gyms
gym	gym
habit	habited
habit	habited
habit	habiting
habit	habits
habit	habit
half	half
half	halved
half	halved
half	halves
half	halving
halt	halted
halt	halted
halt	halting
halt	halts
halt	halt
ham	hammed
ham	hammed
ham	hamming
ham	hams
ham	ham
hammer	hammered
hammer	hammered
hammer	hammering
hammer	hammers
hammer	hammer
handball	handballed
handball	handballed
handball	handballing
handball	handballs
handball	handball
hand	handed
hand	handed
hand	handing
hand	hands
hand	hand
handicap	handicapped
handicap	handicapped
handicap	handicapping
handicap	handicaps
handicap	handicap
handle	handled
handle	handled
handle	handles
handle	handle
handle	handling
hang	hanged
hang	hanged
hang	hanging
hang	hangs
hang	hang
hang	hung
hang	hung
hank	hanked
hank	hanked
hank	hanking
hank	hanks
hank	hank
happen	happened
happen	happened
happen	happening
happen	happens
happen	happen
harbor	harbored
harbor	harbored
harbor	harboring
harbor	harbors
harbor	harbor
harbour	harboured
harbour	harboured
harbour	harbouring
harbour	harbours
harbour	harbour
harm	harmed
harm	harmed
harm	harming
harm	harms
harm	harm
harry	harried
harry	harried
harry	harries
harry	harrying
harry	harry
harsh	harshed
harsh	harshed
harsh	harshes
harsh	harshing
harsh	harsh
harvest	harvested
harvest	harvested
harvest	harvesting
harvest	harvests
harvest	harvest
hatch	hatched
hatch	hatched
hatch	hatches
hatch	hatching
hatch	hatch
hate	hated
hate	hated
hate	hates
hate	hate
hate	hating
have	had
have	had
have	has
have	have
have	having
haven	havened
haven	havened
haven	havening
haven	havens
haven	haven
hawk	hawked
hawk	hawked
hawk	hawking
hawk	hawks
hawk	hawk
hay	hayed
hay	hayed
hay	haying
hay	hays
hay	hay
head	headed
head	headed
head	heading
head	heads
head	head
hear	heard
hear	heard
hear	hearing
hear	hears
hear	hear
heart	hearted
heart	hearted
heart	hearting
heart	hearts
heart	heart
heat	heated
heat	heated
heat	heating
heat	heats
heat	heat
heaven	heavened
heaven	heavened
heaven	heavening
heaven	heavens
heaven	heaven
heavy	heavied
heavy	heavied
heavy	heavies
heavy	heavying
heavy	heavy
helicopter	helicoptered
helicopter	helicoptered
helicopter	helicoptering
helicopter	helicopters
helicopter	helicopter
help	helped
help	helped
help	helping
help	helps
help	help
help	holpen
help	holp
hence	henced
hence	henced
hence	hences
hence	hence
hence	hencing
herald	heralded
herald	heralded
herald	heralding
herald	heralds
herald	herald
hide	hidden
hide	hided
hide	hided
hide	hides
hide	hide
hide	hiding
hide	hid
hide	hid
high	highed
high	highed
high	highing
high	highs
high	high
highlight	highlighted
highlight	highlighted
highlight	highlighting
highlight	highlights
highlight	highlight
hill	hilled
hill	hilled
hill	hilling
hill	hills
hill	hill
hip	hipped
hip	hipped
hip	hipping
hip	hips
hip	hip
hire	hired
hire	hired
hire	hires
hire	hire
hire	hiring
history	historied
history	historied
history	histories
history	historying
history	history
hit	hits
hit	hitting
hit	hit
hit	hit
hit	hit
hold	held
hold	held
hold	holden
hold	holding
hold	holds
hold	hold
hole	holed
hole	holed
hole	holes
hole	hole
hole	holing
holiday	holidayed
holiday	holidayed
holiday	holidaying
holiday	holidays
holiday	holiday
hollow	hollowed
hollow	hollowed
hollow	hollowing
hollow	hollows
hollow	hollow
homage	homaged
homage	homaged
homage	homages
homage	homage
homage	homaging
home	homed
home	homed
home	homes
home	home
home	homing
homer	homered
homer	homered
homer	homering
homer	homers
homer	homer
homestead	homesteaded
homestead	homesteaded
homestead	homesteading
homestead	homesteads
homestead	homestead
honest	honested
honest	honested
honest	honesting
honest	honests
honest	honest
honey	honeyed
honey	honeyed
honey	honeying
honey	honeys
honey	honey
honor	honored
honor	honored
honor	honoring
honor	honors
honor	honor
honour	honoured
honour	honoured
honour	honouring
honour	honours
honour	honour
hood	hooded
hood	hooded
hood	hooding
hood	hoods
hood	hood
hook	hooked
hook	hooked
hook	hooking
hook	hooks
hook	hook
hope	hoped
hope	hoped
hope	hopes
hope	hope
hope	hoping
hop	hopped
hop	hopped
hop	hopping
hop	hops
hop	hop
horn	horned
horn	horned
horn	horning
horn	horns
horn	horn
horse	horsed
horse	horsed
horse	horses
horse	horse
horse	horsing
host	hosted
host	hosted
host	hosting
host	hosts
host	host
hot	hots
hot	hotted
hot	hotted
hot	hotting
hot	hot
house	housed
house	housed
house	houses
house	house
house	housing
hull	hulled
hull	hulled
hull	hulling
hull	hulls
hull	hull
human	humaned
human	humaned
human	humaning
human	humans
human	human
humor	humored
humor	humored
humor	humoring
humor	humors
humor	humor
hunger	hungered
hunger	hungered
hunger	hungering
hunger	hungers
hunger	hunger
hunt	hunted
hunt	hunted
hunt	hunting
hunt	hunts
hunt	hunt
hurt	hurting
hurt	hurts
hurt	hurt
hurt	hurt
hurt	hurt
husband	husbanded
husband	husbanded
husband	husbanding
husband	husbands
husband	husband
hymn	hymned
hymn	hymned
hymn	hymning
hymn	hymns
hymn	hymn
ice	iced
ice	iced
ice	ices
ice	ice
ice	icing
identify	identified
identify	identified
identify	identifies
identify	identifying
identify	identify
image	imaged
image	imaged
image	images
image	image
image	imaging
immune	immuned
immune	immuned
immune	immunes
immune	immune
immune	immuning
impact	impacted
impact	impacted
impact	impacting
impact	impacts
impact	impact
implement	implemented
implement	implemented
implement	implementing
implement	implements
implement	implement
import	imported
import	imported
import	importing
import	imports
import	import
imprint	imprinted
imprint	imprinted
imprint	imprinting
imprint	imprints
imprint	imprint
improve	improved
improve	improved
improve	improves
improve	improve
improve	improving
inch	inched
inch	inched
inch	inches
inch	inching
inch	inch
include	included
include	included
include	includes
include	include
include	including
incorporate	incorporated
incorporate	incorporated
incorporate	incorporates
incorporate	incorporate
incorporate	incorporating
increase	increased
increase	increased
increase	increases
increase	increase
increase	increasing
index	indexed
index	indexed
index	indexes
index	indexing
index	index
indicate	indicated
indicate	indicated
indicate	indicates
indicate	indicate
indicate	indicating
infant	infanted
infant	infanted
infant	infanting
infant	infants
infant	infant
influence	influenced
influence	influenced
influence	influences
influence	influence
influence	influencing
inform	informed
inform	informed
inform	informing
inform	informs
inform	inform
in	inned
in	inned
in	inning
in	ins
in	in
initial	initialed
initial	initialed
initial	initialing
initial	initialled
initial	initialled
initial	initialling
initial	initials
initial	initial
injury	injuried
injury	injuried
injury	injuries
injury	injurying
injury	injury
ink	inked
ink	inked
ink	inking
ink	inks
ink	ink
inlet	inlets
inlet	inletting
inlet	inlet
inlet	inlet
inlet	inlet
inn	inned
inn	inned
inn	inning
inn	inns
inn	inn
input	inputs
input	inputted
input	inputted
input	inputting
input	input
input	input
input	input
instance	instanced
instance	instanced
instance	instances
instance	instance
instance	instancing
institute	instituted
institute	instituted
institute	institutes
institute	institute
institute	instituting
instrument	instrumented
instrument	instrumented
instrument	instrumenting
instrument	instruments
instrument	instrument
intake	intaken
intake	intakes
intake	intake
intake	intaking
intake	intook
interact	interacted
interact	interacted
interact	interacting
interact	interacts
interact	interact
interchange	interchanged
interchange	interchanged
interchange	interchanges
interchange	interchange
interchange	interchanging
interest	interested
interest	interested
interest	interesting
interest	interests
interest	interest
interface	interfaced
interface	interfaced
interface	interfaces
interface	interface
interface	interfacing
inter	interred
inter	interred
inter	interring
inter	inters
inter	inter
intermediate	intermediated
intermediate	intermediated
intermediate	intermediates
intermediate	intermediate
intermediate	intermediating
interview	interviewed
interview	interviewed
interview	interviewing
interview	interviews
interview	interview
intimate	intimated
intimate	intimated
intimate	intimates
intimate	intimate
intimate	intimating
introduce	introduced
introduce	introduced
introduce	introduces
introduce	introduce
introduce	introducing
inventory	inventoried
inventory	inventoried
inventory	inventories
inventory	inventorying
inventory	inventory
investigate	investigated
investigate	investigated
investigate	investigates
investigate	investigate
investigate	investigating
invest	invested
invest	invested
invest	investing
invest	invests
invest	invest
involve	involved
involve	involved
involve	involves
involve	involve
involve	involving
iris	irised
iris	irised
iris	irises
iris	irising
iris	iris
iron	ironed
iron	ironed
iron	ironing
iron	irons
iron	iron
island	islanded
island	islanded
island	islanding
island	islands
island	island
issue	issued
issue	issued
issue	issues
issue	issue
issue	issuing
jack	jacked
jack	jacked
jack	jacking
jack	jacks
jack	jack
jail	jailed
jail	jailed
jail	jailing
jail	jails
jail	jail
jam	jammed
jam	jammed
jam	jamming
jam	jams
jam	jam
japan	japanned
japan	japanned
japan	japanning
japan	japans
japan	japan
jaw	jawed
jaw	jawed
jaw	jawing
jaw	jaws
jaw	jaw
jazz	jazzed
jazz	jazzed
jazz	jazzes
jazz	jazzing
jazz	jazz
jeff	jeffed
jeff	jeffed
jeff	jeffing
jeff	jeffs
jeff	jeff
jet	jets
jet	jetted
jet	jetted
jet	jetting
jet	jet
jimmy	jimmied
jimmy	jimmied
jimmy	jimmies
jimmy	jimmying
jimmy	jimmy
job	jobbed
job	jobbed
job	jobbing
job	jobs
job	job
jockey	jockeyed
jockey	jockeyed
jockey	jockeying
jockey	jockeys
jockey	jockey
join	joined
join	joined
join	joining
join	joins
join	join
joint	jointed
joint	jointed
joint	jointing
joint	joints
joint	joint
joke	joked
joke	joked
joke	jokes
joke	joke
joke	joking
jones	jonesed
jones	jonesed
jones	joneses
jones	jonesing
jones	jones
josh	joshed
josh	joshed
josh	joshes
josh	joshing
josh	josh
journal	journaled
journal	journaled
journal	journaling
journal	journals
journal	journal
journey	journeyed
journey	journeyed
journey	journeying
journey	journeys
journey	journey
joy	joyed
joy	joyed
joy	joying
joy	joys
joy	joy
judge	judged
judge	judged
judge	judges
judge	judge
judge	judging
juice	juiced
juice	juiced
juice	juices
juice	juice
juice	juicing
jump	jumped
jump	jumped
jump	jumping
jump	jumps
jump	jump
jury	juried
jury	juried
jury	juries
jury	jurying
jury	jury
just	justed
just	justed
just	justing
just	justs
just	just
keen	keened
keen	keened
keen	keening
keen	keens
keen	keen
keep	keeping
keep	keeps
keep	keep
keep	kept
keep	kept
ken	kenned
ken	kenned
ken	kenning
ken	kens
ken	kent
ken	kent
ken	ken
keyboard	keyboarded
keyboard	keyboarded
keyboard	keyboarding
keyboard	keyboards
keyboard	keyboard
key	keyed
key	keyed
key	keying
key	keys
key	key
kick	kicked
kick	kicked
kick	kicking
kick	kicks
kick	kick
kid	kidded
kid	kidded
kid	kidding
kid	kids
kid	kid
kill	killed
kill	killed
kill	killing
kill	kills
kill	kill
king	kinged
king	kinged
king	kinging
king	kings
king	king
kiss	kissed
kiss	kissed
kiss	kisses
kiss	kissing
kiss	kiss
kit	kits
kit	kitted
kit	kitted
kit	kitting
kit	kit
knee	kneed
knee	kneed
knee	kneeing
knee	knees
knee	knee
knife	knifed
knife	knifed
knife	knifes
knife	knife
knife	knifing
knight	knighted
knight	knighted
knight	knighting
knight	knights
knight	knight
know	knew
know	knowing
know	known
know	knows
know	know
knowledge	knowledged
knowledge	knowledged
knowledge	knowledges
knowledge	knowledge
knowledge	knowledging
label	labeled
label	labeled
label	labeling
label	labelled
label	labelled
label	labelling
label	labels
label	label
labor	labored
labor	labored
labor	laboring
labor	labors
labor	labor
labour	laboured
labour	laboured
labour	labouring
labour	labours
labour	labour
lack	lacked
lack	lacked
lack	lacking
lack	lacks
lack	lack
ladder	laddered
ladder	laddered
ladder	laddering
ladder	ladders
ladder	ladder
lake	laked
lake	laked
lake	lakes
lake	lake
lake	laking
lamb	lambed
lamb	lambed
lamb	lambing
lamb	lambs
lamb	lamb
lance	lanced
lance	lanced
lance	lances
lance	lance
lance	lancing
land	landed
land	landed
land	landing
land	lands
land	land
landmark	landmarked
landmark	landmarked
landmark	landmarking
landmark	landmarks
landmark	landmark
landscape	landscaped
landscape	landscaped
landscape	landscapes
landscape	landscape
landscape	landscaping
language	languaged
language	languaged
language	languages
language	language
language	languaging
lap	lapped
lap	lapped
lap	lapping
lap	laps
lap	lap
laser	lasered
laser	lasered
laser	lasering
laser	lasers
laser	laser
last	lasted
last	lasted
last	lasting
last	lasts
last	last
lateral	lateraled
lateral	lateraled
lateral	lateraling
lateral	lateralled
lateral	lateralled
lateral	lateralling
lateral	laterals
lateral	lateral
launch	launched
launch	launched
launch	launches
launch	launching
launch	launch
lawyer	lawyered
lawyer	lawyered
lawyer	lawyering
lawyer	lawyers
lawyer	lawyer
layer	layered
layer	layered
layer	layering
layer	layers
layer	layer
lay	laid
lay	laid
lay	laying
lay	lays
lay	lay
lead	leaded
lead	leaded
lead	leading
lead	leads
lead	lead
lead	led
lead	led
leaf	leafed
leaf	leafed
leaf	leafing
leaf	leafs
leaf	leaf
league	leagued
league	leagued
league	leagues
league	league
league	leaguing
learn	learned
learn	learned
learn	learning
learn	learns
learn	learnt
learn	learnt
learn	learn
lease	leased
lease	leased
lease	leases
lease	lease
lease	leasing
leather	leathered
leather	leathered
leather	leathering
leather	leathers
leather	leather
leave	leaved
leave	leaved
leave	leaves
leave	leave
leave	leaving
leave	left
leave	left
lecture	lectured
lecture	lectured
lecture	lectures
lecture	lecture
lecture	lecturing
legend	legended
legend	legended
legend	legending
legend	legends
legend	legend
legitimate	legitimated
legitimate	legitimated
legitimate	legitimates
legitimate	legitimate
legitimate	legitimating
leg	legged
leg	legged
leg	legging
leg	legs
leg	leg
length	lengthed
length	lengthed
length	lengthing
length	lengths
length	length
lens	lensed
lens	lensed
lens	lenses
lens	lensing
lens	lens
less	lessed
less	lessed
less	lesses
less	lessing
less	less
let	leet
let	lets
let	letted
let	letten
let	letting
let	let
let	let
let	let
letter	lettered
letter	lettered
letter	lettering
letter	letters
letter	letter
level	leveled
level	leveled
level	leveling
level	levelled
level	levelled
level	levelling
level	levels
level	level
levy	levied
levy	levied
levy	levies
levy	levying
levy	levy
licence	licenced
licence	licenced
licence	licences
licence	licence
licence	licencing
license	licensed
license	licensed
license	licenses
license	license
license	licensing
lie	lain
lie	lay
lie	lied
lie	lied
lie	lies
lie	lie
lie	lying
lift	lifted
lift	lifted
lift	lifting
lift	lifts
lift	lift
lighter	lightered
lighter	lightered
lighter	lightering
lighter	lighters
lighter	lighter
light	lighted
light	lighted
light	lighting
light	lights
light	light
light	lit
light	lit
lightning	lightninged
lightning	lightninged
lightning	lightninging
lightning	lightnings
lightning	lightning
like	liked
like	liked
like	likes
like	like
like	liking
limit	limited
limit	limited
limit	limiting
limit	limits
limit	limit
line	lined
line	lined
line	lines
line	line
line	lining
link	linked
link	linked
link	linking
link	links
link	link
lin	lan
lin	linned
lin	linned
lin	linning
lin	lins
lin	lin
lin	lun
listen	listened
listen	listened
listen	listening
listen	listens
listen	listen
list	listed
list	listed
list	listing
list	lists
list	list
list	list
list	list
lit	lits
lit	litted
lit	litted
lit	litting
lit	lit
live	lived
live	lived
live	lives
live	live
live	living
load	loaded
load	loaded
load	loaden
load	loading
load	loads
load	load
loan	loaned
loan	loaned
loan	loaning
loan	loans
loan	loan
lobby	lobbied
lobby	lobbied
lobby	lobbies
lobby	lobbying
lobby	lobby
locate	located
locate	located
locate	locates
locate	locate
locate	locating
lock	locked
lock	locked
lock	locking
lock	locks
lock	lock
lodge	lodged
lodge	lodged
lodge	lodges
lodge	lodge
lodge	lodging
logic	logicked
logic	logicked
logic	logicking
logic	logics
logic	logic
log	logged
log	logged
log	logging
log	logs
log	log
long	longed
long	longed
long	longing
long	longs
long	long
look	looked
look	looked
look	looking
look	looks
look	look
loop	looped
loop	looped
loop	looping
loop	loops
loop	loop
loose	loosed
loose	loosed
loose	looses
loose	loose
loose	loosing
lord	lorded
lord	lorded
lord	lording
lord	lords
lord	lord
lose	loses
lose	lose
lose	losing
lose	lost
lose	lost
lot	lots
lot	lotted
lot	lotted
lot	lotting
lot	lot
lounge	lounged
lounge	lounged
lounge	lounges
lounge	lounge
lounge	lounging
love	loved
love	loved
love	loves
love	love
love	loving
lower	lowered
lower	lowered
lower	lowering
lower	lowers
lower	lower
low	lowed
low	lowed
low	lowing
low	lows
low	low
luck	lucked
luck	lucked
luck	lucking
luck	lucks
luck	luck
lu	lued
lu	lued
lu	luing
lu	lus
lu	lu
lumber	lumbered
lumber	lumbered
lumber	lumbering
lumber	lumbers
lumber	lumber
lunch	lunched
lunch	lunched
lunch	lunches
lunch	lunching
lunch	lunch
lynch	lynched
lynch	lynched
lynch	lynches
lynch	lynching
lynch	lynch
machine	machined
machine	machined
machine	machines
machine	machine
machine	machining
mad	madded
mad	madded
mad	madding
mad	mads
mad	mad
magic	magicked
magic	magicked
magic	magicking
magic	magics
magic	magic
mail	mailed
mail	mailed
mail	mailing
mail	mails
mail	mail
mainstream	mainstreamed
mainstream	mainstreamed
mainstream	mainstreaming
mainstream	mainstreams
mainstream	mainstream
maintain	maintained
maintain	maintained
maintain	maintaining
maintain	maintains
maintain	maintain
major	majored
major	majored
major	majoring
major	majors
major	major
make	made
make	made
make	makes
make	make
make	making
mall	malled
mall	malled
mall	malling
mall	malls
mall	mall
manage	managed
manage	managed
manage	manages
manage	manage
manage	managing
mandate	mandated
mandate	mandated
mandate	mandates
mandate	mandate
mandate	mandating
man	manned
man	manned
man	manning
man	mans
man	man
manufacture	manufactured
manufacture	manufactured
manufacture	manufactures
manufacture	manufacture
manufacture	manufacturing
map	mapped
map	mapped
map	mapping
map	maps
map	map
marble	marbled
marble	marbled
marble	marbles
marble	marble
marble	marbling
marcel	marcelled
marcel	marcelled
marcel	marcelling
marcel	marcels
marcel	marcel
march	marched
march	marched
march	marches
march	marching
march	march
margin	margined
margin	margined
margin	margining
margin	margins
margin	margin
marker	markered
marker	markered
marker	markering
marker	markers
marker	marker
market	marketed
market	marketed
market	marketing
market	markets
market	market
mark	marked
mark	marked
mark	marking
mark	marks
mark	mark
mar	marred
mar	marred
mar	marring
mar	mars
mar	mar
marry	married
marry	married
marry	marries
marry	marrying
marry	marry
marshal	marshaled
marshal	marshaled
marshal	marshaling
marshal	marshalled
marshal	marshalled
marshal	marshalling
marshal	marshals
marshal	marshal
marvel	marveled
marvel	marveled
marvel	marveling
marvel	marvelled
marvel	marvelled
marvel	marvelling
marvel	marvels
marvel	marvel
mask	masked
mask	masked
mask	masking
mask	masks
mask	mask
mason	masoned
mason	masoned
mason	masoning
mason	masons
mason	mason
massacre	massacred
massacre	massacred
massacre	massacres
massacre	massacre
massacre	massacring
mass	massed
mass	massed
mass	masses
mass	massing
mass	mass
master	mastered
master	mastered
master	mastering
master	masters
master	master
match	matched
match	matched
match	matches
match	matching
match	match
mate	mated
mate	mated
mate	mates
mate	mate
mate	mating
material	materialled
material	materialled
material	materialling
material	materials
material	material
matter	mattered
matter	mattered
matter	mattering
matter	matters
matter	matter
mature	matured
mature	matured
mature	matures
mature	mature
mature	maturing
max	maxed
max	maxed
max	maxes
max	maxing
max	max
may	mayed
may	mayed
may	maying
may	mays
may	may
may	may
may	might
meal	mealed
meal	mealed
meal	mealing
meal	meals
meal	meal
mean	meaned
mean	meaned
mean	meaning
mean	means
mean	meant
mean	meant
mean	mean
measure	measured
measure	measured
measure	measures
measure	measure
measure	measuring
medal	medaled
medal	medaled
medal	medaling
medal	medalled
medal	medalled
medal	medalling
medal	medals
medal	medal
medicine	medicined
medicine	medicined
medicine	medicines
medicine	medicine
medicine	medicining
medley	medleyed
medley	medleyed
medley	medleying
medley	medleys
medley	medley
meet	meeting
meet	meets
meet	meet
meet	met
meet	met
member	membered
member	membered
member	membering
member	members
member	member
mention	mentioned
mention	mentioned
mention	mentioning
mention	mentions
mention	mention
mentor	mentored
mentor	mentored
mentor	mentoring
mentor	mentors
mentor	mentor
merchandise	merchandised
merchandise	merchandised
merchandise	merchandises
merchandise	merchandise
merchandise	merchandising
merchant	merchanted
merchant	merchanted
merchant	merchanting
merchant	merchants
merchant	merchant
mere	mered
mere	mered
mere	meres
mere	mere
mere	mering
merge	merged
merge	merged
merge	merges
merge	merge
merge	merging
merit	merited
merit	merited
merit	meriting
merit	merits
merit	merit
message	messaged
message	messaged
message	messages
message	message
message	messaging
messenger	messengered
messenger	messengered
messenger	messengering
messenger	messengers
messenger	messenger
metal	metalled
metal	metalled
metal	metalling
metal	metals
metal	metal
meter	metered
meter	metered
meter	metering
meter	meters
meter	meter
metre	metred
metre	metred
metre	metres
metre	metre
metre	metring
metric	metricked
metric	metricked
metric	metricking
metric	metric
mike	miked
mike	miked
mike	mikes
mike	mike
mike	miking
milk	milked
milk	milked
milk	milking
milk	milks
milk	milk
mill	milled
mill	milled
mill	milling
mill	mills
mill	mill
mind	minded
mind	minded
mind	minding
mind	minds
mind	mind
mine	mined
mine	mined
mine	mines
mine	mine
mine	mining
ming	meint
ming	meint
ming	ment
ming	ment
ming	minged
ming	minged
ming	minging
ming	mings
ming	ming
miniature	miniatured
miniature	miniatured
miniature	miniatures
miniature	miniature
miniature	miniaturing
minister	ministered
minister	ministered
minister	ministering
minister	ministers
minister	minister
min	minned
min	minned
min	minning
min	mins
min	min
minor	minored
minor	minored
minor	minoring
minor	minors
minor	minor
mint	minted
mint	minted
mint	minting
mint	mints
mint	mint
minute	minuted
minute	minuted
minute	minutes
minute	minute
minute	minuting
mirror	mirrored
mirror	mirrored
mirror	mirroring
mirror	mirrors
mirror	mirror
mission	missioned
mission	missioned
mission	missioning
mission	missions
mission	mission
miss	missed
miss	missed
miss	misses
miss	missing
miss	miss
mistake	mistaken
mistake	mistakes
mistake	mistake
mistake	mistaking
mistake	mistook
mix	mixed
mix	mixed
mix	mixes
mix	mixing
mix	mixt
mix	mixt
mix	mix
mob	mobbed
mob	mobbed
mob	mobbing
mob	mobs
mob	mob
model	modeled
model	modeled
model	modeling
model	modelled
model	modelled
model	modelling
model	models
model	model
moderate	moderated
moderate	moderated
moderate	moderates
moderate	moderate
moderate	moderating
monitor	monitored
monitor	monitored
monitor	monitoring
monitor	monitors
monitor	monitor
monkey	monkeyed
monkey	monkeyed
monkey	monkeying
monkey	monkeys
monkey	monkey
monkey	monkied
monkey	monkied
monster	monstered
monster	monstered
monster	monstering
monster	monsters
monster	monster
moon	mooned
moon	mooned
moon	mooning
moon	moons
moon	moon
more	mored
more	mored
more	mores
more	more
more	moring
mortar	mortared
mortar	mortared
mortar	mortaring
mortar	mortars
mortar	mortar
mortgage	mortgaged
mortgage	mortgaged
mortgage	mortgages
mortgage	mortgage
mortgage	mortgaging
moss	mossed
moss	mossed
moss	mosses
moss	mossing
moss	moss
mother	mothered
mother	mothered
mother	mothering
mother	mothers
mother	mother
moth	mothed
moth	mothed
moth	mothing
moth	moths
moth	moth
motion	motioned
motion	motioned
motion	motioning
motion	motions
motion	motion
motorcycle	motorcycled
motorcycle	motorcycled
motorcycle	motorcycles
motorcycle	motorcycle
motorcycle	motorcycling
motor	motored
motor	motored
motor	motoring
motor	motors
motor	motor
mound	mounded
mound	mounded
mound	mounding
mound	mounds
mound	mound
mount	mounted
mount	mounted
mount	mounting
mount	mounts
mount	mount
mouse	moused
mouse	moused
mouse	mouses
mouse	mouse
mouse	mousing
mouth	mouthed
mouth	mouthed
mouth	mouthing
mouth	mouths
mouth	mouth
move	moved
move	moved
move	moves
move	move
move	moving
mud	mudded
mud	mudded
mud	mudding
mud	muds
mud	mud
murder	murdered
murder	murdered
murder	murdering
murder	murders
murder	murder
muscle	muscled
muscle	muscled
muscle	muscles
muscle	muscle
muscle	muscling
music	musicked
music	musicked
music	musicking
music	musics
music	music
must	musted
must	musted
must	musting
must	musts
must	must
name	named
name	named
name	names
name	name
name	naming
namesake	namesaked
namesake	namesaked
namesake	namesakes
namesake	namesake
namesake	namesaking
narrow	narrowed
narrow	narrowed
narrow	narrowing
narrow	narrows
narrow	narrow
nature	natured
nature	natured
nature	natures
nature	nature
nature	naturing
near	neared
near	neared
near	nearing
near	nears
near	near
neck	necked
neck	necked
neck	necking
neck	necks
neck	neck
need	needed
need	needed
need	needing
need	needs
need	need
negative	negatived
negative	negatived
negative	negatives
negative	negative
negative	negativing
negotiate	negotiated
negotiate	negotiated
negotiate	negotiates
negotiate	negotiate
negotiate	negotiating
neighbor	neighbored
neighbor	neighbored
neighbor	neighboring
neighbor	neighbors
neighbor	neighbor
nerve	nerved
nerve	nerved
nerve	nerves
nerve	nerve
nerve	nerving
nest	nested
nest	nested
nest	nesting
nest	nests
nest	nest
net	nets
net	netted
net	netted
net	netting
net	net
network	networked
network	networked
network	networking
network	networks
network	network
new	newed
new	newed
new	newing
new	news
new	new
newspaper	newspapered
newspaper	newspapered
newspaper	newspapering
newspaper	newspapers
newspaper	newspaper
nice	niced
nice	niced
nice	nices
nice	nice
nice	nicing
nickname	nicknamed
nickname	nicknamed
nickname	nicknames
nickname	nickname
nickname	nicknaming
nick	nicked
nick	nicked
nick	nicking
nick	nicks
nick	nick
nightclub	nightclubbed
nightclub	nightclubbed
nightclub	nightclubbing
nightclub	nightclubs
nightclub	nightclub
night	nighted
night	nighted
night	nighting
night	nights
night	night
noise	noised
noise	noised
noise	noises
noise	noise
noise	noising
noon	nooned
noon	nooned
noon	nooning
noon	noons
noon	noon
norm	normed
norm	normed
norm	norming
norm	norms
norm	norm
north	northed
north	northed
north	northing
north	norths
north	north
nose	nosed
nose	nosed
nose	noses
nose	nose
nose	nosing
note	noted
note	noted
note	notes
note	note
note	noting
notice	noticed
notice	noticed
notice	notices
notice	notice
notice	noticing
noun	nouned
noun	nouned
noun	nouning
noun	nouns
noun	noun
number	numbered
number	numbered
number	numbering
number	numbers
number	number
nurse	nursed
nurse	nursed
nurse	nurses
nurse	nurse
nurse	nursing
oath	oathed
oath	oathed
oath	oathing
oath	oaths
oath	oath
object	objected
object	objected
object	objecting
object	objects
object	object
obscure	obscured
obscure	obscured
obscure	obscures
obscure	obscure
obscure	obscuring
observe	observed
observe	observed
observe	observes
observe	observe
observe	observing
obsolete	obsoleted
obsolete	obsoleted
obsolete	obsoletes
obsolete	obsolete
obsolete	obsoleting
obtain	obtained
obtain	obtained
obtain	obtaining
obtain	obtains
obtain	obtain
occasion	occasioned
occasion	occasioned
occasion	occasioning
occasion	occasions
occasion	occasion
occupy	occupied
occupy	occupied
occupy	occupies
occupy	occupying
occupy	occupy
occur	occurred
occur	occurred
occur	occurring
occur	occurs
occur	occur
offer	offered
offer	offered
offer	offering
offer	offers
offer	offer
officer	officered
officer	officered
officer	officering
officer	officers
officer	officer
off	offed
off	offed
off	offing
off	offs
off	off
offshore	offshored
offshore	offshored
offshore	offshores
offshore	offshore
offshore	offshoring
oh	ohed
oh	ohed
oh	ohing
oh	ohs
oh	oh
oil	oiled
oil	oiled
oil	oiling
oil	oils
oil	oil
one	oned
one	oned
one	ones
one	one
one	oning
on	oned
on	oned
on	oning
on	ons
on	on
onset	onsets
onset	onsetting
onset	onset
onset	onset
onset	onset
open	opened
open	opened
open	opening
open	opens
open	open
operate	operated
operate	operated
operate	operates
operate	operate
operate	operating
opinion	opinioned
opinion	opinioned
opinion	opinioning
opinion	opinions
opinion	opinion
op	opped
op	opped
op	opping
op	ops
op	op
oppose	opposed
oppose	opposed
oppose	opposes
oppose	oppose
oppose	opposing
option	optioned
option	optioned
option	optioning
option	options
option	option
orange	oranged
orange	oranged
orange	oranges
orange	orange
orange	oranging
orbit	orbited
orbit	orbited
orbit	orbiting
orbit	orbits
orbit	orbit
order	ordered
order	ordered
order	ordering
order	orders
order	order
organize	organized
organize	organized
organize	organizes
organize	organize
organize	organizing
organ	organed
organ	organed
organ	organing
organ	organs
organ	organ
other	othered
other	othered
other	othering
other	others
other	other
outbreak	outbreaking
outbreak	outbreaks
outbreak	outbreak
outbreak	outbroken
outbreak	outbroke
outfit	outfits
outfit	outfitted
outfit	outfitted
outfit	outfitting
outfit	outfit
outline	outlined
outline	outlined
outline	outlines
outline	outline
outline	outlining
out	outed
out	outed
out	outing
out	outs
out	out
output	outputs
output	outputted
output	outputted
output	outputting
output	output
output	output
output	output
outreach	outreached
outreach	outreached
outreach	outreaches
outreach	outreaching
outreach	outreach
outright	outrighted
outright	outrighted
outright	outrighting
outright	outrights
outright	outright
overcome	overcame
overcome	overcomes
overcome	overcome
overcome	overcome
overcome	overcoming
overnight	overnighted
overnight	overnighted
overnight	overnighting
overnight	overnights
overnight	overnight
overview	overviewed
overview	overviewed
overview	overviewing
overview	overviews
overview	overview
own	owned
own	owned
own	owning
own	owns
own	own
pace	paced
pace	paced
pace	paces
pace	pace
pace	pacing
package	packaged
package	packaged
package	packages
package	package
package	packaging
pack	packed
pack	packed
pack	packing
pack	packs
pack	pack
pageant	pageanted
pageant	pageanted
pageant	pageanting
pageant	pageants
pageant	pageant
page	paged
page	paged
page	pages
page	page
page	paging
pain	pained
pain	pained
pain	paining
pain	pains
pain	pain
paint	painted
paint	painted
paint	painting
paint	paints
paint	paint
pair	paired
pair	paired
pair	pairing
pair	pairs
pair	pair
palace	palaced
palace	palaced
palace	palaces
palace	palace
palace	palacing
pale	paled
pale	paled
pale	pales
pale	pale
pale	paling
palm	palmed
palm	palmed
palm	palming
palm	palms
palm	palm
panel	paneled
panel	paneled
panel	paneling
panel	panelled
panel	panelled
panel	panelling
panel	panels
panel	panel
panic	panicked
panic	panicked
panic	panicking
panic	panics
panic	panic
pan	panned
pan	panned
pan	panning
pan	pans
pan	pan
paper	papered
paper	papered
paper	papering
paper	papers
paper	paper
parachute	parachuted
parachute	parachuted
parachute	parachutes
parachute	parachute
parachute	parachuting
parade	paraded
parade	paraded
parade	parades
parade	parade
parade	parading
paradise	paradised
paradise	paradised
paradise	paradises
paradise	paradise
paradise	paradising
parallel	paralleled
parallel	paralleled
parallel	paralleling
parallel	parallelled
parallel	parallelled
parallel	parallelling
parallel	parallels
parallel	parallel
parent	parented
parent	parented
parent	parenting
parent	parents
parent	parent
parish	parished
parish	parished
parish	parishes
parish	parishing
parish	parish
park	parked
park	parked
park	parking
park	parks
park	park
parody	parodied
parody	parodied
parody	parodies
parody	parodying
parody	parody
par	parred
par	parred
par	parring
par	pars
par	par
participate	participated
participate	participated
participate	participates
participate	participate
participate	participating
partition	partitioned
partition	partitioned
partition	partitioning
partition	partitions
partition	partition
partner	partnered
partner	partnered
partner	partnering
partner	partners
partner	partner
part	parted
part	parted
part	parting
part	parts
part	part
party	partied
party	partied
party	parties
party	partying
party	party
passage	passaged
passage	passaged
passage	passages
passage	passage
passage	passaging
passenger	passengered
passenger	passengered
passenger	passengering
passenger	passengers
passenger	passenger
passion	passioned
passion	passioned
passion	passioning
passion	passions
passion	passion
pass	passed
pass	passed
pass	passes
pass	passing
pass	pass
pastor	pastored
pastor	pastored
pastor	pastoring
pastor	pastors
pastor	pastor
patch	patched
patch	patched
patch	patches
patch	patching
patch	patch
patent	patented
patent	patented
patent	patenting
patent	patents
patent	patent
path	pathed
path	pathed
path	pathing
path	paths
path	path
pat	pats
pat	patted
pat	patted
pat	patting
pat	pat
patrol	patrolled
patrol	patrolled
patrol	patrolling
patrol	patrols
patrol	patrol
patronage	patronaged
patronage	patronaged
patronage	patronages
patronage	patronage
patronage	patronaging
patron	patroned
patron	patroned
patron	patroning
patron	patrons
patron	patron
pattern	patterned
pattern	patterned
pattern	patterning
pattern	patterns
pattern	pattern
pavilion	pavilioned
pavilion	pavilioned
pavilion	pavilioning
pavilion	pavilions
pavilion	pavilion
pay	paid
pay	paid
pay	payed
pay	payed
pay	paying
pay	pays
pay	pay
peace	peaced
peace	peaced
peace	peaces
peace	peace
peace	peacing
peak	peaked
peak	peaked
peak	peaking
peak	peaks
peak	peak
pearl	pearled
pearl	pearled
pearl	pearling
pearl	pearls
pearl	pearl
peer	peered
peer	peered
peer	peering
peer	peers
peer	peer
penny	pennied
penny	pennied
penny	pennies
penny	pennying
penny	penny
pen	penned
pen	penned
pen	penning
pen	pens
pen	pent
pen	pent
pen	pen
pension	pensioned
pension	pensioned
pension	pensioning
pension	pensions
pension	pension
people	peopled
people	peopled
people	peoples
people	people
people	peopling
pepper	peppered
pepper	peppered
pepper	peppering
pepper	peppers
pepper	pepper
perfect	perfected
perfect	perfected
perfect	perfecting
perfect	perfects
perfect	perfect
perform	performed
perform	performed
perform	performing
perform	performs
perform	perform
period	perioded
period	perioded
period	perioding
period	periods
period	period
permanent	permanented
permanent	permanented
permanent	permanenting
permanent	permanents
permanent	permanent
permit	permits
permit	permitted
permit	permitted
permit	permitting
permit	permit
person	personed
person	personed
person	personing
person	persons
person	person
persuade	persuaded
persuade	persuaded
persuade	persuades
persuade	persuade
persuade	persuading
peter	petered
peter	petered
peter	petering
peter	peters
peter	peter
petition	petitioned
petition	petitioned
petition	petitioning
petition	petitions
petition	petition
pet	pets
pet	petted
pet	petted
pet	petting
pet	pet
phase	phased
phase	phased
phase	phases
phase	phase
phase	phasing
philosophy	philosophied
philosophy	philosophied
philosophy	philosophies
philosophy	philosophying
philosophy	philosophy
phone	phoned
phone	phoned
phone	phones
phone	phone
phone	phoning
photograph	photographed
photograph	photographed
photograph	photographing
photograph	photographs
photograph	photograph
photo	photoed
photo	photoed
photo	photoing
photo	photos
photo	photo
phrase	phrased
phrase	phrased
phrase	phrases
phrase	phrase
phrase	phrasing
pick	picked
pick	picked
pick	picking
pick	picks
pick	pick
picture	pictured
picture	pictured
picture	pictures
picture	picture
picture	picturing
piece	pieced
piece	pieced
piece	pieces
piece	piece
piece	piecing
piece	together
piece	together
pierce	pierced
pierce	pierced
pierce	pierces
pierce	pierce
pierce	piercing
pig	pigged
pig	pigged
pig	pigging
pig	pigs
pig	pig
pike	piked
pike	piked
pike	pikes
pike	pike
pike	piking
pilgrimage	pilgrimaged
pilgrimage	pilgrimaged
pilgrimage	pilgrimages
pilgrimage	pilgrimage
pilgrimage	pilgrimaging
pilot	piloted
pilot	piloted
pilot	piloting
pilot	pilots
pilot	pilot
pine	pined
pine	pined
pine	pines
pine	pine
pine	pining
pink	pinked
pink	pinked
pink	pinking
pink	pinks
pink	pink
pin	pinned
pin	pinned
pin	pinning
pin	pins
pin	pin
pioneer	pioneered
pioneer	pioneered
pioneer	pioneering
pioneer	pioneers
pioneer	pioneer
pipeline	pipelined
pipeline	pipelined
pipeline	pipelines
pipeline	pipeline
pipeline	pipelining
pipe	piped
pipe	piped
pipe	pipes
pipe	pipe
pipe	piping
pirate	pirated
pirate	pirated
pirate	pirates
pirate	pirate
pirate	pirating
pistol	pistoled
pistol	pistoled
pistol	pistoling
pistol	pistols
pistol	pistol
pitch	pitched
pitch	pitched
pitch	pitches
pitch	pitching
pitch	pitch
pit	pits
pit	pitted
pit	pitted
pit	pitting
pit	pit
place	placed
place	placed
place	places
place	place
place	placing
plague	plagued
plague	plagued
plague	plagues
plague	plague
plague	plaguing
plain	plained
plain	plained
plain	plaining
plain	plains
plain	plain
plane	planed
plane	planed
plane	planes
plane	plane
plane	planing
plan	planned
plan	planned
plan	planning
plan	plans
plan	plan
plant	planted
plant	planted
plant	planting
plant	plants
plant	plant
plateau	plateaued
plateau	plateaued
plateau	plateauing
plateau	plateaus
plateau	plateau
plate	plated
plate	plated
plate	plates
plate	plate
plate	plating
platform	platformed
platform	platformed
platform	platforming
platform	platforms
platform	platform
platoon	platooned
platoon	platooned
platoon	platooning
platoon	platoons
platoon	platoon
play	played
play	played
play	playing
play	plays
play	play
please	pleased
please	pleased
please	pleases
please	please
please	pleasing
pleasure	pleasured
pleasure	pleasured
pleasure	pleasures
pleasure	pleasure
pleasure	pleasuring
plot	plots
plot	plotted
plot	plotted
plot	plotting
plot	plot
plus	plused
plus	plused
plus	pluses
plus	plusing
plus	plussed
plus	plussed
plus	plusses
plus	plussing
plus	plus
pocket	pocketed
pocket	pocketed
pocket	pocketing
pocket	pockets
pocket	pocket
podcast	podcasting
podcast	podcasts
podcast	podcast
podcast	podcast
podcast	podcast
podium	podiumed
podium	podiumed
podium	podiuming
podium	podiums
podium	podium
point	pointed
point	pointed
point	pointing
point	points
point	point
poison	poisoned
poison	poisoned
poison	poisoning
poison	poisons
poison	poison
pole	poled
pole	poled
pole	poles
pole	pole
pole	poling
police	policed
police	policed
police	polices
police	police
police	policing
policy	policied
policy	policied
policy	policies
policy	policying
policy	policy
polish	polished
polish	polished
polish	polishes
polish	polishing
polish	polish
poll	polled
poll	polled
poll	polling
poll	polls
poll	poll
pond	ponded
pond	ponded
pond	ponding
pond	ponds
pond	pond
pool	pooled
pool	pooled
pool	pooling
pool	pools
pool	pool
pope	poped
pope	poped
pope	popes
pope	pope
pope	poping
pop	popped
pop	popped
pop	popping
pop	pops
pop	pop
porter	portered
porter	portered
porter	portering
porter	porters
porter	porter
portion	portioned
portion	portioned
portion	portioning
portion	portions
portion	portion
port	ported
port	ported
port	porting
port	ports
port	port
portrait	portraited
portrait	portraited
portrait	portraiting
portrait	portraits
portrait	portrait
position	positioned
position	positioned
position	positioning
position	positions
position	position
possession	possessioned
possession	possessioned
possession	possessioning
possession	possessions
possession	possession
possess	possessed
possess	possessed
possess	possesses
possess	possessing
possess	possess
post	posted
post	posted
post	posting
post	posts
post	post
pot	pots
pot	potted
pot	potted
pot	potting
pot	pot
potter	pottered
potter	pottered
potter	pottering
potter	potters
potter	potter
pound	pounded
pound	pounded
pound	pounding
pound	pounds
pound	pound
powder	powdered
powder	powdered
powder	powdering
powder	powders
powder	powder
power	powered
power	powered
power	powering
power	powers
power	power
practice	practiced
practice	practiced
practice	practices
practice	practice
practice	practicing
praise	praised
praise	praised
praise	praises
praise	praise
praise	praising
preference	preferenced
preference	preferenced
preference	preferences
preference	preference
preference	preferencing
prefer	preferred
prefer	preferred
prefer	preferring
prefer	prefers
prefer	prefer
premiere	premiered
premiere	premiered
premiere	premieres
premiere	premiere
premiere	premiering
premier	premiered
premier	premiered
premier	premiering
premier	premiers
premier	premier
premise	premised
premise	premised
premise	premises
premise	premise
premise	premising
prepare	prepared
prepare	prepared
prepare	prepares
prepare	prepare
prepare	preparing
preseason	preseasoned
preseason	preseasoned
preseason	preseasoning
preseason	preseasons
preseason	preseason
presence	presenced
presence	presenced
presence	presences
presence	presence
presence	presencing
present	presented
present	presented
present	presenting
present	presents
present	present
preserve	preserved
preserve	preserved
preserve	preserves
preserve	preserve
preserve	preserving
press	pressed
press	pressed
press	presses
press	pressing
press	press
press	prest
press	prest
pressure	pressured
pressure	pressured
pressure	pressures
pressure	pressure
pressure	pressuring
pretty	prettied
pretty	prettied
pretty	pretties
pretty	prettying
pretty	pretty
prevent	prevented
prevent	prevented
prevent	preventing
prevent	prevents
prevent	prevent
price	priced
price	priced
price	prices
price	price
price	pricing
pride	prided
pride	prided
pride	prides
pride	pride
pride	priding
priest	priested
priest	priested
priest	priesting
priest	priests
priest	priest
primary	primaried
primary	primaried
primary	primaries
primary	primarying
primary	primary
prime	primed
prime	primed
prime	primes
prime	prime
prime	priming
principle	principled
principle	principled
principle	principles
principle	principle
principle	principling
print	printed
print	printed
print	printing
print	prints
print	print
prison	prisoned
prison	prisoned
prison	prisoning
prison	prisons
prison	prison
privilege	privileged
privilege	privileged
privilege	privileges
privilege	privilege
privilege	privileging
prize	prized
prize	prized
prize	prizes
prize	prize
prize	prizing
proceed	proceeded
proceed	proceeded
proceed	proceeding
proceed	proceeds
proceed	proceed
procession	processioned
procession	processioned
procession	processioning
procession	processions
procession	procession
process	processed
process	processed
process	processes
process	processing
process	process
produce	produced
produce	produced
produce	produces
produce	produce
produce	producing
profile	profiled
profile	profiled
profile	profiles
profile	profile
profile	profiling
profit	profited
profit	profited
profit	profiting
profit	profits
profit	profit
profound	profounded
profound	profounded
profound	profounding
profound	profounds
profound	profound
programme	programmed
programme	programmed
programme	programmes
programme	programme
programme	programming
program	programmed
program	programmed
program	programming
program	programs
program	program
progress	progressed
progress	progressed
progress	progresses
progress	progressing
progress	progress
project	projected
project	projected
project	projecting
project	projects
project	project
promise	promised
promise	promised
promise	promises
promise	promise
promise	promising
promote	promoted
promote	promoted
promote	promotes
promote	promote
promote	promoting
proof	proofed
proof	proofed
proof	proofing
proof	proofs
proof	proof
property	propertied
property	propertied
property	properties
property	propertying
property	property
proportion	proportioned
proportion	proportioned
proportion	proportioning
proportion	proportions
proportion	proportion
proposition	propositioned
proposition	propositioned
proposition	propositioning
proposition	propositions
proposition	proposition
prose	prosed
prose	prosed
prose	proses
prose	prose
prose	prosing
prospect	prospected
prospect	prospected
prospect	prospecting
prospect	prospects
prospect	prospect
protect	protected
protect	protected
protect	protecting
protect	protects
protect	protect
protest	protested
protest	protested
protest	protesting
protest	protests
protest	protest
protocol	protocoled
protocol	protocoled
protocol	protocoling
protocol	protocolled
protocol	protocolled
protocol	protocolling
protocol	protocols
protocol	protocol
prototype	prototyped
prototype	prototyped
prototype	prototypes
prototype	prototype
prototype	prototyping
prove	proved
prove	proved
prove	proven
prove	proves
prove	prove
prove	proving
provide	provided
provide	provided
provide	provides
provide	provide
provide	providing
provision	provisioned
provision	provisioned
provision	provisioning
provision	provisions
provision	provision
publish	published
publish	published
publish	publishes
publish	publishing
publish	publish
pub	pubbed
pub	pubbed
pub	pubbing
pub	pubs
pub	pub
pull	pulled
pull	pulled
pull	pulling
pull	pulls
pull	pull
pulse	pulsed
pulse	pulsed
pulse	pulses
pulse	pulse
pulse	pulsing
pump	pumped
pump	pumped
pump	pumping
pump	pumps
pump	pump
punch	punched
punch	punched
punch	punches
punch	punching
punch	punch
punk	punked
punk	punked
punk	punking
punk	punks
punk	punk
punt	punted
punt	punted
punt	punting
punt	punts
punt	punt
purchase	purchased
purchase	purchased
purchase	purchases
purchase	purchase
purchase	purchasing
purple	purpled
purple	purpled
purple	purples
purple	purple
purple	purpling
purpose	purposed
purpose	purposed
purpose	purposes
purpose	purpose
purpose	purposing
pursue	pursued
pursue	pursued
pursue	pursues
pursue	pursue
pursue	pursuing
push	pushed
push	pushed
push	pushes
push	pushing
push	push
put	puts
put	putting
put	put
put	put
put	put
pyramid	pyramided
pyramid	pyramided
pyramid	pyramiding
pyramid	pyramids
pyramid	pyramid
qualify	qualified
qualify	qualified
qualify	qualifies
qualify	qualifying
qualify	qualify
quarry	quarried
quarry	quarried
quarry	quarries
quarry	quarrying
quarry	quarry
quarterback	quarterbacked
quarterback	quarterbacked
quarterback	quarterbacking
quarterback	quarterbacks
quarterback	quarterback
quarter	quartered
quarter	quartered
quarter	quartering
quarter	quarters
quarter	quarter
queen	queened
queen	queened
queen	queening
queen	queens
queen	queen
question	questioned
question	questioned
question	questioning
question	questions
question	question
quest	quested
quest	quested
quest	questing
quest	quests
quest	quest
quick	quicked
quick	quicked
quick	quicking
quick	quicks
quick	quick
quiet	quieted
quiet	quieted
quiet	quieting
quiet	quiets
quiet	quiet
quit	quits
quit	quitted
quit	quitted
quit	quitting
quit	quit
quit	quit
quit	quit
rabbit	rabbited
rabbit	rabbited
rabbit	rabbiting
rabbit	rabbits
rabbit	rabbitted
rabbit	rabbitted
rabbit	rabbitting
rabbit	rabbit
race	raced
race	raced
race	races
race	race
race	racing
radar	radared
radar	radared
radar	radaring
radar	radars
radar	radar
radio	radioed
radio	radioed
radio	radioing
radio	radios
radio	radio
rage	raged
rage	raged
rage	rages
rage	rage
rage	raging
raid	raided
raid	raided
raid	raiding
raid	raids
raid	raid
rail	railed
rail	railed
rail	railing
rail	rails
rail	rail
railroad	railroaded
railroad	railroaded
railroad	railroading
railroad	railroads
railroad	railroad
rainbow	rainbowed
rainbow	rainbowed
rainbow	rainbowing
rainbow	rainbows
rainbow	rainbow
rain	rained
rain	rained
rain	raining
rain	rains
rain	rain
raise	raised
raise	raised
raise	raises
raise	raise
raise	raising
rally	rallied
rally	rallied
rally	rallies
rally	rallying
rally	rally
ralph	ralphed
ralph	ralphed
ralph	ralphing
ralph	ralphs
ralph	ralph
ram	rammed
ram	rammed
ram	ramming
ram	rams
ram	ram
ranch	ranched
ranch	ranched
ranch	ranches
ranch	ranching
ranch	ranch
range	ranged
range	ranged
range	ranges
range	range
range	ranging
rank	ranked
rank	ranked
rank	ranking
rank	ranks
rank	rank
rape	raped
rape	raped
rape	rapes
rape	rape
rape	raping
rap	rapped
rap	rapped
rap	rapping
rap	raps
rap	rap
rare	rared
rare	rared
rare	rares
rare	rare
rare	raring
rate	rated
rate	rated
rate	rates
rate	rate
rate	rating
rather	rathered
rather	rathered
rather	rathering
rather	rathers
rather	rather
rat	rats
rat	ratted
rat	ratted
rat	ratting
rat	rat
ray	rayed
ray	rayed
ray	raying
ray	rays
ray	ray
reach	reached
reach	reached
reach	reaches
reach	reaching
reach	reach
read	readen
read	reading
read	reads
read	read
read	read
read	read
ready	readied
ready	readied
ready	readies
ready	readying
ready	ready
realize	realized
realize	realized
realize	realizes
realize	realize
realize	realizing
rear	reared
rear	reared
rear	rearing
rear	rears
rear	rear
reason	reasoned
reason	reasoned
reason	reasoning
reason	reasons
reason	reason
rebel	rebelled
rebel	rebelled
rebel	rebelling
rebel	rebels
rebel	rebel
rebuild	rebuilding
rebuild	rebuilds
rebuild	rebuild
rebuild	rebuilt
rebuild	rebuilt
recall	recalled
recall	recalled
recall	recalling
recall	recalls
recall	recall
receive	received
receive	received
receive	receives
receive	receive
receive	receiving
recognize	recognized
recognize	recognized
recognize	recognizes
recognize	recognize
recognize	recognizing
record	recorded
record	recorded
record	recording
record	records
record	record
recover	recovered
recover	recovered
recover	recovering
recover	recovers
recover	recover
recruit	recruited
recruit	recruited
recruit	recruiting
recruit	recruits
recruit	recruit
red	redded
red	redded
red	redding
red	reds
red	red
reduce	reduced
reduce	reduced
reduce	reduces
reduce	reduce
reduce	reducing
reed	reeded
reed	reeded
reed	reeding
reed	reeds
reed	reed
reef	reefed
reef	reefed
reef	reefing
reef	reefs
reef	reef
referee	refereed
referee	refereed
referee	refereeing
referee	referees
referee	referee
reference	referenced
reference	referenced
reference	references
reference	reference
reference	referencing
refer	referred
refer	referred
refer	referring
refer	refers
refer	refer
reflect	reflected
reflect	reflected
reflect	reflecting
reflect	reflects
reflect	reflect
reform	reformed
reform	reformed
reform	reforming
reform	reforms
reform	reform
ref	reffed
ref	reffed
ref	reffing
ref	refs
ref	ref
refugee	refugeed
refugee	refugeed
refugee	refugeeing
refugee	refugees
refugee	refugee
refuge	refuged
refuge	refuged
refuge	refuges
refuge	refuge
refuge	refuging
refuse	refused
refuse	refused
refuse	refuses
refuse	refuse
refuse	refusing
regain	regained
regain	regained
regain	regaining
regain	regains
regain	regain
regard	regarded
regard	regarded
regard	regarding
regard	regards
regard	regard
regiment	regimented
regiment	regimented
regiment	regimenting
regiment	regiments
regiment	regiment
register	registered
register	registered
register	registering
register	registers
register	register
regulate	regulated
regulate	regulated
regulate	regulates
regulate	regulate
regulate	regulating
reign	reigned
reign	reigned
reign	reigning
reign	reigns
reign	reign
relate	related
relate	related
relate	relates
relate	relate
relate	relating
relay	relaid
relay	relaid
relay	relayed
relay	relayed
relay	relaying
relay	relays
relay	relay
release	released
release	released
release	releases
release	release
release	releasing
rely	relied
rely	relied
rely	relies
rely	relying
rely	rely
remainder	remaindered
remainder	remaindered
remainder	remaindering
remainder	remainders
remainder	remainder
remain	remained
remain	remained
remain	remaining
remain	remains
remain	remain
remake	remade
remake	remade
remake	remakes
remake	remake
remake	remaking
remember	remembered
remember	remembered
remember	remembering
remember	remembers
remember	remember
remix	remixed
remix	remixed
remix	remixes
remix	remixing
remix	remix
remote	remoted
remote	remoted
remote	remotes
remote	remote
remote	remoting
remove	removed
remove	removed
remove	removes
remove	remove
remove	removing
rent	rented
rent	rented
rent	renting
rent	rents
rent	rent
repair	repaired
repair	repaired
repair	repairing
repair	repairs
repair	repair
repeat	repeated
repeat	repeated
repeat	repeating
repeat	repeats
repeat	repeat
replace	replaced
replace	replaced
replace	replaces
replace	replace
replace	replacing
report	reported
report	reported
report	reporting
report	reports
report	report
represent	represented
represent	represented
represent	representing
represent	represents
represent	represent
request	requested
request	requested
request	requesting
request	requests
request	request
require	required
require	required
require	requires
require	require
require	requiring
rescue	rescued
rescue	rescued
rescue	rescues
rescue	rescue
rescue	rescuing
research	researched
research	researched
research	researches
research	researching
research	research
resemble	resembled
resemble	resembled
resemble	resembles
resemble	resemble
resemble	resembling
reserve	reserved
reserve	reserved
reserve	reserves
reserve	reserve
reserve	reserving
reside	resided
reside	resided
reside	resides
reside	reside
reside	residing
resign	resigned
resign	resigned
resign	resigning
resign	resigns
resign	resign
resist	resisted
resist	resisted
resist	resisting
resist	resists
resist	resist
resolve	resolved
resolve	resolved
resolve	resolves
resolve	resolve
resolve	resolving
resort	resorted
resort	resorted
resort	resorting
resort	resorts
resort	resort
resource	resourced
resource	resourced
resource	resources
resource	resource
resource	resourcing
respect	respected
respect	respected
respect	respecting
respect	respects
respect	respect
respond	responded
respond	responded
respond	responding
respond	responds
respond	respond
restore	restored
restore	restored
restore	restores
restore	restore
restore	restoring
rest	rested
rest	rested
rest	resting
rest	rests
rest	rest
result	resulted
result	resulted
result	resulting
result	results
result	result
resume	resumed
resume	resumed
resume	resumes
resume	resume
resume	resuming
retail	retailed
retail	retailed
retail	retailing
retail	retails
retail	retail
retain	retained
retain	retained
retain	retaining
retain	retains
retain	retain
retire	retired
retire	retired
retire	retires
retire	retire
retire	retiring
retreat	retreated
retreat	retreated
retreat	retreating
retreat	retreats
retreat	retreat
return	returned
return	returned
return	returning
return	returns
return	return
reveal	revealed
reveal	revealed
reveal	revealing
reveal	reveals
reveal	reveal
revenge	revenged
revenge	revenged
revenge	revenges
revenge	revenge
revenge	revenging
reverse	reversed
reverse	reversed
reverse	reverses
reverse	reverse
reverse	reversing
review	reviewed
review	reviewed
review	reviewing
review	reviews
review	review
revision	revisioned
revision	revisioned
revision	revisioning
revision	revisions
revision	revision
revolt	revolted
revolt	revolted
revolt	revolting
revolt	revolts
revolt	revolt
rev	revs
rev	revved
rev	revved
rev	revving
rev	rev
reward	rewarded
reward	rewarded
reward	rewarding
reward	rewards
reward	reward
ribbon	ribboned
ribbon	ribboned
ribbon	ribboning
ribbon	ribbons
ribbon	ribbon
rice	riced
rice	riced
rice	rices
rice	rice
rice	ricing
rich	riched
rich	riched
rich	riches
rich	riching
rich	rich
rick	ricked
rick	ricked
rick	ricking
rick	ricks
rick	rick
ride	ridden
ride	rides
ride	ride
ride	riding
ride	rode
ridge	ridged
ridge	ridged
ridge	ridges
ridge	ridge
ridge	ridging
rifle	rifled
rifle	rifled
rifle	rifles
rifle	rifle
rifle	rifling
right	righted
right	righted
right	righting
right	rights
right	right
rim	rimmed
rim	rimmed
rim	rimming
rim	rims
rim	rim
ring	rang
ring	ringed
ring	ringed
ring	ringing
ring	rings
ring	ring
ring	rung
ring	rung
riot	rioted
riot	rioted
riot	rioting
riot	riots
riot	riot
rise	risen
rise	rises
rise	rise
rise	rising
rise	rose
risk	risked
risk	risked
risk	risking
risk	risks
risk	risk
rival	rivaled
rival	rivaled
rival	rivaling
rival	rivalled
rival	rivalled
rival	rivalling
rival	rivals
rival	rival
river	rivered
river	rivered
river	rivering
river	rivers
river	river
rob	robbed
rob	robbed
rob	robbing
rob	robs
rob	rob
rocket	rocketed
rocket	rocketed
rocket	rocketing
rocket	rockets
rocket	rocket
rock	rocked
rock	rocked
rock	rocking
rock	rocks
rock	rock
rod	rodded
rod	rodded
rod	rodding
rod	rods
rod	rod
roger	rogered
roger	rogered
roger	rogering
roger	rogers
roger	roger
roll	rolled
roll	rolled
roll	rolling
roll	rolls
roll	roll
romance	romanced
romance	romanced
romance	romances
romance	romance
romance	romancing
roof	roofed
roof	roofed
roof	roofing
roof	roofs
roof	roof
room	roomed
room	roomed
room	rooming
room	rooms
room	room
root	rooted
root	rooted
root	rooting
root	roots
root	root
rope	roped
rope	roped
rope	ropes
rope	rope
rope	roping
rosa	rosaed
rosa	rosaed
rosa	rosaing
rosa	rosas
rosa	rosa
rose	rosed
rose	rosed
rose	roses
rose	rose
rose	rosing
ross	rossed
ross	rossed
ross	rosses
ross	rossing
ross	ross
roster	rostered
roster	rostered
roster	rostering
roster	rosters
roster	roster
rouge	rouged
rouge	rouged
rouge	rouges
rouge	rouge
rouge	rouging
rough	roughed
rough	roughed
rough	roughing
rough	roughs
rough	rough
round	rounded
round	rounded
round	rounding
round	rounds
round	round
route	routed
route	routed
route	routeing
route	routes
route	route
route	routing
row	rowed
row	rowed
row	rowing
row	rows
row	row
rubber	rubbered
rubber	rubbered
rubber	rubbering
rubber	rubbers
rubber	rubber
ruby	rubied
ruby	rubied
ruby	rubies
ruby	rubying
ruby	ruby
rue	rued
rue	rued
rue	rueing
rue	rues
rue	rue
rue	ruing
rule	ruled
rule	ruled
rule	rules
rule	rule
rule	ruling
run	ran
run	running
run	runs
run	run
run	run
rush	rushed
rush	rushed
rush	rushes
rush	rushing
rush	rush
sacrifice	sacrificed
sacrifice	sacrificed
sacrifice	sacrifices
sacrifice	sacrifice
sacrifice	sacrificing
sage	saged
sage	saged
sage	sages
sage	sage
sage	saging
sail	sailed
sail	sailed
sail	sailing
sail	sails
sail	sail
saint	sainted
saint	sainted
saint	sainting
saint	saints
saint	saint
salary	salaried
salary	salaried
salary	salaries
salary	salarying
salary	salary
sally	sallied
sally	sallied
sally	sallies
sally	sallying
sally	sally
salt	salted
salt	salted
salt	salting
salt	salts
salt	salt
sample	sampled
sample	sampled
sample	samples
sample	sample
sample	sampling
sam	sammed
sam	sammed
sam	samming
sam	sams
sam	sam
sand	sanded
sand	sanded
sand	sanding
sand	sands
sand	sand
satisfy	satisfied
satisfy	satisfied
satisfy	satisfies
satisfy	satisfying
satisfy	satisfy
savage	savaged
savage	savaged
savage	savages
savage	savage
savage	savaging
save	saved
save	saved
save	saves
save	save
save	saving
saw	sawed
saw	sawed
saw	sawing
saw	sawn
saw	saws
saw	saw
say	said
say	said
say	sayed
say	sayed
say	saying
say	says
say	say
scale	scaled
scale	scaled
scale	scales
scale	scale
scale	scaling
scandal	scandaled
scandal	scandaled
scandal	scandaling
scandal	scandalled
scandal	scandalled
scandal	scandalling
scandal	scandals
scandal	scandal
scene	scened
scene	scened
scene	scenes
scene	scene
scene	scening
schedule	scheduled
schedule	scheduled
schedule	schedules
schedule	schedule
schedule	scheduling
scheme	schemed
scheme	schemed
scheme	schemes
scheme	scheme
scheme	scheming
school	schooled
school	schooled
school	schooling
school	schools
school	school
science	scienced
science	scienced
science	sciences
science	science
science	sciencing
scope	scoped
scope	scoped
scope	scopes
scope	scope
scope	scoping
score	scored
score	scored
score	scores
score	score
score	scoring
scout	scouted
scout	scouted
scout	scouting
scout	scouts
scout	scout
screen	screened
screen	screened
screen	screening
screen	screens
screen	screen
script	scripted
script	scripted
script	scripting
script	scripts
script	script
sculpture	sculptured
sculpture	sculptured
sculpture	sculptures
sculpture	sculpture
sculpture	sculpturing
seal	sealed
seal	sealed
seal	sealing
seal	seals
seal	seal
sean	seaned
sean	seaned
sean	seaning
sean	seans
sean	sean
search	searched
search	searched
search	searches
search	searching
search	search
season	seasoned
season	seasoned
season	seasoning
season	seasons
season	season
seat	seated
seat	seated
seat	seating
seat	seats
seat	seat
second	seconded
second	seconded
second	seconding
second	seconds
second	second
secretary	secretaried
secretary	secretaried
secretary	secretaries
secretary	secretarying
secretary	secretary
secret	secreted
secret	secreted
secret	secreting
secret	secrets
secret	secretted
secret	secretted
secret	secretting
secret	secret
section	sectioned
section	sectioned
section	sectioning
section	sections
section	section
secure	secured
secure	secured
secure	secures
secure	secure
secure	securing
seed	seeded
seed	seeded
seed	seeding
seed	seeds
seed	seed
seek	seeking
seek	seeks
seek	seek
seek	sought
seek	sought
seem	seemed
seem	seemed
seem	seeming
seem	seems
seem	seem
see	saw
see	seeing
see	seen
see	sees
see	see
segment	segmented
segment	segmented
segment	segmenting
segment	segments
segment	segment
select	selected
select	selected
select	selecting
select	selects
select	select
self	selfed
self	selfed
self	selfing
self	selfs
self	self
sell	selling
sell	sells
sell	sell
sell	sold
sell	sold
send	sending
send	sends
send	send
send	sent
send	sent
sense	sensed
sense	sensed
sense	senses
sense	sense
sense	sensing
sentence	sentenced
sentence	sentenced
sentence	sentences
sentence	sentence
sentence	sentencing
separate	separated
separate	separated
separate	separates
separate	separate
separate	separating
sequence	sequenced
sequence	sequenced
sequence	sequences
sequence	sequence
sequence	sequencing
servant	servanted
servant	servanted
servant	servanting
servant	servants
servant	servant
serve	served
serve	served
serve	serves
serve	serve
serve	serving
service	serviced
service	serviced
service	services
service	service
service	servicing
set	sets
set	setted
set	setted
set	setten
set	setting
set	set
set	set
set	set
settle	settled
settle	settled
settle	settles
settle	settle
settle	settling
sex	sexed
sex	sexed
sex	sexes
sex	sexing
sex	sex
shade	shaded
shade	shaded
shade	shades
shade	shade
shade	shading
shadow	shadowed
shadow	shadowed
shadow	shadowing
shadow	shadows
shadow	shadow
shaft	shafted
shaft	shafted
shaft	shafting
shaft	shafts
shaft	shaft
shallow	shallowed
shallow	shallowed
shallow	shallowing
shallow	shallows
shallow	shallow
shall	shall
shall	shall
shall	should
shanghai	shanghaied
shanghai	shanghaied
shanghai	shanghaiing
shanghai	shanghais
shanghai	shanghai
shape	shaped
shape	shaped
shape	shapen
shape	shapes
shape	shape
shape	shaping
shape	shope
share	shared
share	shared
share	shares
share	share
share	sharing
shark	sharked
shark	sharked
shark	sharking
shark	sharks
shark	shark
sharp	sharped
sharp	sharped
sharp	sharping
sharp	sharps
sharp	sharp
shed	shedding
shed	sheds
shed	shed
shed	shed
shed	shed
sheet	sheeted
sheet	sheeted
sheet	sheeting
sheet	sheets
sheet	sheet
shell	shelled
shell	shelled
shell	shelling
shell	shells
shell	shell
shelter	sheltered
shelter	sheltered
shelter	sheltering
shelter	shelters
shelter	shelter
shepherd	shepherded
shepherd	shepherded
shepherd	shepherding
shepherd	shepherds
shepherd	shepherd
sheriff	sheriffed
sheriff	sheriffed
sheriff	sheriffing
sheriff	sheriffs
sheriff	sheriff
shield	shielded
shield	shielded
shield	shielding
shield	shields
shield	shield
shift	shifted
shift	shifted
shift	shifting
shift	shifts
shift	shift
ship	shipped
ship	shipped
ship	shipping
ship	ships
ship	ship
shire	shired
shire	shired
shire	shires
shire	shire
shire	shiring
shirt	shirted
shirt	shirted
shirt	shirting
shirt	shirts
shirt	shirt
shock	shocked
shock	shocked
shock	shocking
shock	shocks
shock	shock
shoe	shodden
shoe	shod
shoe	shod
shoe	shoed
shoe	shoed
shoe	shoeing
shoe	shoes
shoe	shoe
shoot	shooting
shoot	shoots
shoot	shoot
shoot	shotten
shoot	shot
shoot	shot
shop	shopped
shop	shopped
shop	shopping
shop	shops
shop	shop
shore	shored
shore	shored
shore	shores
shore	shore
shore	shoring
short	shorted
short	shorted
short	shorting
short	shorts
short	short
shot	shots
shot	shotted
shot	shotted
shot	shotting
shot	shot
shoulder	shouldered
shoulder	shouldered
shoulder	shouldering
shoulder	shoulders
shoulder	shoulder
showcase	showcased
showcase	showcased
showcase	showcases
showcase	showcase
showcase	showcasing
show	showed
show	showed
show	showing
show	shown
show	shows
show	show
shrine	shrined
shrine	shrined
shrine	shrines
shrine	shrine
shrine	shrining
shrub	shrubbed
shrub	shrubbed
shrub	shrubbing
shrub	shrubs
shrub	shrub
shut	shuts
shut	shutting
shut	shut
shut	shut
shut	shut
shuttle	shuttled
shuttle	shuttled
shuttle	shuttles
shuttle	shuttle
shuttle	shuttling
sick	sicked
sick	sicked
sick	sicking
sick	sicks
sick	sick
side	sided
side	sided
side	sides
side	side
side	siding
siege	sieged
siege	sieged
siege	sieges
siege	siege
siege	sieging
sight	sighted
sight	sighted
sight	sighting
sight	sights
sight	sight
signal	signaled
signal	signaled
signal	signaling
signal	signalled
signal	signalled
signal	signalling
signal	signals
signal	signal
sign	signed
sign	signed
sign	signing
sign	signs
sign	sign
silence	silenced
silence	silenced
silence	silences
silence	silence
silence	silencing
silk	silked
silk	silked
silk	silking
silk	silks
silk	silk
silver	silvered
silver	silvered
silver	silvering
silver	silvers
silver	silver
simple	simpled
simple	simpled
simple	simples
simple	simple
simple	simpling
single	singled
single	singled
single	singles
single	single
single	singling
sing	sang
sing	singing
sing	sings
sing	sing
sing	sungen
sing	sung
sin	sinned
sin	sinned
sin	sinning
sin	sins
sin	sin
sir	sirred
sir	sirred
sir	sirring
sir	sirs
sir	sir
sister	sistered
sister	sistered
sister	sistering
sister	sisters
sister	sister
site	sited
site	sited
site	sites
site	site
site	siting
sit	sat
sit	sat
sit	sits
sit	sitten
sit	sitting
sit	sit
size	sized
size	sized
size	sizes
size	size
size	sizing
skeleton	skeletoned
skeleton	skeletoned
skeleton	skeletoning
skeleton	skeletons
skeleton	skeleton
sketch	sketched
sketch	sketched
sketch	sketches
sketch	sketching
sketch	sketch
skill	skilled
skill	skilled
skill	skilling
skill	skills
skill	skill
skin	skinned
skin	skinned
skin	skinning
skin	skins
skin	skin
ski	skied
ski	skied
ski	skies
ski	skiing
ski	skis
ski	ski
skull	skulled
skull	skulled
skull	skulling
skull	skulls
skull	skull
sky	skied
sky	skied
sky	skies
sky	skyed
sky	skyed
sky	skying
sky	sky
slam	slammed
slam	slammed
slam	slamming
slam	slams
slam	slam
slate	slated
slate	slated
slate	slates
slate	slate
slate	slating
slave	slaved
slave	slaved
slave	slaves
slave	slave
slave	slaving
sleep	sleeping
sleep	sleeps
sleep	sleep
sleep	slept
sleep	slept
slide	slidden
slide	slides
slide	slide
slide	sliding
slide	slid
slide	slid
slight	slighted
slight	slighted
slight	slighting
slight	slights
slight	slight
slope	sloped
slope	sloped
slope	slopes
slope	slope
slope	sloping
slot	slots
slot	slotted
slot	slotted
slot	slotting
slot	slot
slow	slowed
slow	slowed
slow	slowing
slow	slows
slow	slow
small	smalled
small	smalled
small	smalling
small	smalls
small	small
smart	smarted
smart	smarted
smart	smarting
smart	smarts
smart	smart
smart	smorten
smart	smort
smith	smithed
smith	smithed
smith	smithing
smith	smiths
smith	smith
smoke	smoked
smoke	smoked
smoke	smokes
smoke	smoke
smoke	smoking
smooth	smoothed
smooth	smoothed
smooth	smoothing
smooth	smooths
smooth	smooth
snail	snailed
snail	snailed
snail	snailing
snail	snails
snail	snail
snake	snaked
snake	snaked
snake	snakes
snake	snake
snake	snaking
snow	snew
snow	snowed
snow	snowed
snow	snowing
snow	snown
snow	snows
snow	snow
soap	soaped
soap	soaped
soap	soaping
soap	soaps
soap	soap
soccer	soccered
soccer	soccered
soccer	soccering
soccer	soccers
soccer	soccer
soil	soiled
soil	soiled
soil	soiling
soil	soils
soil	soil
soldier	soldiered
soldier	soldiered
soldier	soldiering
soldier	soldiers
soldier	soldier
sole	soled
sole	soled
sole	soles
sole	sole
sole	soling
solo	soloed
solo	soloed
solo	soloes
solo	soloing
solo	solo
solve	solved
solve	solved
solve	solves
solve	solve
solve	solving
something	somethinged
something	somethinged
something	somethinging
something	somethings
something	something
sort	sorted
sort	sorted
sort	sorting
sort	sorts
sort	sort
soul	souled
soul	souled
soul	souling
soul	souls
soul	soul
sound	sounded
sound	sounded
sound	sounding
sound	sounds
sound	sound
soundtrack	soundtracked
soundtrack	soundtracked
soundtrack	soundtracking
soundtrack	soundtracks
soundtrack	soundtrack
source	sourced
source	sourced
source	sources
source	source
source	sourcing
south	southed
south	southed
south	southing
south	souths
south	south
space	spaced
space	spaced
space	spaces
space	space
space	spacing
spanish	spanished
spanish	spanished
spanish	spanishes
spanish	spanishing
spanish	spanish
span	spanned
span	spanned
span	spanning
span	spans
span	span
spare	spared
spare	spared
spare	spares
spare	spare
spare	sparing
speak	spake
speak	speaking
speak	speaks
speak	speak
speak	spoken
speak	spoke
speed	sped
speed	sped
speed	speeded
speed	speeded
speed	speeding
speed	speeds
speed	speed
spell	spelled
spell	spelled
spell	spelling
spell	spells
spell	spell
spell	spelt
spell	spelt
spend	spending
spend	spends
spend	spend
spend	spent
spend	spent
sphere	sphered
sphere	sphered
sphere	spheres
sphere	sphere
sphere	sphering
spider	spidered
spider	spidered
spider	spidering
spider	spiders
spider	spider
spike	spiked
spike	spiked
spike	spikes
spike	spike
spike	spiking
spin	span
spin	spinning
spin	spins
spin	spin
spin	spun
spin	spun
spiral	spiraled
spiral	spiraled
spiral	spiraling
spiral	spiralled
spiral	spiralled
spiral	spiralling
spiral	spirals
spiral	spiral
spirit	spirited
spirit	spirited
spirit	spiriting
spirit	spirits
spirit	spirit
spite	spited
spite	spited
spite	spites
spite	spite
spite	spiting
split	splits
split	splitting
split	split
split	split
split	split
spoke	spoked
spoke	spoked
spoke	spokes
spoke	spoke
spoke	spoking
sponsor	sponsored
sponsor	sponsored
sponsor	sponsoring
sponsor	sponsors
sponsor	sponsor
sport	sported
sport	sported
sport	sporting
sport	sports
sport	sport
spot	spots
spot	spotted
spot	spotted
spot	spotting
spot	spot
spread	spreading
spread	spreads
spread	spread
spread	spread
spread	spread
spring	sprang
spring	springing
spring	springs
spring	spring
spring	sprung
spring	sprung
sprint	sprant
sprint	sprinted
sprint	sprinted
sprint	sprinting
sprint	sprints
sprint	sprint
sprint	sprunt
spur	spurred
spur	spurred
spur	spurring
spur	spurs
spur	spur
spy	spied
spy	spied
spy	spies
spy	spying
spy	spy
square	squared
square	squared
square	squares
square	square
square	squaring
stable	stabled
stable	stabled
stable	stables
stable	stable
stable	stabling
stack	stacked
stack	stacked
stack	stacking
stack	stacks
stack	stack
staff	staffed
staff	staffed
staff	staffing
staff	staffs
staff	staff
stage	staged
stage	staged
stage	stages
stage	stage
stage	staging
stake	staked
stake	staked
stake	stakes
stake	stake
stake	staking
stamp	stamped
stamp	stamped
stamp	stamping
stamp	stamps
stamp	stamp
stand	standing
stand	stands
stand	stand
stand	stood
stand	stood
stark	starked
stark	starked
stark	starking
stark	starks
stark	stark
star	starred
star	starred
star	starring
star	stars
star	star
start	started
start	started
start	starting
start	starts
start	start
statement	statemented
statement	statemented
statement	statementing
statement	statements
statement	statement
state	stated
state	stated
state	states
state	state
state	stating
station	stationed
station	stationed
station	stationing
station	stations
station	station
statue	statued
statue	statued
statue	statues
statue	statue
statue	statuing
stay	stayed
stay	stayed
stay	staying
stay	stays
stay	stay
steady	steadied
steady	steadied
steady	steadies
steady	steadying
steady	steady
steal	stealing
steal	steals
steal	steal
steal	stolen
steal	stole
steam	steamed
steam	steamed
steam	steaming
steam	steams
steam	steam
steel	steeled
steel	steeled
steel	steeling
steel	steels
steel	steel
steep	steeped
steep	steeped
steep	steeping
steep	steeps
steep	steep
stem	stemmed
stem	stemmed
stem	stemming
stem	stems
stem	stem
step	stepped
step	stepped
step	stepping
step	steps
step	stept
step	stept
step	step
step	stopen
step	stope
steven	stevened
steven	stevened
steven	stevening
steven	stevens
steven	steven
steve	steved
steve	steved
steve	steves
steve	steve
steve	steving
stick	sticked
stick	sticked
stick	sticking
stick	sticks
stick	stick
stick	stuck
stick	stuck
still	stilled
still	stilled
still	stilling
still	stills
still	still
stint	stinted
stint	stinted
stint	stinting
stint	stints
stint	stint
stock	stocked
stock	stocked
stock	stocking
stock	stocks
stock	stock
stoke	stoked
stoke	stoked
stoke	stokes
stoke	stoke
stoke	stoking
stomach	stomached
stomach	stomached
stomach	stomaching
stomach	stomachs
stomach	stomach
stone	stoned
stone	stoned
stone	stones
stone	stone
stone	stoning
stop	stopped
stop	stopped
stop	stopping
stop	stops
stop	stop
store	stored
store	stored
store	stores
store	store
store	storing
storm	stormed
storm	stormed
storm	storming
storm	storms
storm	storm
story	storied
story	storied
story	stories
story	storying
story	story
straight	straighted
straight	straighted
straight	straighting
straight	straights
straight	straight
strain	strained
strain	strained
strain	straining
strain	strains
strain	strain
strait	straited
strait	straited
strait	straiting
strait	straits
strait	strait
strand	stranded
strand	stranded
strand	stranding
strand	strands
strand	strand
stranger	strangered
stranger	strangered
stranger	strangering
stranger	strangers
stranger	stranger
strange	stranged
strange	stranged
strange	stranges
strange	strange
strange	stranging
streak	streaked
streak	streaked
streak	streaking
streak	streaks
streak	streak
stream	streamed
stream	streamed
stream	streaming
stream	streams
stream	stream
strengthen	strengthened
strengthen	strengthened
strengthen	strengthening
strengthen	strengthens
strengthen	strengthen
strength	strengthed
strength	strengthed
strength	strengthing
strength	strengths
strength	strength
stress	stressed
stress	stressed
stress	stresses
stress	stressing
stress	stress
stretch	straight
stretch	straight
stretch	straught
stretch	straught
stretch	stretched
stretch	stretched
stretch	stretches
stretch	stretching
stretch	stretch
strike	stricken
strike	strikes
strike	strike
strike	striking
strike	struck
strike	struck
string	stringing
string	strings
string	string
string	strung
string	strung
strip	stripped
strip	stripped
strip	stripping
strip	strips
strip	stript
strip	stript
strip	strip
stroke	stroked
stroke	stroked
stroke	strokes
stroke	stroke
stroke	stroking
structure	structured
structure	structured
structure	structures
structure	structure
structure	structuring
struggle	struggled
struggle	struggled
struggle	struggles
struggle	struggle
struggle	struggling
stud	studded
stud	studded
stud	studding
stud	studs
stud	stud
study	studied
study	studied
study	studies
study	studying
study	study
stuff	stuffed
stuff	stuffed
stuff	stuffing
stuff	stuffs
stuff	stuff
style	styled
style	styled
style	styles
style	style
style	styling
subject	subjected
subject	subjected
subject	subjecting
subject	subjects
subject	subject
submarine	submarined
submarine	submarined
submarine	submarines
submarine	submarine
submarine	submarining
submit	submits
submit	submitted
submit	submitted
submit	submitting
submit	submit
subordinate	subordinated
subordinate	subordinated
subordinate	subordinates
subordinate	subordinate
subordinate	subordinating
substitute	substituted
substitute	substituted
substitute	substitutes
substitute	substitute
substitute	substituting
substrate	substrated
substrate	substrated
substrate	substrates
substrate	substrate
substrate	substrating
sub	subbed
sub	subbed
sub	subbing
sub	subs
sub	sub
subway	subwayed
subway	subwayed
subway	subwaying
subway	subways
subway	subway
succeed	succeeded
succeed	succeeded
succeed	succeeding
succeed	succeeds
succeed	succeed
sue	sued
sue	sued
sue	sues
sue	sue
sue	suing
suffer	suffered
suffer	suffered
suffer	suffering
suffer	suffers
suffer	suffer
suffix	suffixed
suffix	suffixed
suffix	suffixes
suffix	suffixing
suffix	suffix
sugar	sugared
sugar	sugared
sugar	sugaring
sugar	sugars
sugar	sugar
suggest	suggested
suggest	suggested
suggest	suggesting
suggest	suggests
suggest	suggest
suicide	suicided
suicide	suicided
suicide	suicides
suicide	suicide
suicide	suiciding
suit	suited
suit	suited
suit	suiting
suit	suits
suit	suit
summer	summered
summer	summered
summer	summering
summer	summers
summer	summer
summit	summited
summit	summited
summit	summiting
summit	summits
summit	summitted
summit	summitted
summit	summitting
summit	summit
sum	summed
sum	summed
sum	summing
sum	sums
sum	sum
sunset	sunsets
sunset	sunsetted
sunset	sunsetted
sunset	sunsetting
sunset	sunset
sun	sunned
sun	sunned
sun	sunning
sun	suns
sun	sun
super	supered
super	supered
super	supering
super	supers
super	super
supplement	supplemented
supplement	supplemented
supplement	supplementing
supplement	supplements
supplement	supplement
supply	supplied
supply	supplied
supply	supplies
supply	supplying
supply	supply
support	supported
support	supported
support	supporting
support	supports
support	support
supreme	supremed
supreme	supremed
supreme	supremes
supreme	supreme
supreme	supreming
surface	surfaced
surface	surfaced
surface	surfaces
surface	surface
surface	surfacing
surname	surnamed
surname	surnamed
surname	surnames
surname	surname
surname	surnaming
surprise	surprised
surprise	surprised
surprise	surprises
surprise	surprise
surprise	surprising
surrender	surrendered
surrender	surrendered
surrender	surrendering
surrender	surrenders
surrender	surrender
survey	surveyed
survey	surveyed
survey	surveying
survey	surveys
survey	survey
survive	survived
survive	survived
survive	survives
survive	survive
survive	surviving
suspect	suspected
suspect	suspected
suspect	suspecting
suspect	suspects
suspect	suspect
suspicion	suspicioned
suspicion	suspicioned
suspicion	suspicioning
suspicion	suspicions
suspicion	suspicion
swamp	swamped
swamp	swamped
swamp	swamping
swamp	swamps
swamp	swamp
swan	swanned
swan	swanned
swan	swanning
swan	swans
swan	swan
swim	swam
swim	swimming
swim	swims
swim	swim
swim	swum
swim	swum
swing	swang
swing	swinging
swing	swings
swing	swing
swing	swungen
swing	swung
swing	swung
swiss	swissed
swiss	swissed
swiss	swisses
swiss	swissing
swiss	swiss
switch	switched
switch	switched
switch	switches
switch	switching
switch	switch
symbol	symboled
symbol	symboled
symbol	symboling
symbol	symbolled
symbol	symbolled
symbol	symbolling
symbol	symbols
symbol	symbol
table	tabled
table	tabled
table	tables
table	table
table	tabling
tackle	tackled
tackle	tackled
tackle	tackles
tackle	tackle
tackle	tackling
tag	tagged
tag	tagged
tag	tagging
tag	tags
tag	tag
tail	tailed
tail	tailed
tail	tailing
tail	tails
tail	tail
take	taken
take	takes
take	take
take	taking
take	took
tale	taled
tale	taled
tale	tales
tale	tale
tale	taling
talk	talked
talk	talked
talk	talking
talk	talks
talk	talk
tang	tanged
tang	tanged
tang	tanging
tang	tangs
tang	tang
tank	tanked
tank	tanked
tank	tanking
tank	tanks
tank	tank
tan	tanned
tan	tanned
tan	tanning
tan	tans
tan	tan
tape	taped
tape	taped
tape	tapes
tape	tape
tape	taping
target	targeted
target	targeted
target	targeting
target	targets
target	targetted
target	targetted
target	targetting
target	target
task	tasked
task	tasked
task	tasking
task	tasks
task	task
taste	tasted
taste	tasted
taste	tastes
taste	taste
taste	tasting
taxi	taxied
taxi	taxied
taxi	taxies
taxi	taxiing
taxi	taxis
taxi	taxi
taxi	taxying
tax	taxed
tax	taxed
tax	taxes
tax	taxing
tax	tax
teach	taught
teach	taught
teach	teaches
teach	teaching
teach	teach
team	teamed
team	teamed
team	teaming
team	teams
team	team
tea	teaed
tea	teaed
tea	teaing
tea	teas
tea	tea
ted	tedded
ted	tedded
ted	tedding
ted	teds
ted	ted
teen	teened
teen	teened
teen	teening
teen	teens
teen	teen
telegraph	telegraphed
telegraph	telegraphed
telegraph	telegraphing
telegraph	telegraphs
telegraph	telegraph
telephone	telephoned
telephone	telephoned
telephone	telephones
telephone	telephone
telephone	telephoning
telescope	telescoped
telescope	telescoped
telescope	telescopes
telescope	telescope
telescope	telescoping
television	televisioned
television	televisioned
television	televisioning
television	televisions
television	television
tell	telling
tell	tells
tell	tell
tell	told
tell	told
temperate	temperated
temperate	temperated
temperate	temperates
temperate	temperate
temperate	temperating
temple	templed
temple	templed
temple	temples
temple	temple
temple	templing
tender	tendered
tender	tendered
tender	tendering
tender	tenders
tender	tender
tend	tended
tend	tended
tend	tending
tend	tends
tend	tend
tennis	tennised
tennis	tennised
tennis	tennises
tennis	tennising
tennis	tennis
tension	tensioned
tension	tensioned
tension	tensioning
tension	tensions
tension	tension
tenure	tenured
tenure	tenured
tenure	tenures
tenure	tenure
tenure	tenuring
term	termed
term	termed
term	terming
term	terms
term	term
terrace	terraced
terrace	terraced
terrace	terraces
terrace	terrace
terrace	terracing
test	tested
test	tested
test	testing
test	tests
test	test
text	texted
text	texted
text	texting
text	texts
text	text
text	text
text	text
theme	themed
theme	themed
theme	themes
theme	theme
theme	theming
theorem	theoremed
theorem	theoremed
theorem	theoreming
theorem	theorems
theorem	theorem
therapy	therapied
therapy	therapied
therapy	therapies
therapy	therapying
therapy	therapy
thick	thicked
thick	thicked
thick	thicking
thick	thicks
thick	thick
thing	thinged
thing	thinged
thing	thinging
thing	things
thing	thing
think	thinking
think	thinks
think	think
think	thought
think	thought
thin	thinned
thin	thinned
thin	thinning
thin	thins
thin	thin
third	thirded
third	thirded
third	thirding
third	thirds
third	third
threat	threated
threat	threated
threat	threating
threat	threats
threat	threat
throat	throated
throat	throated
throat	throating
throat	throats
throat	throat
throne	throned
throne	throned
throne	thrones
throne	throne
throne	throning
throw	threw
throw	throwing
throw	thrown
throw	throws
throw	throw
thrust	thrusted
thrust	thrusted
thrust	thrusting
thrust	thrusts
thrust	thrust
thrust	thrust
thrust	thrust
thunder	thundered
thunder	thundered
thunder	thundering
thunder	thunders
thunder	thunder
ticket	ticketed
ticket	ticketed
ticket	ticketing
ticket	tickets
ticket	ticket
tide	tided
tide	tided
tide	tides
tide	tide
tide	tiding
tier	tiered
tier	tiered
tier	tiering
tier	tiers
tier	tier
tie	tied
tie	tied
tie	ties
tie	tie
tie	tying
tight	tighted
tight	tighted
tight	tighting
tight	tights
tight	tight
till	tilled
till	tilled
till	tilling
till	tills
till	till
timber	timbered
timber	timbered
timber	timbering
timber	timbers
timber	timber
timeline	timelined
timeline	timelined
timeline	timelines
timeline	timeline
timeline	timelining
times	timesed
times	timesed
times	timeses
times	timesing
times	times
time	timed
time	timed
time	times
time	time
time	timing
tin	tinned
tin	tinned
tin	tinning
tin	tins
tin	tin
tip	tipped
tip	tipped
tip	tipping
tip	tips
tip	tip
tissue	tissued
tissue	tissued
tissue	tissues
tissue	tissue
tissue	tissuing
title	titled
title	titled
title	titles
title	title
title	titling
toll	tolled
toll	tolled
toll	tolling
toll	tolls
toll	toll
tomb	tombed
tomb	tombed
tomb	tombing
tomb	tombs
tomb	tomb
tom	tommed
tom	tommed
tom	tomming
tom	toms
tom	tom
tone	toned
tone	toned
tone	tones
tone	tone
tone	toning
tongue	tongued
tongue	tongued
tongue	tongues
tongue	tongue
tongue	tonguing
tool	tooled
tool	tooled
tool	tooling
tool	tools
tool	tool
tooth	toothed
tooth	toothed
tooth	toothing
tooth	tooths
tooth	tooth
top	topped
top	topped
top	topping
top	tops
top	top
torpedo	torpedoed
torpedo	torpedoed
torpedo	torpedoes
torpedo	torpedoing
torpedo	torpedo
torture	tortured
torture	tortured
torture	tortures
torture	torture
torture	torturing
total	totaled
total	totaled
total	totaling
total	totalled
total	totalled
total	totalling
total	totals
total	total
touch	touched
touch	touched
touch	touches
touch	touching
touch	touch
tough	toughed
tough	toughed
tough	toughing
tough	toughs
tough	tough
tour	toured
tour	toured
tour	touring
tour	tours
tour	tour
tower	towered
tower	towered
tower	towering
tower	towers
tower	tower
toy	toyed
toy	toyed
toy	toying
toy	toys
toy	toy
trace	traced
trace	traced
trace	traces
trace	trace
trace	tracing
track	tracked
track	tracked
track	tracking
track	tracks
track	track
traction	tractioned
traction	tractioned
traction	tractioning
traction	tractions
traction	traction
tract	tracted
tract	tracted
tract	tracting
tract	tracts
tract	tract
trademark	trademarked
trademark	trademarked
trademark	trademarking
trademark	trademarks
trademark	trademark
trade	traded
trade	traded
trade	trades
trade	trade
trade	trading
tradition	traditioned
tradition	traditioned
tradition	traditioning
tradition	traditions
tradition	tradition
traffic	trafficked
traffic	trafficked
traffic	trafficking
traffic	traffics
traffic	traffic
trailer	trailered
trailer	trailered
trailer	trailering
trailer	trailers
trailer	trailer
trail	trailed
trail	trailed
trail	trailing
trail	trails
trail	trail
train	trained
train	trained
train	training
train	trains
train	train
tram	trammed
tram	trammed
tram	tramming
tram	trams
tram	tram
transfer	transferred
transfer	transferred
transfer	transferring
transfer	transfers
transfer	transfer
transform	transformed
transform	transformed
transform	transforming
transform	transforms
transform	transform
transgender	transgendered
transgender	transgendered
transgender	transgendering
transgender	transgenders
transgender	transgender
transition	transitioned
transition	transitioned
transition	transitioning
transition	transitions
transition	transition
transit	transited
transit	transited
transit	transiting
transit	transits
transit	transit
transport	transported
transport	transported
transport	transporting
transport	transports
transport	transport
trap	trapped
trap	trapped
trap	trapping
trap	traps
trap	trap
travel	traveled
travel	traveled
travel	traveling
travel	travelled
travel	travelled
travel	travelling
travel	travels
travel	travel
treasure	treasured
treasure	treasured
treasure	treasures
treasure	treasure
treasure	treasuring
treat	treated
treat	treated
treat	treating
treat	treats
treat	treat
tree	treed
tree	treed
tree	treeing
tree	trees
tree	tree
trek	trekked
trek	trekked
trek	trekking
trek	treks
trek	trek
trend	trended
trend	trended
trend	trending
trend	trends
trend	trend
trial	trialed
trial	trialed
trial	trialing
trial	trialled
trial	trialled
trial	trialling
trial	trials
trial	trial
tribe	tribed
tribe	tribed
tribe	tribes
tribe	tribe
tribe	tribing
tribute	tributed
tribute	tributed
tribute	tributes
tribute	tribute
tribute	tributing
trick	tricked
trick	tricked
trick	tricking
trick	tricks
trick	trick
trigger	triggered
trigger	triggered
trigger	triggering
trigger	triggers
trigger	trigger
trim	trimmed
trim	trimmed
trim	trimming
trim	trims
trim	trim
triple	tripled
triple	tripled
triple	triples
triple	triple
triple	tripling
trip	tripped
trip	tripped
trip	tripping
trip	trips
trip	trip
triumph	triumphed
triumph	triumphed
triumph	triumphing
triumph	triumphs
triumph	triumph
troop	trooped
troop	trooped
troop	trooping
troop	troops
troop	troop
trouble	troubled
trouble	troubled
trouble	troubles
trouble	trouble
trouble	troubling
trout	trouted
trout	trouted
trout	trouting
trout	trouts
trout	trout
truck	trucked
truck	trucked
truck	trucking
truck	trucks
truck	truck
true	trued
true	trued
true	trueing
true	trues
true	true
true	truing
trumpet	trumpeted
trumpet	trumpeted
trumpet	trumpeting
trumpet	trumpets
trumpet	trumpet
trump	trumped
trump	trumped
trump	trumping
trump	trumps
trump	trump
trunk	trunked
trunk	trunked
trunk	trunking
trunk	trunks
trunk	trunk
trustee	trusteeed
trustee	trusteeed
trustee	trusteeing
trustee	trustees
trustee	trustee
trust	trusted
trust	trusted
trust	trusting
trust	trusts
trust	trust
truth	truthed
truth	truthed
truth	truthing
truth	truths
truth	truth
try	tried
try	tried
try	tries
try	trying
try	try
tube	tubed
tube	tubed
tube	tubes
tube	tube
tube	tubing
tucker	tuckered
tucker	tuckered
tucker	tuckering
tucker	tuckers
tucker	tucker
tune	tuned
tune	tuned
tune	tunes
tune	tune
tune	tuning
tunnel	tunneled
tunnel	tunneled
tunnel	tunneling
tunnel	tunnelled
tunnel	tunnelled
tunnel	tunnelling
tunnel	tunnels
tunnel	tunnel
turf	turfed
turf	turfed
turf	turfing
turf	turfs
turf	turf
turn	turned
turn	turned
turn	turning
turn	turns
turn	turn
turtle	turtled
turtle	turtled
turtle	turtles
turtle	turtle
turtle	turtling
tutor	tutored
tutor	tutored
tutor	tutoring
tutor	tutors
tutor	tutor
twin	twinned
twin	twinned
twin	twinning
twin	twins
twin	twin
twist	twisted
twist	twisted
twist	twisting
twist	twists
twist	twist
twitter	twittered
twitter	twittered
twitter	twittering
twitter	twitters
twitter	twitter
type	typed
type	typed
type	types
type	type
type	typing
umbrella	umbrellaed
umbrella	umbrellaed
umbrella	umbrellaing
umbrella	umbrellas
umbrella	umbrella
undergo	undergoes
undergo	undergoing
undergo	undergone
undergo	undergo
undergo	underwent
underground	undergrounded
underground	undergrounded
underground	undergrounding
underground	undergrounds
underground	underground
understand	understanding
understand	understands
understand	understand
understand	understood
understand	understood
underwater	underwatered
underwater	underwatered
underwater	underwatering
underwater	underwaters
underwater	underwater
uniform	uniformed
uniform	uniformed
uniform	uniforming
uniform	uniforms
uniform	uniform
unlike	unliked
unlike	unliked
unlike	unlikes
unlike	unlike
unlike	unliking
update	updated
update	updated
update	updates
update	update
update	updating
upgrade	upgraded
upgrade	upgraded
upgrade	upgrades
upgrade	upgrade
upgrade	upgrading
upset	upsets
upset	upsetting
upset	upset
upset	upset
upset	upset
up	upped
up	upped
up	upping
up	ups
up	up
use	used
use	used
use	uses
use	use
use	using
utilize	utilized
utilize	utilized
utilize	utilizes
utilize	utilize
utilize	utilizing
vacation	vacationed
vacation	vacationed
vacation	vacationing
vacation	vacations
vacation	vacation
vacuum	vacuumed
vacuum	vacuumed
vacuum	vacuuming
vacuum	vacuums
vacuum	vacuum
value	valued
value	valued
value	values
value	value
value	valuing
valve	valved
valve	valved
valve	valves
valve	valve
valve	valving
van	vanned
van	vanned
van	vanning
van	vans
van	van
vary	varied
vary	varied
vary	varies
vary	varying
vary	vary
vault	vaulted
vault	vaulted
vault	vaulting
vault	vaults
vault	vault
vector	vectored
vector	vectored
vector	vectoring
vector	vectors
vector	vector
venture	ventured
venture	ventured
venture	ventures
venture	venture
venture	venturing
verbal	verballed
verbal	verballed
verbal	verballing
verbal	verbals
verbal	verbal
verb	verbed
verb	verbed
verb	verbing
verb	verbs
verb	verb
verse	versed
verse	versed
verse	verses
verse	verse
verse	versing
version	versioned
version	versioned
version	versioning
version	versions
version	version
vessel	vesselled
vessel	vesselled
vessel	vesselling
vessel	vessels
vessel	vessel
vice	viced
vice	viced
vice	vices
vice	vice
vice	vicing
victory	victoried
victory	victoried
victory	victories
victory	victorying
victory	victory
video	videoed
video	videoed
video	videoes
video	videoing
video	video
view	viewed
view	viewed
view	viewing
view	views
view	view
villain	villained
villain	villained
villain	villaining
villain	villains
villain	villain
vintage	vintaged
vintage	vintaged
vintage	vintages
vintage	vintage
vintage	vintaging
violent	violented
violent	violented
violent	violenting
violent	violents
violent	violent
visa	visaed
visa	visaed
visa	visaing
visa	visas
visa	visa
vision	visioned
vision	visioned
vision	visioning
vision	visions
vision	vision
visit	visited
visit	visited
visit	visiting
visit	visits
visit	visit
voice	voiced
voice	voiced
voice	voices
voice	voice
voice	voicing
void	voided
void	voided
void	voiding
void	voids
void	void
volunteer	volunteered
volunteer	volunteered
volunteer	volunteering
volunteer	volunteers
volunteer	volunteer
vote	voted
vote	voted
vote	votes
vote	vote
vote	voting
voyage	voyaged
voyage	voyaged
voyage	voyages
voyage	voyage
voyage	voyaging
wade	waded
wade	waded
wade	wades
wade	wade
wade	wading
wage	waged
wage	waged
wage	wages
wage	wage
wage	waging
wagon	wagoned
wagon	wagoned
wagon	wagoning
wagon	wagons
wagon	wagon
wait	waited
wait	waited
wait	waiting
wait	waits
wait	wait
wake	wakes
wake	wake
wake	waking
walk	walked
walk	walked
walk	walking
walk	walks
walk	walk
wall	walled
wall	walled
wall	walling
wall	walls
wall	wall
walter	waltered
walter	waltered
walter	waltering
walter	walters
walter	walter
wang	wanged
wang	wanged
wang	wanging
wang	wangs
wang	wang
want	wanted
want	wanted
want	wanting
want	wants
want	want
ward	warded
ward	warded
ward	warding
ward	wards
ward	ward
warehouse	warehoused
warehouse	warehoused
warehouse	warehouses
warehouse	warehouse
warehouse	warehousing
warfare	warfared
warfare	warfared
warfare	warfares
warfare	warfare
warfare	warfaring
warm	warmed
warm	warmed
warm	warming
warm	warms
warm	warm
warrant	warranted
warrant	warranted
warrant	warranting
warrant	warrants
warrant	warrant
war	warred
war	warred
war	warring
war	wars
war	war
waste	wasted
waste	wasted
waste	wastes
waste	waste
waste	wasting
watch	watched
watch	watched
watch	watches
watch	watching
watch	watch
water	watered
water	watered
water	watering
water	waters
water	water
wave	waved
wave	waved
wave	waves
wave	wave
wave	waving
way	wayed
way	wayed
way	waying
way	ways
way	way
wear	weared
wear	weared
wear	wearing
wear	wears
wear	wear
wear	wore
wear	worn
weather	weathered
weather	weathered
weather	weathering
weather	weathers
weather	weather
web	webbed
web	webbed
web	webbing
web	webs
web	web
wedding	weddinged
wedding	weddinged
wedding	weddinging
wedding	weddings
wedding	wedding
weekend	weekended
weekend	weekended
weekend	weekending
weekend	weekends
weekend	weekend
weight	weighted
weight	weighted
weight	weighting
weight	weights
weight	weight
welcome	welcomed
welcome	welcomed
welcome	welcomes
welcome	welcome
welcome	welcoming
well	welled
well	welled
well	welling
well	wells
well	well
welsh	welshed
welsh	welshed
welsh	welshes
welsh	welshing
welsh	welsh
west	wested
west	wested
west	westing
west	wests
west	west
wet	wets
wet	wetted
wet	wetted
wet	wetting
wet	wet
wet	wet
wet	wet
whale	whaled
whale	whaled
whale	whales
whale	whale
whale	whaling
wheel	wheeled
wheel	wheeled
wheel	wheeling
wheel	wheels
wheel	wheel
while	whiled
while	whiled
while	whiles
while	while
while	whiling
white	whited
white	whited
white	whites
white	white
white	whiting
widow	widowed
widow	widowed
widow	widowing
widow	widows
widow	widow
wild	wilded
wild	wilded
wild	wilding
wild	wilds
wild	wild
willow	willowed
willow	willowed
willow	willowing
willow	willows
willow	willow
will	willed
will	willed
will	willing
will	wills
will	will
will	would
window	windowed
window	windowed
window	windowing
window	windows
window	window
wind	winded
wind	winded
wind	winding
wind	winds
wind	wind
wind	wound
wind	wound
wine	wined
wine	wined
wine	wines
wine	wine
wine	wining
wing	winged
wing	winged
wing	winging
wing	wings
wing	wing
winter	wintered
winter	wintered
winter	wintering
winter	winters
winter	winter
win	winning
win	wins
win	win
win	won
win	won
wireless	wirelessed
wireless	wirelessed
wireless	wirelesses
wireless	wirelessing
wireless	wireless
wire	wired
wire	wired
wire	wires
wire	wire
wire	wiring
wise	wised
wise	wised
wise	wises
wise	wise
wise	wising
wish	wished
wish	wished
wish	wishes
wish	wishing
wish	wish
witch	witched
witch	witched
witch	witches
witch	witching
witch	witch
withdraw	withdrawing
withdraw	withdrawn
withdraw	withdraws
withdraw	withdraw
withdraw	withdrew
witness	witnessed
witness	witnessed
witness	witnesses
witness	witnessing
witness	witness
wizard	wizarded
wizard	wizarded
wizard	wizarding
wizard	wizards
wizard	wizard
wolf	wolfed
wolf	wolfed
wolf	wolfing
wolf	wolfs
wolf	wolf
woman	womaned
woman	womaned
woman	womaning
woman	womans
woman	woman
wonder	wondered
wonder	wondered
wonder	wondering
wonder	wonders
wonder	wonder
won	wond
won	wond
won	woning
won	wons
won	won
wood	wooded
wood	wooded
wood	wooding
wood	woods
wood	wood
word	worded
word	worded
word	wording
word	words
word	word
workshop	workshopped
workshop	workshopped
workshop	workshopping
workshop	workshops
workshop	workshop
work	worked
work	worked
work	working
work	works
work	work
work	wrought
work	wrought
world	worlded
world	worlded
world	worlding
world	worlds
world	world
worse	worsed
worse	worsed
worse	worses
worse	worse
worse	worsing
worship	worshiped
worship	worshiped
worship	worshiping
worship	worshipped
worship	worshipped
worship	worshipping
worship	worships
worship	worship
worst	worsted
worst	worsted
worst	worsting
worst	worsts
worst	worst
worth	worthed
worth	worthed
worth	worthen
worth	worthing
worth	worths
worth	worth
worth	worth
worth	worth
worthy	worthied
worthy	worthied
worthy	worthies
worthy	worthying
worthy	worthy
wound	wounded
wound	wounded
wound	wounding
wound	wounds
wound	wound
wreck	wrecked
wreck	wrecked
wreck	wrecking
wreck	wrecks
wreck	wreck
wright	wrighted
wright	wrighted
wright	wrighting
wright	wrights
wright	wright
write	writes
write	write
write	writing
write	written
write	writ
write	writ
write	wrote
wrong	wronged
wrong	wronged
wrong	wronging
wrong	wrongs
wrong	wrong
yacht	yachted
yacht	yachted
yacht	yachting
yacht	yachts
yacht	yacht
yang	yanged
yang	yanged
yang	yanging
yang	yangs
yang	yang
yard	yarded
yard	yarded
yard	yarding
yard	yards
yard	yard
yellow	yellowed
yellow	yellowed
yellow	yellowing
yellow	yellows
yellow	yellow
yes	yeses
yes	yessed
yes	yessed
yes	yessing
yes	yes
yet	yets
yet	yetted
yet	yetted
yet	yetting
yet	yet
yield	yielded
yield	yielded
yield	yielding
yield	yields
yield	yield
yield	yolden
yield	yold
york	yorked
york	yorked
york	yorking
york	yorks
york	york
young	younged
young	younged
young	younging
young	youngs
young	young
you	youed
you	youed
you	youing
you	yous
you	you
zero	zeroed
zero	zeroed
zero	zeroes
zero	zeroing
zero	zero
zinc	zinced
zinc	zinced
zinc	zincing
zinc	zincked
zinc	zincked
zinc	zincking
zinc	zincs
zinc	zinc
zinc	zinked
zinc	zinked
zinc	zinking
zone	zoned
zone	zoned
zone	zones
zone	zone
zone	zoning'''

OUTVOCAB = '''swivel	swiveled
overhold	overheld
repass	repassed
deaccession	deaccessioned
cant	canted
unoil	unoiled
outdrive	outdrove
abray	abrayed
hawm	hawmed
handwave	handwaved
publicize	publicizes
ringfence	ringfencing
palter	paltered
handwave	handwaved
whir	whir
euhemerize	euhemerize
regle	regling
embrangle	embrangle
glomp	glomping
recontact	recontacting
shamanise	shamanises
mounch	mounch
scrape	scraped
forclem	forclemming
autophosphorylate	autophosphorylated
autophosphorylate	autophosphorylates
deathmatch	deathmatching
hallucinate	hallucinated
disorientate	disorientate
epistle	epistles
rekick	rekicks
prefine	prefining
gush	gushes
omnibus	omnibussed
knawvshawl	knawvshawled
criminate	criminate
humect	humected
betoil	betoiling
metastatize	metastatize
unsecularize	unsecularizing
scrounge	scrounged
adjust	adjust
reimmerse	reimmerses
palter	paltered
desulphurize	desulphurizes
emphasize	emphasizes
fracture	fractures
ionize	ionize
miscommunicate	miscommunicated
innerduce	innerduced
blackhole	blackholing
overgeneralize	overgeneralize
grind	grinded
shard	sharding
hydroentangle	hydroentangle
chide	chid
incrust	incrusting
demotivate	demotivated
outscream	outscreamed
belock	belocked
devote	devoted
strap	straps
parrot	parroted
pander	pander
backhoe	backhoed
bewray	bewraying
deprioritize	deprioritizing
cripple	cripple
desalinate	desalinate
sandbox	sandboxing
sade	sade
bescribble	bescribble
lazy	lazies
rotavate	rotavated
talliate	talliated
calefy	calefy
beveil	beveiled
begore	begored
comingle	comingled
hooray	hooray
belock	belocked
devenustate	devenustated
robotize	robotize
degranulate	degranulates
tutoy	tutoyed
mesh	meshing
lojbanize	lojbanizing
pander	pander
undress	undress
envenom	envenomed
begore	begored
innerduce	innerduced
grind	grinded
editorialize	editorialize
favorize	favorizes
adjust	adjust
decimate	decimating
disspirit	disspirited
disspirit	disspirited
upsweep	upswept
pimple	pimples
enterlace	enterlaced
paganise	paganised
hydrate	hydrate
unvizard	unvizards
safari	safaris
spray	sprayed
beveil	beveiled
rotate	rotates
appreciate	appreciating
grice	gricing
vogue	voguing
wanton	wantoning
chicane	chicaning
knawvshawl	knawvshawled
kvitch	kvitches
kvitch	kvitching
refoment	refomented
drein	dreined
gush	gushes
trouser	trouser
disaccommodate	disaccommodate
untaste	untasted
peen	peened
southernize	southernizing
toe	toe
suntan	suntanned
indef	indeffed
nuncupate	nuncupated
adaunt	adaunts
autophosphorylate	autophosphorylated
autophosphorylate	autophosphorylates
pill	pilled
unclasp	unclasp
friz	frizzes
lid	lid
sacerdotalize	sacerdotalized
underpackage	underpackaged
butle	butled
uncoil	uncoils
disanoint	disanointing
mumble	mumbles
recarnify	recarnified
outcant	outcants
glomp	glomping
forehalsen	forehalsened
forecaddie	forecaddied
chaperon	chaperoning
misunderstand	misunderstand
besnowball	besnowball
victimise	victimises
shrithe	shrithes
overpush	overpushed
uncongeal	uncongealed
prechew	prechewed
soapstone	soapstones
outcant	outcants
flawter	flawters
quitclaim	quitclaims
overseed	overseeded
chitter	chittering
forseek	forseek
reman	remanned
resynthesize	resynthesized
renest	renested
grommet	grommets
emule	emuling
epistle	epistles
sustentate	sustentated
belecture	belectured
disinsert	disinserts
forban	forbanned
eventerate	eventerating
dramatise	dramatised
ragequit	ragequits
tesselate	tesselated
rerate	rerate
uproll	uprolled
mosey	moseys
retorque	retorques
deracialize	deracializes
legalise	legalised
befeather	befeathered
disinthrall	disinthralled
disparkle	disparkled
yip	yipped
emphasize	emphasizes
plasticize	plasticizing
ouse	oused
nudzh	nudzh
misknow	misknown
outdrive	outdrove
sauter	sautering
inrail	inrail
comix	comix
gloop	glooped
dispone	disponing
dissyllabize	dissyllabizes
tenon	tenons
arrect	arrected
taboo	taboo
forehalsen	forehalsened
praetermit	praetermitted
callus	callused
exorcize	exorcized
kart	kart
belock	belocked
talliate	talliated
readd	readded
metamorphosize	metamorphosizing
talliate	talliated
recenter	recenter
grangerize	grangerizes
baffle	baffle
animalize	animalized
aggravate	aggravating
humectate	humectate
plasticize	plasticizing
prechew	prechewed
leap	leaping
demosaic	demosaicing
midwife	midwiving
consolate	consolate
unwatch	unwatch
uplay	uplays
wanton	wantoning
layn	layn
besnowball	besnowball
chicane	chicaning
unpleat	unpleated
forclem	forclemming
rendezvous	rendezvouses
extrude	extruded
pullulate	pullulates
hock	hock
rip	ripped
metamorphosize	metamorphosizing
predetermine	predetermined
beheart	beheart
spraff	spraffing
dow	dought
lant	lants
reflip	reflipped
outfight	outfights
outfight	outfight
vaut	vauted
layn	layn
flawter	flawters
forworship	forworshipping
redound	redound
eternalise	eternalised
backtalk	backtalked
backtalk	backtalks
filst	filsted
defenestrate	defenestrated
scrounge	scrounged
hele	heled
mumble	mumbles
overslaugh	overslaughed
doorstep	doorstepped
luncheon	luncheon
disenthrall	disenthralled
dl	dled
dedecorate	dedecorated
maximalize	maximalized
compose	composes
primp	primped
corporatise	corporatising
rehabilitate	rehabilitated
distend	distending
bescribble	bescribble
woad	woaded
blotch	blotch
zag	zags
segmentalize	segmentalizes
twoc	twoc
mercerise	mercerising
skol	skolled
buz	buzzed
constringe	constringing
dow	dought
trouser	trouser
oversell	overselling
misapprehend	misapprehends
civilize	civilizes
interset	interset
interset	interset
slither	slithers
misimprint	misimprinted
canter	cantered
oversell	overselling
beblubber	beblubbers
tween	tweened
mislive	mislived
declaw	declawed
skeletalize	skeletalizes
skeletalize	skeletalizing
absterge	absterged
outnoise	outnoised
quash	quashing
ribroast	ribroasting
orthogonalise	orthogonalised
sabotage	sabotaged
ragequit	ragequits
encircle	encircle
meliorate	meliorate
corporatise	corporatising
scrape	scraped
downrush	downrushing
barnstorm	barnstorming
lazy	lazies
prerequire	prerequiring
dissyllabize	dissyllabizes
handwave	handwaved
upsweep	upswept
swear	swearing
overseal	oversealed
beheart	beheart
acronymise	acronymised
ul	ul
metastatize	metastatize
vaccinize	vaccinized
decypher	decyphering
waive	waived
waive	waiving
mensurate	mensurated
outslut	outslutting
vogue	voguing
prolate	prolated
hawm	hawmed
biwreye	biwreye
enstraiten	enstraitened
palter	paltered
decypher	decyphering
frag	fragged
chuff	chuffed
swelt	swelted
cnawvshawl	cnawvshawled
garter	gartering
harp	harping
deprioritize	deprioritizing
robotize	robotize
canter	cantered
recess	recess
disgorge	disgorging
readd	readded
dissonate	dissonating
manuscribe	manuscribed
regrout	regrouted
regrout	regrouting
forshorten	forshortening
underfund	underfunding
nuncupate	nuncupated
disavaunce	disavaunced
overdry	overdries
joust	jousting
swelt	swelted
garter	gartering
uncoil	uncoils
resmudge	resmudged
pun	puns
quash	quashing
passivate	passivates
anthologize	anthologizes
lojbanize	lojbanizing
scrape	scraped
bulwark	bulwarks
bulwark	bulwark
swipe	swipes
unqueen	unqueening
tense	tensed
meliorate	meliorate
outprize	outprizing
devenustate	devenustated
recarnify	recarnified
spit	spit
sermon	sermons
bemourn	bemourns
autophosphorylate	autophosphorylated
autophosphorylate	autophosphorylates
decompound	decompounded
empearl	empearled
atstand	atstanding
cobble	cobbled
mumble	mumbles
declaw	declawed
chicane	chicaning
addulce	addulced
flite	flite
tesselate	tesselated
ensphere	ensphere
oversay	oversaid
exfoliate	exfoliated
underserve	underserved
spit	spit
imbastardize	imbastardizes
palter	paltered
deprioritize	deprioritizing
deinstitutionalise	deinstitutionalise
foreshow	foreshown
overabound	overabounds
beride	beriding
sweal	sweals
terrestrialize	terrestrialize
tranch	tranches
prolate	prolated
cheat	cheating
underachieve	underachieved
feeble	feebled
sacerdotalize	sacerdotalized
tiller	tiller
dawb	dawbed
profiteer	profiteers
layn	layn
vogue	voguing
manace	manace
underprescribe	underprescribe
mistell	mistold
disaccommodate	disaccommodate
retromod	retromodded
incrust	incrusting
flawter	flawters
forseek	forseek
infrigidate	infrigidating
bouche	bouched
unvizard	unvizards
conveyance	conveyanced
synthesise	synthesise
endorse	endorsed
smirt	smirts
appreciate	appreciating
woad	woaded
overhend	overhent
joust	jousting
miscommunicate	miscommunicated
numericize	numericize
outquench	outquenching
intreat	intreats
gladden	gladden
interset	interset
interset	interset
halo	haloed
hydrate	hydrate
underpackage	underpackaged
accurize	accurized
vinegar	vinegars
declaw	declawed
drowse	drowsed
deinstitutionalise	deinstitutionalise
disgest	disgested
tween	tweened
visard	visards
defunctionalize	defunctionalizes
chicane	chicaning
overtrip	overtrip
zag	zags
fret	fret
undersee	undersees
counterindoctrinate	counterindoctrinated
overutilize	overutilizing
sauter	sautering
outfight	outfights
outfight	outfight
safari	safaris
demosaic	demosaicing
bepuzzle	bepuzzling
lojbanize	lojbanizing
dumfounder	dumfounder
exfoliate	exfoliated
writhe	writhes
belout	belouted
swound	swounding
knawvshawl	knawvshawled
handwave	handwaved
gutturize	gutturized
upcycle	upcycled
dow	dought'''

# Lexicon wFST
This wFST encodes the provided lexicon. It should take word forms as inputs at return lemmas as outputs.

If the following code compiles with no error, you've imported the data correctly - otherwise, make sure to click "run" for the data cell!

In [ ]:
#DO NOT CHANGE THIS

lexicon_fst = wfst4str.WeightedFst()
lexicon_fst.set_input_symbols(SYMS)
lexicon_fst.set_output_symbols(SYMS)
lexicon_fst.add_state()
lexicon_fst.set_start(0)
lexicon_fst.set_final(0, 0.0)

for line in INVOCAB.split("\n"):
  lemma, form = line.split("\t")
  form, lemma = form.lower(), lemma.lower()
  fst = lexicon_fst.to_linear_transducer(lemma, form)
  lexicon_fst.union_in_place(fst)

assert 'jammed' in lexicon_fst.apply('jam')
assert 'jamming' in lexicon_fst.apply('jam')

# Your wFSTs Here
Here is where you will work on your wFSTs (Morphotactic, E-Deletion, E-Insertion, Y-Replacement), and consonant doubling. This is the cell that you will copy and paste into the `lemmatizer.py` file when you submit to Gradescope.

Note that in order to actually construct the wFSTs from your string, you will have to run the cells for the corresponding wFST. For example, after filling in `MORPHOTACTIC`, you may test your wFST in the "Morphotactic wFST" cell below.

Before filling this out, make sure that you read the cells below which have additional code and explanations.

In [ ]:
# TODO: Consonants subject to the doubling rule
DCONSONANTS = ["p", "b", "m", "t", "n", "f", "g"]

# TODO: Morphotactic wFST
MORPHOTACTIC = '''
0 1 <oth> <oth> 0.0
1 2 # # 0.0
1 1 <oth> <oth> 0.0
2 0
1 3 <eps> ^ 0.0
3 4 <eps> i 0.0
4 5 <eps> n 0.0
5 6 <eps> g 0.0
6 7 # # 0.0
3 6 <eps> s 0.0
3 8 <eps> e 0.0
8 6 <eps> d 0.0
7 0
'''

# TODO: Silent E-Deletion wFST
E_DELETION = '''
0 1 <oth> <oth> 0.0
1 2 # # 0.0
1 1 <oth> <oth> 1.0
2 0

1 3 e e 0.5
3 4 ^ ^ 0.0
4 5 i i 0.0
5 6 n n 0.0
6 7 g g 0.0
7 8 # # 0.0
8 0

1 9 e e 0.5
9 10 ^ ^ 0.0
10 11 e e 0.0
11 12 d d 0.0
12 13 # # 0.0
13 0

1 14 e <eps> 0.0
14 15 ^ ^ 0.0
15 16 e e 0.0
16 17 d d 0.0
17 18 # # 0.0
18 0

1 19 e <eps> 0.0
19 20 ^ ^ 0.0
20 21 i i 0.0
21 22 n n 0.0
22 23 g g 0.0
23 24 # # 0.0
24 0

1 25 ^ ^ 1.0
25 26 e e 0.0
26 27 d d 0.0
27 28 # # 0.0
28 0

1 29 e e 1000.0
29 30 # # 0.0
30 0

1 31 ^ ^ 0.0
31 32 s s 0.0
32 33 # # 0.0
33 0

1 34 e e 0.0
34 35 ^ ^ 0.0
35 36 s s 0.0
36 37 # # 0.0
37 0

34 39 e e 0.0
39 40 ^ ^ 0.0
40 41 i i 0.0
41 42 n n 0.0
42 43 g g 0.0
43 44 # # 0.0
44 0

40 45 s s 0.0
45 46 # # 0.0
46 0

1 47 ^ ^ 1.0
47 48 e e 0.0
48 49 s s 0.0
49 50 # # 0.0
50 0

1 51 <eps> <eps> 0.0
51 51 <oth> <oth> 0.0
51 52 ^ ^ 1.0
52 53 <oth> <oth> 10.0
53 53 <oth> <oth> 10.0
53 54 # # 0.0
54 0

1 55 <oth> <oth> 0.0
55 55 <oth> <oth> 10.0
55 56 # # 0.0
56 0

1 57 e e 0.0
57 57 <oth> <oth> 10.0
57 58 # # 0.0
58 0
'''

# TODO: Silent E-Insertion wFST
E_INSERTION = '''
0 1 <oth> <oth> 0.0
1 2 # # 0.3
1 1 <oth> <oth> 1.0
2 0

1 3 <oth> <oth> 0.0
3 3 <oth> <oth> 0.0
3 4 s s 0.0
4 5 ^ ^ 0.0
5 6 <eps> e 0.0
6 7 s s 0.0
7 8 # # 0.0
8 0

3 4 x x 0.0

3 4 z z 0.0

4 9 h h 0.0
9 5 ^ ^ 0.0

3 10 c c 0.0
10 11 h h 0.0
11 5 ^ ^ 0.0

3 12 ^ ^ 0.0
12 12 <oth> <oth> 0.0
12 13 # # 0.0
13 0

1 14 <eps> <eps> 1.0
14 14 <oth> <oth> 1.0
14 15 ^ ^ 0.0
15 16 <oth> <oth> 1.0
16 16 <oth> <oth> 0.0
16 17 # # 0.0
17 0
'''

# TODO: Y-Replacement wFST
Y_REPLACEMENT = '''
0 1 <oth> <oth> 0.0
1 2 # # 0.3
1 1 <oth> <oth> 1.0
2 0

1 3 <oth> <oth> 2
3 3 <oth> <oth> 1
3 4 y i 0.0
4 5 ^ ^ 0.0
5 6 <eps> e 0.0
6 7 s s 0.0
7 8 # # 0.0
8 0

5 9 e e 0.0
9 10 d d 0.0
10 11 # # 0.0
11 0

1 12 <eps> <eps> 0.0
12 13 <v> <v> 0.0
13 17 y y 0.0

3 17 y y 0.5
17 18 ^ ^ 0.0
18 19 e e 0.0
19 20 d d 0.0
20 21 # # 0.0
21 0

18 22 s s 0.0
22 21 # # 0.0

1 23 <oth> <oth> 1
23 23 <oth> <oth> 1
23 24 ^ ^ 0.0
24 25 <oth> <oth> 1
25 25 <oth> <oth> 1
25 26 # # 0.0
26 0
'''

# Morphotactic wFST
This wFST enodes the morphotactics of English verbs. You must write it yourself. You should account for `-s`, `-ing`, and `-ed` as well as the bare form of verbs (no suffix). Your solution should have 14–17 lines. The input to the wFST would essentially by the stem with the word boundary such as `help#` and the expected outputs would be the stem itself `help#`, along with other forms of the verbs `help^ing#, help^s#, help^ed#`

**You will have to implement this wFST.**

In [ ]:
#DO NOT CHANGE THIS

tactic_fst = wfst4str.WeightedFst()
tactic_fst.set_input_symbols(SYMS)
tactic_fst.set_output_symbols(SYMS)
tactic_fst.add_state()
tactic_fst.set_start(0)
tactic_fst.populate_from_att(MORPHOTACTIC)
tactic_fst.explode_oth({'#', '^'})

# Testing code. You can add to these lines
assert tactic_fst.apply('lick#') == {'lick#', 'lick^ing#', 'lick^s#', 'lick^ed#'}
assert tactic_fst.apply('abscind#') == {'abscind#', 'abscind^s#', 'abscind^ing#', 'abscind^ed#'}
assert tactic_fst.apply('smite#') == {'smite#', 'smite^s#', 'smite^ing#', 'smite^ed#'}
assert tactic_fst.apply('fly#') == {'fly#', 'fly^s#', 'fly^ing#', 'fly^ed#'}



# Allomorphic wFST (Orthographic Rules)
You will write wFSTs for a few orthographic rules. Most of them are not complicated, but you should pay attention to **weights**, making certain that the paths that you want preferred have **lower** weights than other paths.

## Silent E-Deletion
This rule deletes the silent `e` at the end of stems when it occurs before the vowel-initial suffixes `-ing` and `-ed`, as in `"smite^ing#"` → `"smit^ing#"`. Remember to think about the stem cases, and cases of verbs where there isn't an `e` to be deleted

**You will have to implement this wFST.**

In [ ]:
# DO NOT CHANGE THIS

e_del_fst = wfst4str.WeightedFst()
e_del_fst.set_input_symbols(SYMS)
e_del_fst.set_output_symbols(SYMS)
e_del_fst.add_state()
e_del_fst.set_start(0)
e_del_fst.populate_from_att(E_DELETION)
e_del_fst.sub('<c>', CONSONANTS)
e_del_fst.sub('<v>', VOWELS)
e_del_fst.sub('<g>', GLIDES)
e_del_fst.explode_oth({'#', '^'})
print(e_del_fst.best_output('smite^ing#'))

# Testing code. You can add to these lines
assert e_del_fst.apply('smite^ing#') == {'smit^ing#', 'smite^ing#'}
assert e_del_fst.best_output('smite^ing#') == 'smit^ing#'
assert e_del_fst.best_output('agglutinat^ed#') == 'agglutinat^ed#'
assert e_del_fst.best_output('smite^ing#') == 'smit^ing#'
assert e_del_fst.best_output('smite#') == 'smite#'
assert e_del_fst.best_output('fox^s#') == 'fox^s#'
assert e_del_fst.best_output('smile^s#') == 'smile^s#'
assert e_del_fst.best_output('flee^ing#') == 'flee^ing#'
assert e_del_fst.best_output('agree^s#') == 'agree^s#'
assert e_del_fst.best_output('agree^ing#') == 'agree^ing#'

# DO NOT CHANGE THIS
e_del_fst2 = e_del_fst.invert()
# Testing code. You can add to these lines
assert e_del_fst2.best_output('kot^ing#') == 'kote^ing#'
assert e_del_fst2.best_output('agglutinat^ed#') == 'agglutinate^ed#'
assert e_del_fst2.best_output('fox^s#') == 'fox^s#'


smit^ing#


# Silent E-Insertion
This rule inserts a silent `e` between sibiliant fricatives and affricates (words ending in `ch,x,sh..` etc. mentioned earlier in the writeup) and the `-s` suffixes, as in `"fizz^s#"` → `"fizz^es#"`.

**You will have to implement this wFST.**

In [ ]:
# DO NOT CHANGE THIS

e_ins_fst = wfst4str.WeightedFst()
e_ins_fst.set_input_symbols(SYMS)
e_ins_fst.set_output_symbols(SYMS)
e_ins_fst.add_state()
e_ins_fst.set_start(0)
e_ins_fst.populate_from_att(E_INSERTION)
e_ins_fst.sub('<c>', CONSONANTS)
e_ins_fst.sub('<v>', VOWELS)
e_ins_fst.sub('<g>', GLIDES)
e_ins_fst.explode_oth({'^', '#'})

# Testing code. You can add to these lines
assert e_ins_fst.best_output('fox^s#') == 'fox^es#'
assert e_ins_fst.best_output('bash^s#') == 'bash^es#'
assert e_ins_fst.apply('bash^s#') == {'bash^es#', 'bash^s#'}
assert e_ins_fst.best_output('batch^s#') == 'batch^es#'
assert e_ins_fst.best_output('pass^s#') == 'pass^es#'
assert e_ins_fst.best_output('fox^ed#') == 'fox^ed#'
assert e_ins_fst.best_output('bash^ed#') == 'bash^ed#'
assert e_ins_fst.best_output('pass^ed#') == 'pass^ed#'
assert e_ins_fst.best_output('batch^ed#') == 'batch^ed#'
assert e_ins_fst.best_output('bash^s#') == 'bash^es#'
assert e_ins_fst.best_output('fish#') == 'fish#'
assert e_ins_fst.best_output('fish^ed#') == 'fish^ed#'
assert e_ins_fst.best_output('smile^ing#') == 'smile^ing#'
assert e_ins_fst.best_output('shoot^ing#') == 'shoot^ing#'


## Y-Replacement
Y-replacement changes 


1.   `y` into `i` before `-ed`. For example, `"try^ed#"` → `"tri^ed#"`
2.   `y` into `i` and inserts an `e` before  `-s`. For example, `"try^s#"` → `"tri^es#"`.

**You will have to implement this wFST.**

In [ ]:
# DO NOT CHANGE THIS

y_rep_fst = wfst4str.WeightedFst()
y_rep_fst.set_input_symbols(SYMS)
y_rep_fst.set_output_symbols(SYMS)
y_rep_fst.add_state()
y_rep_fst.set_start(0)
y_rep_fst.populate_from_att(Y_REPLACEMENT)
y_rep_fst.sub('<c>', CONSONANTS)
y_rep_fst.sub('<v>', VOWELS)
y_rep_fst.sub('<g>', GLIDES)
y_rep_fst.explode_oth({'#', '^'})

# Testing code. You can add to these lines

print(y_rep_fst.apply('fly^s#'))
print(y_rep_fst.apply('try^ed#'))
print(y_rep_fst.apply('play^s#'))
print(y_rep_fst.apply('play^s#'))
print(y_rep_fst.apply('squig#'))
print(y_rep_fst.apply('squig^ing#'))
print(y_rep_fst.apply('squig^s#'))
print(y_rep_fst.apply('squig^ed#'))
print(y_rep_fst.apply('cried#'))


assert y_rep_fst.best_output('fly^s#') == 'fli^es#'
assert y_rep_fst.best_output('try^ed#') == 'tri^ed#'
assert y_rep_fst.best_output('play^s#') == 'play^s#'
assert y_rep_fst.best_output('toy^ed#') == 'toy^ed#'


{'fly^s#', 'fli^es#'}
{'tri^ed#', 'try^ed#'}
{'plai^es#', 'play^s#'}
{'plai^es#', 'play^s#'}
{'squig#'}
{'squig^ing#'}
{'squig^s#'}
{'squig^ed#'}
{'cried#'}


## Consonant Doubling
This is the most complicated of the allomorphic rules. Because of its complexity, we have given you code to start with. Because it has transitions for each consonant in a set, we construct part of the wFST using python code.

The rule is subject to a lot of variability but the basic idea is as follows: a consonant drawn from a certain set is "doubled" if it is preceded by word-initial `C+V` and following by `ing` or `ed`. For example, `"tap^ing#"` → `"tapp^ing#"` and `"trap^ing#"` → `"trapp^ing#"`

You do not have to change the following cell content, but you should define the consonants which would be doubled in the variable `DCONSONANTS`. The assert statements are not exhaustive, which means you will have to think of more consonants which would be doubled.

In [ ]:
#DO NOT CHANGE THIS

doubling = '''
0 0 <c> <c> 0.0
0 0 <g> <g> 0.0
0 1 q q 0.0
0 2 <c> <c> 0.0
0 2 <g> <g> 0.0
0 12 <v> <v> 0.2
1 2 u u 0.0
2 3 <v> <v> 0.0
5 13 ^ ^ 0.0
13 6 i i 0.0
6 7 n n 0.0
7 8 g g 0.0
8 9 # # 0.0
9 0.0
13 10 e e 0.0
10 11 d d 0.0
11 9 # # 0.0
12 12 <v> <v> 0.0
12 12 <c> <c> 0.0
12 12 <g> <g> 0.0
12 12 ^ ^ 0.0
12 9 # # 1.0
'''

doubling_fst = wfst4str.WeightedFst()
doubling_fst.set_input_symbols(SYMS)
doubling_fst.set_output_symbols(SYMS)
doubling_fst.add_state()
doubling_fst.set_start(0)
doubling_fst.populate_from_att(doubling)
doubling_fst.sub('<c>', CONSONANTS)
doubling_fst.sub('<v>', VOWELS)
doubling_fst.sub('<g>', GLIDES)

for sym in DCONSONANTS:
    n = doubling_fst.add_state()
    doubling_fst.add_tr(3, n, sym, sym, 0.0)
    doubling_fst.add_tr(n, 5, '<eps>', sym, 0.0)

doubling_fst.explode_oth({'^', '#'})

# Testing code. You can add to these lines

assert doubling_fst.best_output('quip^ing#') == 'quipp^ing#'
assert doubling_fst.best_output('tap^ing#') == 'tapp^ing#'
assert doubling_fst.best_output('squat^ing#') == 'squatt^ing#'
assert doubling_fst.best_output('squig^ing#') == 'squigg^ing#'

In [ ]:
allomorphic_fst = doubling_fst.compose(e_ins_fst)
allomorphic_fst = allomorphic_fst.compose(e_del_fst)
allomorphic_fst.set_input_symbols(SYMS)
allomorphic_fst.set_output_symbols(SYMS)


assert allomorphic_fst.best_output('like^ing#') == 'lik^ing#'
assert allomorphic_fst.best_output('squig^ed#') == 'squigg^ed#'
assert allomorphic_fst.best_output('pitch^s#') == 'pitch^es#'

# Pulling It All Together

## Combine allomorphic FST

In [ ]:
guesser_fst = preproc_fst.compose(tactic_fst)
guesser_fst = guesser_fst.compose(y_rep_fst)
guesser_fst = guesser_fst.compose(allomorphic_fst)
guesser_fst = guesser_fst.compose(postproc_fst)
guesser_fst.set_input_symbols(SYMS)
guesser_fst.set_output_symbols(SYMS)

assert guesser_fst.apply('squig') == {'squig', 'squigs', 'squiging', 'squigging', 'squiged', 'squigged'}

## Invert Guesser

In [ ]:
guesser_fst.invert_in_place()

## Optimize Guesser

In [ ]:
guesser_fst.rm_epsilon_in_place()
guesser_fst.minimize_in_place()
guesser_fst.set_input_symbols(SYMS)
guesser_fst.set_output_symbols(SYMS)

## Test Guesser

In [ ]:
assert guesser_fst.best_output('squigged') == 'squig'
assert guesser_fst.best_output('cried') == 'cry'
assert guesser_fst.best_output('agglutinating') == 'agglutinate'
assert guesser_fst.best_output('abscinds') == 'abscind'

print(guesser_fst.apply('squigging'))
print(guesser_fst.best_output('squigging'))

{'squig', 'squigging', 'squigge', 'squigg'}
squigge


## Invert Lexicon

In [ ]:
lexicon_fst.invert_in_place()

## Optimize Lexicon

### Remove Epsilons

In [ ]:
lexicon_fst.rm_epsilon_in_place()

### Determinize

In [ ]:
lexicon_fst = lexicon_fst.determinize()

### Minimize

In [ ]:
lexicon_fst.minimize_in_place()
lexicon_fst.set_input_symbols(SYMS)
lexicon_fst.set_output_symbols(SYMS)

In [ ]:
assert lexicon_fst.best_output('shipping') == 'ship'
assert lexicon_fst.best_output('cried') == 'cry'
assert lexicon_fst.best_output('sang') == 'sing'
print(guesser_fst.apply('tissing'))
print(guesser_fst.best_output('tissing'))
# assert guesser_fst.best_output('tissing') == 'tess'


{'tiss', 'tissing', 'tisse'}
tisse


## Take Union of Lexicon and Guesser

In [ ]:
wfst = guesser_fst.union(lexicon_fst)
syms = wfst.get_input_symbols()

# Testing it
There are 3 different strings (defined previously in the data cell) which contain test data. There is `CARNEGLISH` (consisting of the made up nonce words), `INVOCAB`, and `OUTVOCAB`. 

When you submit to Gradescope, the autograder will grade you on your performance on these datasets. However, the data given are only 1/2 of the full data which will be tested on Gradescope, so be careful not to overfit. Refer to the grading scheme above for a sense of how well you should be doing on these tests (a lower accuracy is expected for `CARNEGLISH` and `OUTVOCAB`.)

In [ ]:
def batch_test(dataset):
    exact_match = 0
    count = 0
    for i, line in enumerate(dataset.split("\n")):
        if i == 1000: break
        lemma, form = line.split("\t")
        count += 1
        hypothesis = wfst.best_output(form)
        if hypothesis == lemma:
            exact_match += 1
        else:
            print(f'Guessed "{hypothesis}" instead of "{lemma}" for "{form}"')
    print(f'Exact match in {exact_match} out of {count} cases ({exact_match/count})')

In [ ]:
batch_test(CARNEGLISH)

Guessed "tisse" instead of "tiss" for "tissing"
Guessed "tis" instead of "tiss" for "tiss"
Guessed "exmogrifye" instead of "exmogrify" for "exmogrifying"
Guessed "blognicat" instead of "blognicate" for "blognicated"
Guessed "squigge" instead of "squig" for "squigging"
Guessed "squigge" instead of "squig" for "squigging"
Guessed "felexe" instead of "felex" for "felexes"
Guessed "reed" instead of "ree" for "reed"
Guessed "foothe" instead of "footh" for "foothing"
Guessed "foothe" instead of "footh" for "foothed"
Guessed "kurrye" instead of "kurry" for "kurrying"
Exact match in 37 out of 48 cases (0.7708333333333334)


In [ ]:
batch_test(INVOCAB)

Guessed "bear" instead of "bare" for "bare"
Guessed "bore" instead of "bear" for "bore"
Exact match in 998 out of 1000 cases (0.998)


In [ ]:
batch_test(OUTVOCAB)

Guessed "overheld" instead of "overhold" for "overheld"
Guessed "outdrove" instead of "outdrive" for "outdrove"
Guessed "handwav" instead of "handwave" for "handwaved"
Guessed "publiciz" instead of "publicize" for "publicizes"
Guessed "ringfenc" instead of "ringfence" for "ringfencing"
Guessed "handwav" instead of "handwave" for "handwaved"
Guessed "regl" instead of "regle" for "regling"
Guessed "glompe" instead of "glomp" for "glomping"
Guessed "shamanis" instead of "shamanise" for "shamanises"
Guessed "forclemm" instead of "forclem" for "forclemming"
Guessed "prefin" instead of "prefine" for "prefining"
Guessed "omnibusse" instead of "omnibus" for "omnibussed"
Guessed "unseculariz" instead of "unsecularize" for "unsecularizing"
Guessed "emphasiz" instead of "emphasize" for "emphasizes"
Guessed "miscommunicat" instead of "miscommunicate" for "miscommunicated"
Guessed "innerduc" instead of "innerduce" for "innerduced"
Guessed "sharde" instead of "shard" for "sharding"
Guessed "chid" in